In [3]:
"""
ASHOKA'S PATH - Complete Act 1
A text adventure game with quests and historical narrative
University of Aberdeen GameJam 2026
"""

import tkinter as tk
from tkinter import scrolledtext, font
from PIL import Image, ImageTk
import os

# ============================================================================
# GAME STATE
# ============================================================================

class GameState:
    def __init__(self):
        self.current_location = "palace_courtyard"
        self.inventory = []
        self.health = 100
        self.riddle_solved = False
        self.game_started = False
        
        # Quest tracking
        self.quests = {
            "beggar_quest": {
                "name": "Help the Blind Beggar",
                "status": "not_started",
                "stage": 0
            },
            "elephant_quest": {
                "name": "Prove Yourself to Airavata",
                "status": "not_started",
                "stage": 0
            },
            "soma_quest": {
                "name": "Obtain the Sacred Soma",
                "status": "not_started",
                "stage": 0
            },
            "cooking_quest": {
                "name": "Learn to Make Honey Cakes",
                "status": "not_started",
                "stage": 0
            },
            "cushion_quest": {
                "name": "Retrieve the Royal Cushion",
                "status": "not_started",
                "stage": 0
            }
        }
        
        # Story flags
        self.flags = {
            "has_golden_bowl": False,
            "vikram_paid": False,
            "elephant_test_started": False,
            "fed_elephant": False,
            "removed_thorn": False,
            "comforted_elephant": False,
            "learned_cooking_steps": False,
            "cooking_steps_correct": False,
            "all_items_collected": False,
            "victory_complete": False
        }
        
        # Items needed to win
        self.items_needed = [
            "golden_bowl",
            "imperial_elephant", 
            "soma",
            "honey_cakes",
            "royal_cushion"
        ]
        
    def has_item(self, item):
        return item in self.inventory
        
    def add_item(self, item):
        if item not in self.inventory:
            self.inventory.append(item)
            if item == "golden_bowl":
                self.flags["has_golden_bowl"] = True
            return True
        return False
            
    def check_victory_items(self):
        return all(item in self.inventory for item in self.items_needed)
    
    def update_quest(self, quest_name, status, stage=None):
        if quest_name in self.quests:
            self.quests[quest_name]["status"] = status
            if stage is not None:
                self.quests[quest_name]["stage"] = stage

# Global game state
game = GameState()

# ============================================================================
# LOCATIONS DATABASE
# ============================================================================

LOCATIONS = {
    "palace_courtyard": {
        "name": "Palace Courtyard",
        "description": """You stand in the grand courtyard of Pataliputra Palace. 
Marble pillars rise around you, carved with scenes of Mauryan victories. 
The morning sun glints off golden domes.

Courtiers whisper as they pass, avoiding your gaze.""",
        "image": "palace_courtyard.png",
        "exits": {
            "north": "fathers_chambers",
            "south": "mothers_chambers", 
            "east": "royal_stables",
            "west": "garden_pavilion"
        },
        "items": [],
        "npcs": []
    },
    
    "mothers_chambers": {
        "name": "Mother's Chambers",
        "description": """A gentle space filled with sandalwood incense and silk cushions.
Your mother, Dharma, sits by the window overlooking the gardens.
Oil lamps flicker, casting warm shadows on painted walls.""",
        "image": "mothers_chambers.png",
        "exits": {
            "north": "palace_courtyard",
            "west": "palace_kitchen"
        },
        "items": [],
        "npcs": ["mother_dharma"]
    },
    
    "palace_kitchen": {
        "name": "Palace Kitchen",
        "description": """Heat and noise fill this busy kitchen. Cooks shout orders while pots clang.

Old Amma Savitri kneads dough with weathered hands, muttering to herself.""",
        "image": "palace_kitchen.png",
        "exits": {
            "east": "mothers_chambers"
        },
        "items": [],
        "npcs": ["amma_savitri"]
    },
    
    "garden_pavilion": {
        "name": "Garden of the Golden Pavilion",
        "description": """An otherworldly space where heaven meets earth. Golden pavilion
rises in the center. Sacred lotus pools reflect the sky.

Ascetic Pingala-vatsajiva sits in meditation beneath a bodhi tree.""",
        "image": "garden_pavilion.png",
        "exits": {
            "east": "palace_courtyard"
        },
        "items": [],
        "npcs": ["ascetic_pingala"]
    },
    
    "royal_stables": {
        "name": "Royal Stables",
        "description": """The earthy smell of elephants and hay fills your nostrils.
Magnificent beasts rest in spacious enclosures.

Minister Radhagupta stands near the largest elephant - Airavata.""",
        "image": "royal_stables.png",
        "exits": {
            "west": "palace_courtyard",
            "south": "marketplace"
        },
        "items": [],
        "npcs": ["minister_radhagupta"]
    },
    
    "marketplace": {
        "name": "Marketplace",
        "description": """Chaos and color explode around you. Merchants shout their wares.
Spices perfume the air - cardamom, saffron, turmeric.

A blind beggar sits near a fountain, golden bowl in his lap.
Merchant Vikram haggles at his stall nearby.""",
        "image": "marketplace.png",
        "exits": {
            "north": "royal_stables",
            "west": "temple_district"
        },
        "items": [],
        "npcs": ["blind_beggar", "merchant_vikram"]
    },
    
    "temple_district": {
        "name": "Temple District",
        "description": """Incense smoke rises from a hundred shrines. Bells chime.
Monks in saffron robes chant ancient sutras.

Priest Acharya Mahinda tends the sacred fire.""",
        "image": "temple_district.png",
        "exits": {
            "east": "marketplace"
        },
        "items": [],
        "npcs": ["priest_mahinda"]
    },
    
    "fathers_chambers": {
        "name": "Emperor's Throne Room",
        "description": """A stern room of dark wood and military maps.
The black stone throne dominates.

Your father Bindusara sits at his desk, reviewing reports.""",
        "image": "fathers_chambers.png",
        "exits": {
            "south": "palace_courtyard"
        },
        "items": ["royal_cushion"],  # Available if 4+ items collected
        "npcs": ["emperor_bindusara"]
    }
}

# ============================================================================
# NPC DIALOGUES
# ============================================================================

NPCS = {
    "mother_dharma": {
        "name": "Mother Dharma",
        "dialogue": """My son... Your father has summoned the ascetic.

Go to the Garden. Face Pingala-vatsajiva's test.
I have seen your destiny - you will be greater than any emperor.

Remember - true power serves the people, not the self."""
    },
    
    "ascetic_pingala": {
        "name": "Ascetic Pingala-vatsajiva",
        "dialogue": """Prince Ashoka. I have examined your items.

You have proven your worth through compassion, wisdom, and humility.

Your father will not be pleased, but destiny cannot be denied.
You shall be the next emperor."""
    },
    
    "minister_radhagupta": {
        "name": "Minister Radhagupta",
        "dialogue": """Prince Ashoka. I have watched you. You see people as people.

This elephant, Airavata, will only accept a rider who proves worthy
through understanding, not force.

Would you like to attempt the test?"""
    },
    
    "blind_beggar": {
        "name": "Dhruva (Blind Beggar)",
        "dialogue": """Kind soul... This golden bowl was my father's.

Merchant Vikram owes my father twenty gold pieces but denies it.

If someone could convince him to pay... I would give them this bowl."""
    },
    
    "merchant_vikram": {
        "name": "Merchant Vikram",
        "dialogue": """That blind beggar sent you? Ha! I owe nothing.

Now leave before I call the guards."""
    },
    
    "priest_mahinda": {
        "name": "Acharya Mahinda",
        "dialogue": """Peace be upon you, seeker.

You seek soma? It requires a proper vessel and a pure heart.

Do you possess what is needed?"""
    },
    
    "amma_savitri": {
        "name": "Amma Savitri",
        "dialogue": """You want to LEARN to cook? First time a prince has asked!

Alright. Wash your hands. We're making honey cakes.
Can you follow steps EXACTLY?"""
    },
    
    "emperor_bindusara": {
        "name": "Emperor Bindusara",
        "dialogue": """What do you want, Ashoka? I am busy with matters of state.

Prove yourself worthy to the ascetic."""
    }
}

# ============================================================================
# GUI APPLICATION
# ============================================================================

class AshokaGameGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("🕉️ ASHOKA'S PATH - ACT 1")
        self.root.geometry("1000x750")
        self.root.configure(bg='#2b2b2b')
        
        # Fonts
        self.title_font = font.Font(family="Arial", size=14, weight="bold")
        self.text_font = font.Font(family="Courier", size=11)
        self.input_font = font.Font(family="Courier", size=11)
        
        # Setup UI
        self.setup_ui()
        
        # Show intro riddle
        self.show_opening_riddle()
        
    def setup_ui(self):
        """Create the UI layout"""
        
        # Top frame for image
        self.image_frame = tk.Frame(self.root, bg='#1a1a1a', height=220)
        self.image_frame.pack(fill=tk.X, padx=10, pady=5)
        self.image_frame.pack_propagate(False)
        
        self.image_label = tk.Label(self.image_frame, bg='#1a1a1a', fg='#4a9eff')
        self.image_label.pack(expand=True)
        
        # Location name label
        self.location_label = tk.Label(
            self.root, 
            text="",
            font=self.title_font,
            bg='#2b2b2b',
            fg='#4a9eff',
            pady=5
        )
        self.location_label.pack()
        
        # Text output area
        self.output_text = scrolledtext.ScrolledText(
            self.root,
            font=self.text_font,
            bg='#1a1a1a',
            fg='#e0e0e0',
            insertbackground='#4a9eff',
            wrap=tk.WORD,
            height=18
        )
        self.output_text.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Configure text tags
        self.output_text.tag_configure("green", foreground="#4ade80")
        self.output_text.tag_configure("yellow", foreground="#fbbf24")
        self.output_text.tag_configure("red", foreground="#f87171")
        self.output_text.tag_configure("cyan", foreground="#22d3ee")
        self.output_text.tag_configure("magenta", foreground="#e879f9")
        self.output_text.tag_configure("gold", foreground="#fbbf24")
        self.output_text.tag_configure("bold", font=self.title_font)
        
        # Bottom frame for input
        input_frame = tk.Frame(self.root, bg='#2b2b2b')
        input_frame.pack(fill=tk.X, padx=10, pady=5)
        
        tk.Label(
            input_frame, 
            text=">", 
            font=self.input_font,
            bg='#2b2b2b',
            fg='#4ade80'
        ).pack(side=tk.LEFT, padx=(0, 5))
        
        self.input_entry = tk.Entry(
            input_frame,
            font=self.input_font,
            bg='#1a1a1a',
            fg='#e0e0e0',
            insertbackground='#4a9eff'
        )
        self.input_entry.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.input_entry.bind('<Return>', self.process_command)
        
    def print(self, text, tag=None, newline=True):
        """Print text to output area"""
        self.output_text.insert(tk.END, text, tag)
        if newline:
            self.output_text.insert(tk.END, "\n")
        self.output_text.see(tk.END)
        
    def clear_output(self):
        """Clear the output text"""
        self.output_text.delete(1.0, tk.END)
        
    def load_image(self, image_filename):
        """Load and display an image"""
        if not image_filename:
            return
            
        image_path = f"images/{image_filename}"
        
        if os.path.exists(image_path):
            try:
                img = Image.open(image_path)
                max_width = 900
                max_height = 210
                img.thumbnail((max_width, max_height), Image.Resampling.LANCZOS)
                photo = ImageTk.PhotoImage(img)
                self.image_label.configure(image=photo, text="")
                self.image_label.image = photo
            except:
                display_name = image_filename.replace('.png', '').replace('_', ' ').title()
                self.image_label.configure(text=f"🖼️ {display_name}", image="")
        else:
            display_name = image_filename.replace('.png', '').replace('_', ' ').title()
            self.image_label.configure(text=f"🖼️ {display_name}", image="")
    
    def show_opening_riddle(self):
        """Show the opening riddle"""
        self.input_entry.config(state='disabled')
        
        self.print("="*80, "bold")
        self.print("🕉️  ASHOKA'S PATH - ACT 1: THE GREAT EMPIRE  🕉️", "bold")
        self.print("="*80, "bold")
        self.print("")
        self.print("You stand in the Palace Courtyard...")
        self.print("")
        self.print("An ancient ascetic appears, staff in hand.", "gold")
        self.print("")
        self.print("ASCETIC:", "bold")
        self.print("'Before you walk Ashoka's path, answer this:'")
        self.print("")
        self.print("  I am won through blood, yet bring no peace")
        self.print("  I am sought by many, yet poison few")
        self.print("  I sit on heads, yet crush many souls")
        self.print("")
        self.print("  What am I?")
        self.print("")
        self.print("  [1] The Crown", "cyan")
        self.print("  [2] A Sword", "cyan")
        self.print("  [3] Gold", "cyan")
        self.print("  [4] Power", "cyan")
        self.print("")
        self.print("Type the number (1-4):", "yellow")
        self.print("")
        
        self.input_entry.config(state='normal')
        self.input_entry.focus()
        
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.process_riddle_answer)
    
    def process_riddle_answer(self, event=None):
        """Process the riddle answer"""
        answer = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        
        if answer in ["1", "4"]:
            self.print(f"> {answer}", "cyan")
            self.print("")
            self.print("The ascetic nods slowly...", "green")
            self.print("")
            self.print("'Wisdom beyond your years. You may proceed...'", "green")
            self.print("")
            self.print("The ascetic vanishes. He waits for you in the Garden...", "gold")
            self.print("")
            self.print("="*80)
            self.print("")
            
            game.riddle_solved = True
            game.game_started = True
            
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_command)
            
            self.show_intro()
            
        else:
            self.print(f"> {answer}", "cyan")
            self.print("")
            self.print("The ascetic shakes his head sadly.", "red")
            self.print("'You see only the surface. Think deeper...'", "red")
            self.print("")
            self.print("Type the number (1-4):", "yellow")
            self.print("")
    
    def show_intro(self):
        """Show game intro"""
        self.print("YOUR JOURNEY BEGINS...", "bold")
        self.print("")
        self.print("You are Prince Ashoka. Find FIVE sacred items:", "yellow")
        self.print("  • A vessel of compassion", "magenta")
        self.print("  • A mount of strength", "magenta")
        self.print("  • A drink of the gods", "magenta")
        self.print("  • Food made with humility", "magenta")
        self.print("  • A seat earned through courage", "magenta")
        self.print("")
        self.print("Type 'help' for commands, 'quest' to track progress.", "cyan")
        self.print("")
        self.print("="*80)
        
        self.display_location()
        
    def display_location(self):
        """Display current location"""
        location = LOCATIONS[game.current_location]
        
        self.load_image(location.get("image"))
        self.location_label.configure(text=location["name"])
        
        self.print("")
        self.print(location["description"])
        self.print("")
        
        if location["exits"]:
            exits = ", ".join([f"{d.upper()}" for d in location["exits"].keys()])
            self.print(f"Exits: {exits}", "yellow")
            self.print("")
        
        if location["npcs"]:
            npc_names = []
            for npc_key in location["npcs"]:
                if npc_key in NPCS:
                    npc_names.append(NPCS[npc_key]['name'])
            if npc_names:
                self.print(f"People here: {', '.join(npc_names)}", "green")
                self.print("")
        
        # Show royal cushion if player has enough items
        if game.current_location == "fathers_chambers" and len(game.inventory) >= 4:
            if "royal_cushion" not in game.inventory:
                self.print("You see: Royal Cushion", "magenta")
                self.print("")
    
    def process_command(self, event=None):
        """Process user command"""
        if not game.game_started:
            return
            
        command = self.input_entry.get().lower().strip()
        self.input_entry.delete(0, tk.END)
        
        if not command:
            return
        
        self.print(f"> {command}", "cyan")
        self.print("")
        
        # Parse commands
        if command in ["north", "n"]:
            self.move("north")
        elif command in ["south", "s"]:
            self.move("south")
        elif command in ["east", "e"]:
            self.move("east")
        elif command in ["west", "w"]:
            self.move("west")
        elif command in ["look", "l"]:
            self.clear_output()
            self.display_location()
        elif command in ["inventory", "i", "inv"]:
            self.show_inventory()
        elif command in ["quest", "quests", "q"]:
            self.show_quests()
        elif command in ["health", "h"]:
            self.show_health()
        elif command in ["help", "?"]:
            self.show_help()
        elif command in ["quit", "exit"]:
            self.quit_game()
        elif command.startswith("pick ") or command.startswith("get ") or command.startswith("take "):
            item = command.split(maxsplit=1)[1] if len(command.split()) > 1 else ""
            self.pick_item(item)
        elif command.startswith("talk to "):
            npc_name = command.split(" to ")[1] if " to " in command else ""
            self.talk_to_npc(npc_name)
        elif "help vikram" in command:
            self.confront_vikram()
        elif "test elephant" in command:
            self.start_elephant_test()
        elif command.startswith("feed "):
            food = command.split(maxsplit=1)[1] if len(command.split()) > 1 else ""
            self.feed_elephant(food)
        elif "remove thorn" in command:
            self.remove_thorn()
        elif "comfort elephant" in command:
            self.comfort_elephant()
        elif "learn cooking" in command:
            self.learn_cooking()
        elif "make cakes" in command:
            self.make_cakes()
        elif "ritual" in command:
            self.soma_ritual()
        else:
            self.print("I don't understand. Type 'help' for commands.", "red")
        
        self.print("")
        
        # Check victory
        if game.check_victory_items() and not game.flags["all_items_collected"]:
            game.flags["all_items_collected"] = True
            self.print("="*70, "gold")
            self.print("✨ You collected all FIVE items! ✨", "gold")
            self.print("="*70, "gold")
            self.print("")
            self.print("Return to the GARDEN and talk to the Ascetic!", "yellow")
            self.print("")
    
    def move(self, direction):
        """Move in a direction"""
        location = LOCATIONS[game.current_location]
        
        if direction in location["exits"]:
            next_location = location["exits"][direction]
            
            # Susima death check
            if next_location == "fathers_chambers" and len(game.inventory) < 4:
                self.death_by_susima()
                return
            
            game.current_location = next_location
            self.print(f"You travel {direction}...", "cyan")
            self.print("")
            self.display_location()
        else:
            self.print("You can't go that way!", "red")
    
    def death_by_susima(self):
        """Death scene"""
        self.clear_output()
        self.load_image("fathers_chambers.png")
        
        self.print("")
        self.print("=" * 70, "red")
        self.print("YOU HAVE BEEN DISCOVERED!", "red")
        self.print("=" * 70, "red")
        self.print("")
        self.print("Your brother SUSIMA stands near the throne.")
        self.print("")
        self.print("SUSIMA: 'Ashoka! Competing for the throne? TRAITOR!'")
        self.print("")
        self.print("SUSIMA: 'GUARDS! Execute him!'")
        self.print("")
        self.print("=" * 70, "red")
        self.print("YOU DIED", "red")
        self.print("=" * 70, "red")
        self.print("")
        self.print("Collect 4+ items before entering the Throne Room.")
        self.print("")
        self.print("Restarting in 5 seconds...", "yellow")
        
        self.input_entry.config(state='disabled')
        self.root.after(5000, self.restart_game)
    
    def restart_game(self):
        """Restart after death"""
        global game
        game = GameState()
        self.clear_output()
        self.input_entry.config(state='normal')
        self.show_opening_riddle()
    
    def pick_item(self, item_name):
        """Pick up item"""
        item_name = item_name.lower().replace(" ", "_")
        
        # Royal cushion only available with 4+ items
        if item_name == "royal_cushion":
            if game.current_location != "fathers_chambers":
                self.print("Not here.", "red")
                return
            if len(game.inventory) < 4:
                self.print("You don't dare enter yet. Collect more items first.", "red")
                return
            if "royal_cushion" in game.inventory:
                self.print("You already have it.", "yellow")
                return
            
            self.print("You carefully take the royal cushion.")
            self.print("The guard pretends not to notice.")
            self.print("")
            game.add_item("royal_cushion")
            game.update_quest("cushion_quest", "completed", 1)
            self.print("✓ Acquired: Royal Cushion (5/5)", "gold")
            return
        
        self.print(f"There's no '{item_name}' to pick up here.", "red")
        self.print("Items are earned through quests!", "yellow")
    
    def talk_to_npc(self, npc_name):
        """Talk to NPC"""
        location = LOCATIONS[game.current_location]
        
        npc_key = None
        for npc in location["npcs"]:
            if npc_name.lower() in NPCS[npc]["name"].lower() or npc_name.lower() in npc.lower():
                npc_key = npc
                break
        
        if not npc_key:
            self.print("Nobody by that name here.", "red")
            return
        
        # Special interactions
        if npc_key == "blind_beggar":
            self.talk_to_beggar()
        elif npc_key == "merchant_vikram":
            self.talk_to_vikram()
        elif npc_key == "minister_radhagupta":
            self.talk_to_radhagupta()
        elif npc_key == "priest_mahinda":
            self.talk_to_priest()
        elif npc_key == "amma_savitri":
            self.talk_to_amma()
        elif npc_key == "ascetic_pingala":
            self.talk_to_ascetic()
        else:
            npc = NPCS[npc_key]
            self.print(f"{npc['name']} says:", "green")
            self.print(npc["dialogue"])
    
    def talk_to_beggar(self):
        """Beggar quest"""
        quest = game.quests["beggar_quest"]
        
        if quest["stage"] == 0:
            self.print("DHRUVA:", "green")
            self.print(NPCS["blind_beggar"]["dialogue"])
            game.update_quest("beggar_quest", "in_progress", 1)
            self.print("")
            self.print("(Try: 'talk to vikram' then 'help vikram')", "yellow")
            
        elif quest["stage"] == 1:
            self.print("DHRUVA: 'Have you spoken to Vikram yet?'")
            
        elif quest["stage"] == 2 and game.flags["vikram_paid"]:
            self.print("DHRUVA: 'You did it! Take my bowl. May it serve you well.'")
            self.print("")
            game.add_item("golden_bowl")
            game.update_quest("beggar_quest", "completed", 3)
            self.print("✓ Quest Complete: Help the Blind Beggar", "gold")
            self.print("✓ Acquired: Golden Bowl (1/5)", "gold")
            
        elif quest["stage"] == 3:
            self.print("DHRUVA: 'Thank you again, kind soul.'")
    
    def talk_to_vikram(self):
        """Vikram dialogue"""
        if game.quests["beggar_quest"]["stage"] < 1:
            self.print("VIKRAM: 'I don't have time for you.'")
            return
        
        self.print("VIKRAM: 'That beggar? I owe him nothing!'")
        self.print("")
        self.print("(Type 'help vikram' to investigate)", "yellow")
    
    def confront_vikram(self):
        """Investigate Vikram"""
        if game.quests["beggar_quest"]["stage"] != 1:
            self.print("Vikram ignores you.", "red")
            return
        
        self.print("You examine his ledgers...")
        self.print("There! A discrepancy. Twenty gold pieces unpaid.")
        self.print("")
        self.print("VIKRAM: (sweating) 'Fine! Take it!'")
        self.print("")
        game.flags["vikram_paid"] = True
        game.update_quest("beggar_quest", "in_progress", 2)
        self.print("Return to the beggar.", "yellow")
    
    def talk_to_radhagupta(self):
        """Radhagupta dialogue"""
        self.print("RADHAGUPTA:", "green")
        self.print(NPCS["minister_radhagupta"]["dialogue"])
        self.print("")
        
        if game.quests["elephant_quest"]["status"] == "not_started":
            self.print("(Type 'test elephant' to begin)", "yellow")
    
    def start_elephant_test(self):
        """Start elephant test"""
        if game.current_location != "royal_stables":
            self.print("Need to be at stables.", "red")
            return
        
        if game.quests["elephant_quest"]["status"] == "completed":
            self.print("Already earned Airavata's trust.", "yellow")
            return
        
        game.update_quest("elephant_quest", "in_progress", 1)
        game.flags["elephant_test_started"] = True
        
        self.print("PART 1: Feed Airavata", "yellow")
        self.print("Available: hay, mangoes, sugarcane")
        self.print("(Type: 'feed hay' etc.)", "yellow")
    
    def feed_elephant(self, food):
        """Feed elephant"""
        if not game.flags["elephant_test_started"]:
            self.print("Start test first.", "red")
            return
        
        if game.flags["fed_elephant"]:
            self.print("Already fed.", "yellow")
            return
        
        if food == "hay":
            self.print("Airavata accepts the humble hay gratefully.")
            self.print("")
            game.flags["fed_elephant"] = True
            game.update_quest("elephant_quest", "in_progress", 2)
            self.print("PART 2: Remove the thorn", "yellow")
            self.print("(Type: 'remove thorn')", "yellow")
        else:
            self.print(f"Wrong choice. Try 'feed hay'", "red")
    
    def remove_thorn(self):
        """Remove thorn"""
        if not game.flags["fed_elephant"]:
            self.print("Feed first.", "red")
            return
        
        if game.flags["removed_thorn"]:
            self.print("Already done.", "yellow")
            return
        
        self.print("You carefully remove the thorn.")
        self.print("Airavata rumbles gratefully.")
        self.print("")
        game.flags["removed_thorn"] = True
        game.update_quest("elephant_quest", "in_progress", 3)
        self.print("PART 3: Show compassion", "yellow")
        self.print("(Type: 'comfort elephant')", "yellow")
    
    def comfort_elephant(self):
        """Comfort elephant"""
        if not game.flags["removed_thorn"]:
            self.print("Remove thorn first.", "red")
            return
        
        self.print("You speak softly about loneliness and understanding...")
        self.print("Airavata touches your face with his trunk.")
        self.print("")
        game.flags["comforted_elephant"] = True
        game.add_item("imperial_elephant")
        game.update_quest("elephant_quest", "completed", 4)
        self.print("✓ Quest Complete: Prove Yourself to Airavata", "gold")
        self.print("✓ Acquired: Imperial Elephant (2/5)", "gold")
    
    def talk_to_priest(self):
        """Priest dialogue"""
        self.print("MAHINDA:", "green")
        self.print(NPCS["priest_mahinda"]["dialogue"])
        self.print("")
        
        if not game.flags["has_golden_bowl"]:
            self.print("MAHINDA: 'You need the golden bowl first.'")
        else:
            self.print("MAHINDA: 'You have the bowl! Ready for the ritual?'")
            self.print("")
            if game.quests["soma_quest"]["status"] == "not_started":
                self.print("(Type 'ritual')", "yellow")
    
    def soma_ritual(self):
        """Soma ritual"""
        if game.current_location != "temple_district":
            self.print("Need to be at temple.", "red")
            return
        
        if not game.flags["has_golden_bowl"]:
            self.print("Need golden bowl.", "red")
            return
        
        if game.quests["soma_quest"]["status"] == "completed":
            self.print("Already completed.", "yellow")
            return
        
        self.print("After the ritual and vision...")
        self.print("")
        game.add_item("soma")
        game.update_quest("soma_quest", "completed", 1)
        self.print("✓ Quest Complete: Obtain Sacred Soma", "gold")
        self.print("✓ Acquired: Soma (3/5)", "gold")
    
    def talk_to_amma(self):
        """Amma dialogue"""
        self.print("AMMA:", "green")
        self.print(NPCS["amma_savitri"]["dialogue"])
        self.print("")
        
        if game.quests["cooking_quest"]["status"] == "not_started":
            self.print("(Type 'learn cooking')", "yellow")
    
    def learn_cooking(self):
        """Learn cooking"""
        if game.current_location != "palace_kitchen":
            self.print("Need to be in kitchen.", "red")
            return
        
        if game.quests["cooking_quest"]["status"] == "completed":
            self.print("Already know how.", "yellow")
            return
        
        game.update_quest("cooking_quest", "in_progress", 1)
        game.flags["learned_cooking_steps"] = True
        
        self.print("AMMA: 'Listen carefully:'")
        self.print("  1. Mix flour and ghee")
        self.print("  2. Add honey slowly")
        self.print("  3. Fold in cardamom")
        self.print("  4. Shape into rounds")
        self.print("  5. Bake until golden")
        self.print("")
        self.print("AMMA: 'Repeat the order. Type: 1 2 3 4 5'")
        self.print("")
        
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', lambda e: self.cooking_test())
    
    def cooking_test(self):
        """Cooking memory test"""
        answer = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        
        correct = ["1 2 3 4 5", "12345", "1,2,3,4,5", "1, 2, 3, 4, 5"]
        
        if answer in correct:
            self.print(f"> {answer}", "cyan")
            self.print("")
            self.print("AMMA: 'Perfect! You listened.'")
            self.print("")
            game.flags["cooking_steps_correct"] = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_command)
            self.print("(Type 'make cakes')", "yellow")
            self.print("")
        else:
            self.print(f"> {answer}", "cyan")
            self.print("")
            self.print("AMMA: 'Wrong! Come back later.'")
            self.print("")
            game.update_quest("cooking_quest", "not_started", 0)
            game.flags["learned_cooking_steps"] = False
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_command)
    
    def make_cakes(self):
        """Make cakes"""
        if not game.flags["cooking_steps_correct"]:
            self.print("Learn steps first.", "red")
            return
        
        if game.quests["cooking_quest"]["status"] == "completed":
            self.print("Already made them.", "yellow")
            return
        
        self.print("You make the honey cakes with care...")
        self.print("AMMA: 'Well done. Remember this humility.'")
        self.print("")
        game.add_item("honey_cakes")
        game.update_quest("cooking_quest", "completed", 2)
        self.print("✓ Quest Complete: Learn to Make Honey Cakes", "gold")
        self.print("✓ Acquired: Honey Cakes (4/5)", "gold")
    
    def talk_to_ascetic(self):
        """Victory check"""
        if not game.check_victory_items():
            self.print("ASCETIC: 'Return with all five items.'")
            return
        
        if game.flags["victory_complete"]:
            self.print("ASCETIC: 'Your journey continues...'")
            return
        
        self.victory_sequence()
    
    def victory_sequence(self):
        """Victory!"""
        game.flags["victory_complete"] = True
        
        self.clear_output()
        self.load_image("coronation.png")
        
        self.print("")
        self.print("=" * 70, "gold")
        self.print("ACT 1 COMPLETE", "bold")
        self.print("=" * 70, "gold")
        self.print("")
        self.print("You are crowned EMPEROR ASHOKA.")
        self.print("")
        self.print("The empire is yours. But what will you do with it?")
        self.print("")
        self.print("=" * 70, "gold")
        self.print("TO BE CONTINUED IN ACT 2...", "bold")
        self.print("=" * 70, "gold")
        self.print("")
        self.print("Thank you for playing!", "cyan")
        self.print("")
    
    def show_inventory(self):
        """Show inventory"""
        if not game.inventory:
            self.print("Inventory empty.", "yellow")
        else:
            self.print("📦 INVENTORY:", "bold")
            for i, item in enumerate(game.inventory, 1):
                self.print(f"  {i}. {item.replace('_', ' ').title()}", "green")
            self.print(f"\n{len(game.inventory)}/{len(game.items_needed)} items", "yellow")
    
    def show_quests(self):
        """Show quests"""
        self.print("📜 QUESTS:", "bold")
        self.print("")
        
        for quest in game.quests.values():
            symbol = "✓" if quest["status"] == "completed" else "◆" if quest["status"] == "in_progress" else "○"
            color = "green" if quest["status"] == "completed" else "yellow" if quest["status"] == "in_progress" else "cyan"
            self.print(f"{symbol} {quest['name']}", color)
        
        self.print("")
    
    def show_health(self):
        """Show health"""
        self.print(f"❤️  Health: {game.health}/100", "green")
    
    def show_help(self):
        """Show help"""
        self.print("=" * 60, "bold")
        self.print("COMMANDS", "bold")
        self.print("=" * 60, "bold")
        self.print("")
        self.print("Movement: north/south/east/west (n/s/e/w)")
        self.print("Actions: look, inventory, quest, talk to <npc>")
        self.print("Quests: help vikram, test elephant, feed <food>,")
        self.print("        remove thorn, comfort elephant, ritual,")
        self.print("        learn cooking, make cakes")
        self.print("Other: help, quit")
        self.print("")
        self.print("=" * 60, "bold")
    
    def quit_game(self):
        """Quit"""
        self.print("Thanks for playing!", "yellow")
        self.root.after(2000, self.root.quit)

# ============================================================================
# MAIN
# ============================================================================

def main():
    root = tk.Tk()
    app = AshokaGameGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()

## Version 2 


In [1]:
"""
ASHOKA'S PATH - COMPLETE ACT 1
All quests, riddles, and adventures fully implemented
University of Aberdeen GameJam 2026
"""

import tkinter as tk
from tkinter import scrolledtext, font
from PIL import Image, ImageTk
import os

# Game State with ALL quest tracking
class GameState:
    def __init__(self):
        self.current_location = "palace_courtyard"
        self.inventory = []
        self.health = 100
        self.riddle_solved = False
        self.game_started = False
        
        self.quests = {
            "beggar": {"status": "not_started", "stage": 0},
            "elephant": {"status": "not_started", "stage": 0},
            "soma": {"status": "not_started", "stage": 0},
            "cooking": {"status": "not_started", "stage": 0},
            "cushion": {"status": "not_started", "stage": 0}
        }
        
        self.flags = {
            "has_golden_bowl": False,
            "vikram_paid": False,
            "elephant_started": False,
            "fed_elephant": False,
            "removed_thorn": False,
            "comforted_elephant": False,
            "learned_steps": False,
            "steps_correct": False,
            "victory": False
        }
        
        self.items_needed = ["golden_bowl", "imperial_elephant", "soma", "honey_cakes", "royal_cushion"]
        
    def has_item(self, item):
        return item in self.inventory
        
    def add_item(self, item):
        if item not in self.inventory:
            self.inventory.append(item)
            if item == "golden_bowl":
                self.flags["has_golden_bowl"] = True
            return True
        return False
            
    def check_victory(self):
        return all(item in self.inventory for item in self.items_needed)

game = GameState()

# Locations
LOCATIONS = {
    "palace_courtyard": {
        "name": "Palace Courtyard",
        "description": "Grand courtyard with marble pillars. Morning sun glints off golden domes.",
        "image": "palace_courtyard.png",
        "exits": {"north": "fathers_chambers", "south": "mothers_chambers", "east": "royal_stables", "west": "garden_pavilion"},
        "npcs": []
    },
    "mothers_chambers": {
        "name": "Mother's Chambers",
        "description": "Gentle space with sandalwood incense. Your mother sits by the window.",
        "image": "mothers_chambers.png",
        "exits": {"north": "palace_courtyard", "west": "palace_kitchen"},
        "npcs": ["mother"]
    },
    "palace_kitchen": {
        "name": "Palace Kitchen",
        "description": "Busy kitchen filled with heat and noise. Amma Savitri kneads dough.",
        "image": "palace_kitchen.png",
        "exits": {"east": "mothers_chambers"},
        "npcs": ["amma"]
    },
    "garden_pavilion": {
        "name": "Garden of the Golden Pavilion",
        "description": "Otherworldly space. Golden pavilion touches clouds. Ascetic meditates beneath bodhi tree.",
        "image": "garden_pavilion.png",
        "exits": {"east": "palace_courtyard"},
        "npcs": ["ascetic"]
    },
    "royal_stables": {
        "name": "Royal Stables",
        "description": "Earthy smell of elephants. Minister Radhagupta stands near Airavata.",
        "image": "royal_stables.png",
        "exits": {"west": "palace_courtyard", "south": "marketplace"},
        "npcs": ["radhagupta"]
    },
    "marketplace": {
        "name": "Marketplace",
        "description": "Chaos and color. Blind beggar sits with golden bowl. Vikram at his stall.",
        "image": "marketplace.png",
        "exits": {"north": "royal_stables", "west": "temple_district"},
        "npcs": ["beggar", "vikram"]
    },
    "temple_district": {
        "name": "Temple District",
        "description": "Incense smoke rises. Priest Mahinda tends the sacred fire.",
        "image": "temple_district.png",
        "exits": {"east": "marketplace"},
        "npcs": ["priest"]
    },
    "fathers_chambers": {
        "name": "Throne Room",
        "description": "Stern room with black stone throne. Your father reviews reports.",
        "image": "fathers_chambers.png",
        "exits": {"south": "palace_courtyard"},
        "npcs": ["emperor"]
    }
}

# GUI Application
class AshokaGame:
    def __init__(self, root):
        self.root = root
        self.root.title("🕉️ ASHOKA'S PATH - ACT 1")
        self.root.geometry("1000x750")
        self.root.configure(bg='#2b2b2b')
        
        self.title_font = font.Font(family="Arial", size=14, weight="bold")
        self.text_font = font.Font(family="Courier", size=11)
        
        self.setup_ui()
        self.show_riddle()
        
    def setup_ui(self):
        # Image frame
        self.image_frame = tk.Frame(self.root, bg='#1a1a1a', height=220)
        self.image_frame.pack(fill=tk.X, padx=10, pady=5)
        self.image_frame.pack_propagate(False)
        self.image_label = tk.Label(self.image_frame, bg='#1a1a1a', fg='#4a9eff')
        self.image_label.pack(expand=True)
        
        # Location label
        self.location_label = tk.Label(self.root, text="", font=self.title_font, bg='#2b2b2b', fg='#4a9eff', pady=5)
        self.location_label.pack()
        
        # Text output
        self.output = scrolledtext.ScrolledText(self.root, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', 
                                                insertbackground='#4a9eff', wrap=tk.WORD, height=18)
        self.output.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Tags
        self.output.tag_configure("green", foreground="#4ade80")
        self.output.tag_configure("yellow", foreground="#fbbf24")
        self.output.tag_configure("red", foreground="#f87171")
        self.output.tag_configure("cyan", foreground="#22d3ee")
        self.output.tag_configure("magenta", foreground="#e879f9")
        self.output.tag_configure("gold", foreground="#fbbf24")
        self.output.tag_configure("bold", font=self.title_font)
        
        # Input
        input_frame = tk.Frame(self.root, bg='#2b2b2b')
        input_frame.pack(fill=tk.X, padx=10, pady=5)
        tk.Label(input_frame, text=">", font=self.text_font, bg='#2b2b2b', fg='#4ade80').pack(side=tk.LEFT, padx=(0, 5))
        self.input_entry = tk.Entry(input_frame, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', insertbackground='#4a9eff')
        self.input_entry.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.input_entry.bind('<Return>', self.process_cmd)
        
    def p(self, text, tag=None):
        self.output.insert(tk.END, text + "\n", tag)
        self.output.see(tk.END)
        
    def clear(self):
        self.output.delete(1.0, tk.END)
        
    def load_img(self, img):
        if not img:
            return
        path = f"images/{img}"
        if os.path.exists(path):
            try:
                im = Image.open(path)
                im.thumbnail((900, 210), Image.Resampling.LANCZOS)
                photo = ImageTk.PhotoImage(im)
                self.image_label.configure(image=photo, text="")
                self.image_label.image = photo
            except:
                self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
        else:
            self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
    
    def show_riddle(self):
        self.input_entry.config(state='disabled')
        self.p("="*80, "bold")
        self.p("🕉️  ASHOKA'S PATH - ACT 1  🕉️", "bold")
        self.p("="*80, "bold")
        self.p("")
        self.p("An ancient ascetic appears...", "gold")
        self.p("")
        self.p("'Before you walk Ashoka's path, answer this:'")
        self.p("  I am won through blood, yet bring no peace")
        self.p("  I am sought by many, yet poison few")
        self.p("  I sit on heads, yet crush many souls")
        self.p("")
        self.p("  [1] The Crown", "cyan")
        self.p("  [2] A Sword", "cyan")
        self.p("  [3] Gold", "cyan")
        self.p("  [4] Power", "cyan")
        self.p("")
        self.input_entry.config(state='normal')
        self.input_entry.focus()
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_riddle)
    
    def check_riddle(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        if ans in ["1", "4"]:
            self.p(f"> {ans}", "cyan")
            self.p("")
            self.p("'Wisdom beyond your years. You may proceed...'", "green")
            self.p("")
            game.riddle_solved = True
            game.game_started = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.start_game()
        else:
            self.p(f"> {ans}", "cyan")
            self.p("'Think deeper...'", "red")
            self.p("")
    
    def start_game(self):
        self.p("="*80)
        self.p("Find FIVE sacred items:", "yellow")
        self.p("  • Vessel of compassion", "magenta")
        self.p("  • Mount of strength", "magenta")
        self.p("  • Drink of gods", "magenta")
        self.p("  • Food with humility", "magenta")
        self.p("  • Seat through courage", "magenta")
        self.p("")
        self.p("Type 'help' for commands, 'quest' for progress", "cyan")
        self.p("="*80)
        self.show_loc()
    
    def show_loc(self):
        loc = LOCATIONS[game.current_location]
        self.load_img(loc["image"])
        self.location_label.configure(text=loc["name"])
        self.p("")
        self.p(loc["description"])
        self.p("")
        exits = ", ".join([d.upper() for d in loc["exits"]])
        self.p(f"Exits: {exits}", "yellow")
        self.p("")
        if loc["npcs"]:
            self.p(f"People: {', '.join(loc['npcs'])}", "green")
            self.p("")
        # Show cushion if ready
        if game.current_location == "fathers_chambers" and len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
            self.p("You see: Royal Cushion", "magenta")
            self.p("")
    
    def process_cmd(self, e=None):
        if not game.game_started:
            return
        cmd = self.input_entry.get().lower().strip()
        self.input_entry.delete(0, tk.END)
        if not cmd:
            return
        self.p(f"> {cmd}", "cyan")
        self.p("")
        
        if cmd in ["n","north"]: self.move("north")
        elif cmd in ["s","south"]: self.move("south")
        elif cmd in ["e","east"]: self.move("east")
        elif cmd in ["w","west"]: self.move("west")
        elif cmd in ["l","look"]: self.clear(); self.show_loc()
        elif cmd in ["i","inv","inventory"]: self.show_inv()
        elif cmd in ["q","quest","quests"]: self.show_quest()
        elif cmd in ["h","health"]: self.p(f"❤️ {game.health}/100", "green")
        elif cmd in ["?","help"]: self.show_help()
        elif cmd in ["quit","exit"]: self.p("Thanks for playing!","yellow"); self.root.after(2000, self.root.quit)
        elif cmd.startswith("talk "): self.talk(cmd.replace("talk to ","").replace("talk ",""))
        elif cmd == "help vikram": self.help_vikram()
        elif cmd == "test elephant": self.test_elephant()
        elif cmd.startswith("feed "): self.feed(cmd.split()[1])
        elif cmd == "remove thorn": self.remove_thorn()
        elif cmd == "comfort elephant": self.comfort()
        elif cmd == "ritual": self.ritual()
        elif cmd == "learn cooking": self.learn_cook()
        elif cmd == "make cakes": self.make_cakes()
        elif cmd.startswith("pick ") or cmd.startswith("get ") or cmd.startswith("take "): self.pick(cmd.split(maxsplit=1)[1])
        else: self.p("Unknown command. Type 'help'", "red")
        
        self.p("")
        if game.check_victory() and not game.flags["victory"]:
            self.p("="*70, "gold")
            self.p("✨ ALL FIVE ITEMS COLLECTED! ✨", "gold")
            self.p("="*70, "gold")
            self.p("Return to GARDEN and talk to ascetic!", "yellow")
            self.p("")
    
    def move(self, dir):
        loc = LOCATIONS[game.current_location]
        if dir in loc["exits"]:
            next_loc = loc["exits"][dir]
            # Susima death check
            if next_loc == "fathers_chambers" and len(game.inventory) < 4:
                self.die()
                return
            game.current_location = next_loc
            self.p(f"You travel {dir}...", "cyan")
            self.p("")
            self.show_loc()
        else:
            self.p("Can't go that way!", "red")
    
    def die(self):
        self.clear()
        self.p("="*70, "red")
        self.p("DISCOVERED BY SUSIMA!", "red")
        self.p("="*70, "red")
        self.p("Your brother accuses you of treason!")
        self.p("YOU DIED", "red")
        self.p("Collect 4+ items first. Restarting...", "yellow")
        self.input_entry.config(state='disabled')
        self.root.after(5000, self.restart)
    
    def restart(self):
        global game
        game = GameState()
        self.clear()
        self.input_entry.config(state='normal')
        self.show_riddle()
    
    def pick(self, item):
        item = item.lower().replace(" ", "_")
        if item == "royal_cushion":
            if game.current_location != "fathers_chambers":
                self.p("Not here", "red")
                return
            if len(game.inventory) < 4:
                self.p("Too dangerous. Need 4+ items first", "red")
                return
            if game.has_item("royal_cushion"):
                self.p("Already have it", "yellow")
                return
            game.add_item("royal_cushion")
            game.quests["cushion"]["status"] = "completed"
            self.p("✓ Acquired: Royal Cushion (5/5)", "gold")
            return
        self.p("Items earned through quests!", "yellow")
    
    def talk(self, npc):
        loc = LOCATIONS[game.current_location]
        if npc not in loc["npcs"]:
            self.p("Nobody by that name here", "red")
            return
        
        if npc == "beggar": self.talk_beggar()
        elif npc == "vikram": self.talk_vikram()
        elif npc == "radhagupta": self.talk_radha()
        elif npc == "priest": self.talk_priest()
        elif npc == "amma": self.talk_amma()
        elif npc == "ascetic": self.talk_ascetic()
        else: self.p(f"{npc} says: Hello", "green")
    
    def talk_beggar(self):
        q = game.quests["beggar"]
        if q["stage"] == 0:
            self.p("DHRUVA:", "green")
            self.p("Vikram owes me 20 gold. Help me get it?")
            self.p("(Try: 'talk to vikram' then 'help vikram')", "yellow")
            q["status"] = "in_progress"
            q["stage"] = 1
        elif q["stage"] == 1:
            self.p("DHRUVA: Spoken to Vikram yet?")
        elif q["stage"] == 2 and game.flags["vikram_paid"]:
            self.p("DHRUVA: Thank you! Take my bowl.")
            game.add_item("golden_bowl")
            q["status"] = "completed"
            q["stage"] = 3
            self.p("✓ Quest Complete! Golden Bowl (1/5)", "gold")
        else:
            self.p("DHRUVA: Thanks again.")
    
    def talk_vikram(self):
        if game.quests["beggar"]["stage"] < 1:
            self.p("VIKRAM: I'm busy.")
            return
        self.p("VIKRAM: I owe him nothing!")
        self.p("(Try: 'help vikram')", "yellow")
    
    def help_vikram(self):
        if game.quests["beggar"]["stage"] != 1:
            return
        self.p("You check his ledgers... Found it!")
        self.p("VIKRAM: Fine, take it!")
        game.flags["vikram_paid"] = True
        game.quests["beggar"]["stage"] = 2
        self.p("Return to beggar", "yellow")
    
    def talk_radha(self):
        self.p("RADHAGUPTA:", "green")
        self.p("Test yourself with Airavata?")
        self.p("(Try: 'test elephant')", "yellow")
    
    def test_elephant(self):
        if game.current_location != "royal_stables":
            return
        if game.quests["elephant"]["status"] == "completed":
            self.p("Already done", "yellow")
            return
        game.quests["elephant"]["status"] = "in_progress"
        game.flags["elephant_started"] = True
        self.p("PART 1: Feed Airavata")
        self.p("Options: hay, mangoes, sugarcane")
        self.p("(Try: 'feed hay')", "yellow")
    
    def feed(self, food):
        if not game.flags["elephant_started"] or game.flags["fed_elephant"]:
            return
        if food == "hay":
            self.p("Airavata accepts gratefully")
            game.flags["fed_elephant"] = True
            self.p("PART 2: Remove thorn", "yellow")
            self.p("(Try: 'remove thorn')", "yellow")
        else:
            self.p("Wrong choice", "red")
    
    def remove_thorn(self):
        if not game.flags["fed_elephant"] or game.flags["removed_thorn"]:
            return
        self.p("You carefully remove the thorn...")
        game.flags["removed_thorn"] = True
        self.p("PART 3: Comfort elephant", "yellow")
        self.p("(Try: 'comfort elephant')", "yellow")
    
    def comfort(self):
        if not game.flags["removed_thorn"]:
            return
        self.p("You speak of loneliness...")
        self.p("Airavata touches your face")
        game.flags["comforted_elephant"] = True
        game.add_item("imperial_elephant")
        game.quests["elephant"]["status"] = "completed"
        self.p("✓ Quest Complete! Imperial Elephant (2/5)", "gold")
    
    def talk_priest(self):
        self.p("MAHINDA:", "green")
        if not game.flags["has_golden_bowl"]:
            self.p("Need golden bowl first")
        else:
            self.p("Ready for ritual?")
            self.p("(Try: 'ritual')", "yellow")
    
    def ritual(self):
        if game.current_location != "temple_district":
            return
        if not game.flags["has_golden_bowl"]:
            return
        if game.quests["soma"]["status"] == "completed":
            return
        self.p("You complete the sacred ritual...")
        self.p("Vision: Two paths - war and peace")
        game.add_item("soma")
        game.quests["soma"]["status"] = "completed"
        self.p("✓ Quest Complete! Soma (3/5)", "gold")
    
    def talk_amma(self):
        self.p("AMMA:", "green")
        self.p("Want to learn cooking?")
        self.p("(Try: 'learn cooking')", "yellow")
    
    def learn_cook(self):
        if game.current_location != "palace_kitchen":
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        game.quests["cooking"]["status"] = "in_progress"
        game.flags["learned_steps"] = True
        self.p("AMMA: 5 steps:")
        self.p("1. Mix flour and ghee")
        self.p("2. Add honey slowly")
        self.p("3. Fold in cardamom")
        self.p("4. Shape into rounds")
        self.p("5. Bake until golden")
        self.p("")
        self.p("Repeat the order. Type: 1 2 3 4 5")
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_steps)
    
    def check_steps(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        correct = ["1 2 3 4 5", "12345", "1,2,3,4,5", "1, 2, 3, 4, 5"]
        if ans in correct:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Perfect!")
            game.flags["steps_correct"] = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.p("(Try: 'make cakes')", "yellow")
            self.p("")
        else:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Wrong! Come back.", "red")
            game.quests["cooking"]["status"] = "not_started"
            game.flags["learned_steps"] = False
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
    
    def make_cakes(self):
        if not game.flags["steps_correct"]:
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        self.p("You make honey cakes with care...")
        self.p("AMMA: Remember this humility")
        game.add_item("honey_cakes")
        game.quests["cooking"]["status"] = "completed"
        self.p("✓ Quest Complete! Honey Cakes (4/5)", "gold")
    
    def talk_ascetic(self):
        if not game.check_victory():
            self.p("ASCETIC: Return with all five items")
            return
        if game.flags["victory"]:
            return
        self.victory()
    
    def victory(self):
        game.flags["victory"] = True
        self.clear()
        self.load_img("coronation.png")
        self.p("")
        self.p("="*70, "gold")
        self.p("ACT 1 COMPLETE", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("You are crowned EMPEROR ASHOKA")
        self.p("")
        self.p("="*70, "gold")
        self.p("TO BE CONTINUED IN ACT 2...", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("Thank you for playing!", "cyan")
    
    def show_inv(self):
        if not game.inventory:
            self.p("Empty", "yellow")
        else:
            self.p("📦 INVENTORY:", "bold")
            for i, item in enumerate(game.inventory, 1):
                self.p(f"  {i}. {item.replace('_', ' ').title()}", "green")
            self.p(f"{len(game.inventory)}/5 items", "yellow")
    
    def show_quest(self):
        self.p("📜 QUESTS:", "bold")
        for name, q in game.quests.items():
            s = "✓" if q["status"] == "completed" else "◆" if q["status"] == "in_progress" else "○"
            c = "green" if q["status"] == "completed" else "yellow" if q["status"] == "in_progress" else "cyan"
            self.p(f"{s} {name}", c)
    
    def show_help(self):
        self.p("COMMANDS:", "bold")
        self.p("Movement: n/s/e/w, look")
        self.p("Actions: talk to <npc>, inventory, quest")
        self.p("Quests: help vikram, test elephant, feed <food>,")
        self.p("        remove thorn, comfort elephant, ritual,")
        self.p("        learn cooking, make cakes, pick cushion")
        self.p("Other: help, quit")

def main():
    root = tk.Tk()
    app = AshokaGame(root)
    root.mainloop()

if __name__ == "__main__":
    main()

# version 3 

In [2]:
"""
ASHOKA'S PATH - COMPLETE ACT 1
All quests, riddles, and adventures fully implemented
University of Aberdeen GameJam 2026
"""

import tkinter as tk
from tkinter import scrolledtext, font
from PIL import Image, ImageTk
import os

# Game State with ALL quest tracking
class GameState:
    def __init__(self):
        self.current_location = "palace_courtyard"
        self.inventory = []
        self.health = 100
        self.riddle_solved = False
        self.game_started = False
        
        self.quests = {
            "beggar": {"status": "not_started", "stage": 0},
            "elephant": {"status": "not_started", "stage": 0},
            "soma": {"status": "not_started", "stage": 0},
            "cooking": {"status": "not_started", "stage": 0},
            "cushion": {"status": "not_started", "stage": 0}
        }
        
        self.flags = {
            "has_golden_bowl": False,
            "vikram_paid": False,
            "elephant_started": False,
            "fed_elephant": False,
            "removed_thorn": False,
            "comforted_elephant": False,
            "learned_steps": False,
            "steps_correct": False,
            "victory": False
        }
        
        self.items_needed = ["golden_bowl", "imperial_elephant", "soma", "honey_cakes", "royal_cushion"]
        
    def has_item(self, item):
        return item in self.inventory
        
    def add_item(self, item):
        if item not in self.inventory:
            self.inventory.append(item)
            if item == "golden_bowl":
                self.flags["has_golden_bowl"] = True
            return True
        return False
            
    def check_victory(self):
        return all(item in self.inventory for item in self.items_needed)

game = GameState()

# Locations
LOCATIONS = {
    "palace_courtyard": {
        "name": "Palace Courtyard",
        "description": "Grand courtyard with marble pillars. Morning sun glints off golden domes.",
        "image": "palace_courtyard.png",
        "exits": {"north": "fathers_chambers", "south": "mothers_chambers", "east": "royal_stables", "west": "garden_pavilion"},
        "npcs": []
    },
    "mothers_chambers": {
        "name": "Mother's Chambers",
        "description": "Gentle space with sandalwood incense. Your mother sits by the window.",
        "image": "mothers_chambers.png",
        "exits": {"north": "palace_courtyard", "west": "palace_kitchen"},
        "npcs": ["mother"]
    },
    "palace_kitchen": {
        "name": "Palace Kitchen",
        "description": "Busy kitchen filled with heat and noise. Amma Savitri kneads dough.",
        "image": "palace_kitchen.png",
        "exits": {"east": "mothers_chambers"},
        "npcs": ["amma"]
    },
    "garden_pavilion": {
        "name": "Garden of the Golden Pavilion",
        "description": "Otherworldly space. Golden pavilion touches clouds. Ascetic meditates beneath bodhi tree.",
        "image": "garden_pavilion.png",
        "exits": {"east": "palace_courtyard"},
        "npcs": ["ascetic"]
    },
    "royal_stables": {
        "name": "Royal Stables",
        "description": "Earthy smell of elephants. Minister Radhagupta stands near Airavata.",
        "image": "royal_stables.png",
        "exits": {"west": "palace_courtyard", "south": "marketplace"},
        "npcs": ["radhagupta"]
    },
    "marketplace": {
        "name": "Marketplace",
        "description": "Chaos and color. Blind beggar sits with golden bowl. Vikram at his stall.",
        "image": "marketplace.png",
        "exits": {"north": "royal_stables", "west": "temple_district"},
        "npcs": ["beggar", "vikram"]
    },
    "temple_district": {
        "name": "Temple District",
        "description": "Incense smoke rises. Priest Mahinda tends the sacred fire.",
        "image": "temple_district.png",
        "exits": {"east": "marketplace"},
        "npcs": ["priest"]
    },
    "fathers_chambers": {
        "name": "Throne Room",
        "description": "Stern room with black stone throne. Your father reviews reports.",
        "image": "fathers_chambers.png",
        "exits": {"south": "palace_courtyard"},
        "npcs": ["emperor"]
    }
}

# GUI Application
class AshokaGame:
    def __init__(self, root):
        self.root = root
        self.root.title("🕉️ ASHOKA'S PATH - ACT 1")
        self.root.geometry("1000x750")
        self.root.configure(bg='#2b2b2b')
        
        self.title_font = font.Font(family="Arial", size=14, weight="bold")
        self.text_font = font.Font(family="Courier", size=11)
        
        self.setup_ui()
        self.show_riddle()
        
    def setup_ui(self):
        # Image frame
        self.image_frame = tk.Frame(self.root, bg='#1a1a1a', height=220)
        self.image_frame.pack(fill=tk.X, padx=10, pady=5)
        self.image_frame.pack_propagate(False)
        self.image_label = tk.Label(self.image_frame, bg='#1a1a1a', fg='#4a9eff')
        self.image_label.pack(expand=True)
        
        # Location label
        self.location_label = tk.Label(self.root, text="", font=self.title_font, bg='#2b2b2b', fg='#4a9eff', pady=5)
        self.location_label.pack()
        
        # Text output
        self.output = scrolledtext.ScrolledText(self.root, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', 
                                                insertbackground='#4a9eff', wrap=tk.WORD, height=18)
        self.output.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Tags
        self.output.tag_configure("green", foreground="#4ade80")
        self.output.tag_configure("yellow", foreground="#fbbf24")
        self.output.tag_configure("red", foreground="#f87171")
        self.output.tag_configure("cyan", foreground="#22d3ee")
        self.output.tag_configure("magenta", foreground="#e879f9")
        self.output.tag_configure("gold", foreground="#fbbf24")
        self.output.tag_configure("bold", font=self.title_font)
        
        # Input
        input_frame = tk.Frame(self.root, bg='#2b2b2b')
        input_frame.pack(fill=tk.X, padx=10, pady=5)
        tk.Label(input_frame, text=">", font=self.text_font, bg='#2b2b2b', fg='#4ade80').pack(side=tk.LEFT, padx=(0, 5))
        self.input_entry = tk.Entry(input_frame, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', insertbackground='#4a9eff')
        self.input_entry.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.input_entry.bind('<Return>', self.process_cmd)
        
    def p(self, text, tag=None):
        self.output.insert(tk.END, text + "\n", tag)
        self.output.see(tk.END)
        
    def clear(self):
        self.output.delete(1.0, tk.END)
        
    def load_img(self, img):
        if not img:
            return
        path = f"images/{img}"
        if os.path.exists(path):
            try:
                im = Image.open(path)
                im.thumbnail((900, 210), Image.Resampling.LANCZOS)
                photo = ImageTk.PhotoImage(im)
                self.image_label.configure(image=photo, text="")
                self.image_label.image = photo
            except:
                self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
        else:
            self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
    
    def show_riddle(self):
        self.input_entry.config(state='disabled')
        self.p("="*80, "bold")
        self.p("🕉️  ASHOKA'S PATH - ACT 1  🕉️", "bold")
        self.p("="*80, "bold")
        self.p("")
        self.p("An ancient ascetic appears...", "gold")
        self.p("")
        self.p("'Before you walk Ashoka's path, answer this:'")
        self.p("  I am won through blood, yet bring no peace")
        self.p("  I am sought by many, yet poison few")
        self.p("  I sit on heads, yet crush many souls")
        self.p("")
        self.p("  [1] The Crown", "cyan")
        self.p("  [2] A Sword", "cyan")
        self.p("  [3] Gold", "cyan")
        self.p("  [4] Power", "cyan")
        self.p("")
        self.input_entry.config(state='normal')
        self.input_entry.focus()
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_riddle)
    
    def check_riddle(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        if ans in ["1", "4"]:
            self.p(f"> {ans}", "cyan")
            self.p("")
            self.p("'Wisdom beyond your years. You may proceed...'", "green")
            self.p("")
            game.riddle_solved = True
            game.game_started = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.start_game()
        else:
            self.p(f"> {ans}", "cyan")
            self.p("'Think deeper...'", "red")
            self.p("")
    
    def start_game(self):
        self.p("="*80)
        self.p("Find FIVE sacred items:", "yellow")
        self.p("  • Vessel of compassion", "magenta")
        self.p("  • Mount of strength", "magenta")
        self.p("  • Drink of gods", "magenta")
        self.p("  • Food with humility", "magenta")
        self.p("  • Seat through courage", "magenta")
        self.p("")
        self.p("Type 'help' for commands, 'map' to see the palace", "cyan")
        self.p("Type 'quest' to track progress", "cyan")
        self.p("="*80)
        self.show_loc()
    
    def show_loc(self):
        loc = LOCATIONS[game.current_location]
        self.load_img(loc["image"])
        self.location_label.configure(text=loc["name"])
        self.p("")
        self.p(loc["description"])
        self.p("")
        exits = ", ".join([d.upper() for d in loc["exits"]])
        self.p(f"Exits: {exits}", "yellow")
        self.p("")
        if loc["npcs"]:
            self.p(f"People: {', '.join(loc['npcs'])}", "green")
            self.p("")
        # Show cushion if ready
        if game.current_location == "fathers_chambers" and len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
            self.p("You see: Royal Cushion", "magenta")
            self.p("")
    
    def process_cmd(self, e=None):
        if not game.game_started:
            return
        cmd = self.input_entry.get().lower().strip()
        self.input_entry.delete(0, tk.END)
        if not cmd:
            return
        self.p(f"> {cmd}", "cyan")
        self.p("")
        
        if cmd in ["n","north"]: self.move("north")
        elif cmd in ["s","south"]: self.move("south")
        elif cmd in ["e","east"]: self.move("east")
        elif cmd in ["w","west"]: self.move("west")
        elif cmd in ["l","look"]: self.clear(); self.show_loc()
        elif cmd in ["i","inv","inventory"]: self.show_inv()
        elif cmd in ["q","quest","quests"]: self.show_quest()
        elif cmd in ["h","health"]: self.p(f"❤️ {game.health}/100", "green")
        elif cmd in ["?","help"]: self.show_help()
        elif cmd in ["m","map"]: self.show_map()
        elif cmd in ["quit","exit"]: self.p("Thanks for playing!","yellow"); self.root.after(2000, self.root.quit)
        elif cmd.startswith("talk "): self.talk(cmd.replace("talk to ","").replace("talk ",""))
        elif cmd == "help vikram": self.help_vikram()
        elif cmd == "test elephant": self.test_elephant()
        elif cmd.startswith("feed "): self.feed(cmd.split()[1])
        elif cmd == "remove thorn": self.remove_thorn()
        elif cmd == "comfort elephant": self.comfort()
        elif cmd == "ritual": self.ritual()
        elif cmd == "learn cooking": self.learn_cook()
        elif cmd == "make cakes": self.make_cakes()
        elif cmd.startswith("pick ") or cmd.startswith("get ") or cmd.startswith("take "): self.pick(cmd.split(maxsplit=1)[1])
        else: self.p("Unknown command. Type 'help'", "red")
        
        self.p("")
        if game.check_victory() and not game.flags["victory"]:
            self.p("="*70, "gold")
            self.p("✨ ALL FIVE ITEMS COLLECTED! ✨", "gold")
            self.p("="*70, "gold")
            self.p("Return to GARDEN and talk to ascetic!", "yellow")
            self.p("")
    
    def move(self, dir):
        loc = LOCATIONS[game.current_location]
        if dir in loc["exits"]:
            next_loc = loc["exits"][dir]
            # Susima death check
            if next_loc == "fathers_chambers" and len(game.inventory) < 4:
                self.die()
                return
            game.current_location = next_loc
            self.p(f"You travel {dir}...", "cyan")
            self.p("")
            self.show_loc()
        else:
            self.p("Can't go that way!", "red")
    
    def die(self):
        self.clear()
        self.p("="*70, "red")
        self.p("DISCOVERED BY SUSIMA!", "red")
        self.p("="*70, "red")
        self.p("Your brother accuses you of treason!")
        self.p("YOU DIED", "red")
        self.p("Collect 4+ items first. Restarting...", "yellow")
        self.input_entry.config(state='disabled')
        self.root.after(5000, self.restart)
    
    def restart(self):
        global game
        game = GameState()
        self.clear()
        self.input_entry.config(state='normal')
        self.show_riddle()
    
    def pick(self, item):
        item = item.lower().replace(" ", "_")
        if item == "royal_cushion":
            if game.current_location != "fathers_chambers":
                self.p("Not here", "red")
                return
            if len(game.inventory) < 4:
                self.p("Too dangerous. Need 4+ items first", "red")
                return
            if game.has_item("royal_cushion"):
                self.p("Already have it", "yellow")
                return
            game.add_item("royal_cushion")
            game.quests["cushion"]["status"] = "completed"
            self.p("✓ Acquired: Royal Cushion (5/5)", "gold")
            return
        self.p("Items earned through quests!", "yellow")
    
    def talk(self, npc):
        loc = LOCATIONS[game.current_location]
        if npc not in loc["npcs"]:
            self.p("Nobody by that name here", "red")
            return
        
        if npc == "beggar": self.talk_beggar()
        elif npc == "vikram": self.talk_vikram()
        elif npc == "radhagupta": self.talk_radha()
        elif npc == "priest": self.talk_priest()
        elif npc == "amma": self.talk_amma()
        elif npc == "ascetic": self.talk_ascetic()
        else: self.p(f"{npc} says: Hello", "green")
    
    def talk_beggar(self):
        q = game.quests["beggar"]
        if q["stage"] == 0:
            self.p("DHRUVA:", "green")
            self.p("Vikram owes me 20 gold. Help me get it?")
            self.p("(Try: 'talk to vikram' then 'help vikram')", "yellow")
            q["status"] = "in_progress"
            q["stage"] = 1
        elif q["stage"] == 1:
            self.p("DHRUVA: Spoken to Vikram yet?")
        elif q["stage"] == 2 and game.flags["vikram_paid"]:
            self.p("DHRUVA: Thank you! Take my bowl.")
            game.add_item("golden_bowl")
            q["status"] = "completed"
            q["stage"] = 3
            self.p("✓ Quest Complete! Golden Bowl (1/5)", "gold")
        else:
            self.p("DHRUVA: Thanks again.")
    
    def talk_vikram(self):
        if game.quests["beggar"]["stage"] < 1:
            self.p("VIKRAM: I'm busy.")
            return
        self.p("VIKRAM: I owe him nothing!")
        self.p("(Try: 'help vikram')", "yellow")
    
    def help_vikram(self):
        if game.quests["beggar"]["stage"] != 1:
            return
        self.p("You check his ledgers... Found it!")
        self.p("VIKRAM: Fine, take it!")
        game.flags["vikram_paid"] = True
        game.quests["beggar"]["stage"] = 2
        self.p("Return to beggar", "yellow")
    
    def talk_radha(self):
        self.p("RADHAGUPTA:", "green")
        self.p("Test yourself with Airavata?")
        self.p("(Try: 'test elephant')", "yellow")
    
    def test_elephant(self):
        if game.current_location != "royal_stables":
            return
        if game.quests["elephant"]["status"] == "completed":
            self.p("Already done", "yellow")
            return
        game.quests["elephant"]["status"] = "in_progress"
        game.flags["elephant_started"] = True
        self.p("PART 1: Feed Airavata")
        self.p("Options: hay, mangoes, sugarcane")
        self.p("(Try: 'feed hay')", "yellow")
    
    def feed(self, food):
        if not game.flags["elephant_started"] or game.flags["fed_elephant"]:
            return
        if food == "hay":
            self.p("Airavata accepts gratefully")
            game.flags["fed_elephant"] = True
            self.p("PART 2: Remove thorn", "yellow")
            self.p("(Try: 'remove thorn')", "yellow")
        else:
            self.p("Wrong choice", "red")
    
    def remove_thorn(self):
        if not game.flags["fed_elephant"] or game.flags["removed_thorn"]:
            return
        self.p("You carefully remove the thorn...")
        game.flags["removed_thorn"] = True
        self.p("PART 3: Comfort elephant", "yellow")
        self.p("(Try: 'comfort elephant')", "yellow")
    
    def comfort(self):
        if not game.flags["removed_thorn"]:
            return
        self.p("You speak of loneliness...")
        self.p("Airavata touches your face")
        game.flags["comforted_elephant"] = True
        game.add_item("imperial_elephant")
        game.quests["elephant"]["status"] = "completed"
        self.p("✓ Quest Complete! Imperial Elephant (2/5)", "gold")
    
    def talk_priest(self):
        self.p("MAHINDA:", "green")
        if not game.flags["has_golden_bowl"]:
            self.p("Need golden bowl first")
        else:
            self.p("Ready for ritual?")
            self.p("(Try: 'ritual')", "yellow")
    
    def ritual(self):
        if game.current_location != "temple_district":
            return
        if not game.flags["has_golden_bowl"]:
            return
        if game.quests["soma"]["status"] == "completed":
            return
        self.p("You complete the sacred ritual...")
        self.p("Vision: Two paths - war and peace")
        game.add_item("soma")
        game.quests["soma"]["status"] = "completed"
        self.p("✓ Quest Complete! Soma (3/5)", "gold")
    
    def talk_amma(self):
        self.p("AMMA:", "green")
        self.p("Want to learn cooking?")
        self.p("(Try: 'learn cooking')", "yellow")
    
    def learn_cook(self):
        if game.current_location != "palace_kitchen":
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        game.quests["cooking"]["status"] = "in_progress"
        game.flags["learned_steps"] = True
        self.p("AMMA: 5 steps:")
        self.p("1. Mix flour and ghee")
        self.p("2. Add honey slowly")
        self.p("3. Fold in cardamom")
        self.p("4. Shape into rounds")
        self.p("5. Bake until golden")
        self.p("")
        self.p("Repeat the order. Type: 1 2 3 4 5")
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_steps)
    
    def check_steps(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        correct = ["1 2 3 4 5", "12345", "1,2,3,4,5", "1, 2, 3, 4, 5"]
        if ans in correct:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Perfect!")
            game.flags["steps_correct"] = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.p("(Try: 'make cakes')", "yellow")
            self.p("")
        else:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Wrong! Come back.", "red")
            game.quests["cooking"]["status"] = "not_started"
            game.flags["learned_steps"] = False
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
    
    def make_cakes(self):
        if not game.flags["steps_correct"]:
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        self.p("You make honey cakes with care...")
        self.p("AMMA: Remember this humility")
        game.add_item("honey_cakes")
        game.quests["cooking"]["status"] = "completed"
        self.p("✓ Quest Complete! Honey Cakes (4/5)", "gold")
    
    def talk_ascetic(self):
        if not game.check_victory():
            self.p("ASCETIC: Return with all five items")
            return
        if game.flags["victory"]:
            return
        self.victory()
    
    def victory(self):
        game.flags["victory"] = True
        self.clear()
        self.load_img("coronation.png")
        self.p("")
        self.p("="*70, "gold")
        self.p("ACT 1 COMPLETE", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("You are crowned EMPEROR ASHOKA")
        self.p("")
        self.p("="*70, "gold")
        self.p("TO BE CONTINUED IN ACT 2...", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("Thank you for playing!", "cyan")
    
    def show_inv(self):
        if not game.inventory:
            self.p("Empty", "yellow")
        else:
            self.p("📦 INVENTORY:", "bold")
            for i, item in enumerate(game.inventory, 1):
                self.p(f"  {i}. {item.replace('_', ' ').title()}", "green")
            self.p(f"{len(game.inventory)}/5 items", "yellow")
    
    def show_quest(self):
        self.p("📜 QUESTS:", "bold")
        for name, q in game.quests.items():
            s = "✓" if q["status"] == "completed" else "◆" if q["status"] == "in_progress" else "○"
            c = "green" if q["status"] == "completed" else "yellow" if q["status"] == "in_progress" else "cyan"
            self.p(f"{s} {name}", c)
    
    def show_map(self):
        """Display game map with current location highlighted"""
        # Load map image if available
        self.load_img("map.png")
        
        self.p("")
        self.p("="*60, "bold")
        self.p("🗺️  PATALIPUTRA PALACE MAP", "bold")
        self.p("="*60, "bold")
        self.p("")
        
        # Create map with current location highlighted
        current = game.current_location
        
        # Map structure
        locations_map = {
            "fathers_chambers": "    [Throne Room]",
            "palace_courtyard_n": "          |",
            "palace_courtyard": "  [Palace Courtyard]",
            "mothers_chambers": "          |",
            "palace_kitchen": "[Kitchen]--+",
            "garden_pavilion": "[Garden]--+--[Stables]",
            "marketplace": "              |",
            "temple_district": "     [Temple]--[Market]"
        }
        
        # Print map with highlights
        self.p("         NORTH", "cyan")
        self.p("")
        
        # Throne Room
        if current == "fathers_chambers":
            self.p("    >>> [THRONE ROOM] <<<", "gold")
        else:
            self.p("        [Throne Room]", "cyan")
        
        self.p("              |", "cyan")
        
        # Palace Courtyard (center)
        if current == "palace_courtyard":
            self.p("                            ", "cyan")
            self.p("  [Garden]------>>> [YOU ARE HERE] <<<------[Stables]", "gold")
            self.p("                            ", "cyan")
        else:
            self.p("                            ", "cyan")
            self.p("  [Garden]--------[Courtyard]--------[Stables]", "cyan")
            self.p("                            ", "cyan")
        
        self.p("                      |                    |", "cyan")
        
        # Bottom row
        left = "              " if current != "garden_pavilion" else ">>> "
        middle_m = " [Mother's] " if current != "mothers_chambers" else " >>> [YOU] <<< "
        middle_k = "[Kitchen]" if current != "palace_kitchen" else ">>>[YOU]<<<"
        right_m = "[Market]" if current != "marketplace" else ">>>[YOU]<<<"
        right_t = "[Temple]" if current != "temple_district" else ">>>[YOU]<<<"
        right_s = " " if current != "royal_stables" else ""
        
        self.p(f"{left}          {middle_m}            {right_s}", "cyan" if current not in ["mothers_chambers", "royal_stables"] else "gold")
        self.p(f"                      |", "cyan")
        self.p(f"                 {middle_k}", "cyan" if current != "palace_kitchen" else "gold")
        self.p("")
        self.p(f"                                   {right_t}---{right_m}", "cyan" if current not in ["marketplace", "temple_district"] else "gold")
        
        self.p("")
        self.p(f"Current Location: {LOCATIONS[current]['name']}", "yellow")
        self.p("")
        self.p("="*60, "bold")
        self.p("")
    
    def show_help(self):
        self.p("COMMANDS:", "bold")
        self.p("Movement: n/s/e/w, look, map")
        self.p("Actions: talk to <npc>, inventory, quest")
        self.p("Quests: help vikram, test elephant, feed <food>,")
        self.p("        remove thorn, comfort elephant, ritual,")
        self.p("        learn cooking, make cakes, pick cushion")
        self.p("Other: help, quit")

def main():
    root = tk.Tk()
    app = AshokaGame(root)
    root.mainloop()

if __name__ == "__main__":
    main()

# version 4 with battle system 

In [1]:
"""
ASHOKA'S PATH - COMPLETE ACT 1
All quests, riddles, and adventures fully implemented
University of Aberdeen GameJam 2026
"""

import tkinter as tk
from tkinter import scrolledtext, font
from PIL import Image, ImageTk
import os

# Game State with ALL quest tracking
class GameState:
    def __init__(self):
        self.current_location = "palace_courtyard"
        self.inventory = []
        self.health = 100
        self.max_health = 100
        self.riddle_solved = False
        self.game_started = False
        
        # Combat state
        self.in_combat = False
        self.combat_enemy = None
        self.combat_enemy_hp = 0
        self.combat_enemy_max_hp = 0
        self.combat_heals_remaining = 3
        self.combat_defending = False
        self.combat_training_complete = False
        
        self.quests = {
            "beggar": {"status": "not_started", "stage": 0},
            "elephant": {"status": "not_started", "stage": 0},
            "soma": {"status": "not_started", "stage": 0},
            "cooking": {"status": "not_started", "stage": 0},
            "cushion": {"status": "not_started", "stage": 0}
        }
        
        self.flags = {
            "has_golden_bowl": False,
            "vikram_paid": False,
            "elephant_started": False,
            "fed_elephant": False,
            "removed_thorn": False,
            "comforted_elephant": False,
            "learned_steps": False,
            "steps_correct": False,
            "victory": False
        }
        
        self.items_needed = ["golden_bowl", "imperial_elephant", "soma", "honey_cakes", "royal_cushion"]
        
    def has_item(self, item):
        return item in self.inventory
        
    def add_item(self, item):
        if item not in self.inventory:
            self.inventory.append(item)
            if item == "golden_bowl":
                self.flags["has_golden_bowl"] = True
            return True
        return False
            
    def check_victory(self):
        return all(item in self.inventory for item in self.items_needed)

game = GameState()

# Locations
LOCATIONS = {
    "palace_courtyard": {
        "name": "Palace Courtyard",
        "description": "Grand courtyard with marble pillars. Morning sun glints off golden domes.",
        "image": "palace_courtyard.png",
        "exits": {"north": "fathers_chambers", "south": "mothers_chambers", "east": "royal_stables", "west": "garden_pavilion"},
        "npcs": []
    },
    "mothers_chambers": {
        "name": "Mother's Chambers",
        "description": "Gentle space with sandalwood incense. Your mother sits by the window.",
        "image": "mothers_chambers.png",
        "exits": {"north": "palace_courtyard", "west": "palace_kitchen"},
        "npcs": ["mother"]
    },
    "palace_kitchen": {
        "name": "Palace Kitchen",
        "description": "Busy kitchen filled with heat and noise. Amma Savitri kneads dough.",
        "image": "palace_kitchen.png",
        "exits": {"east": "mothers_chambers"},
        "npcs": ["amma"]
    },
    "garden_pavilion": {
        "name": "Garden of the Golden Pavilion",
        "description": "Otherworldly space. Golden pavilion touches clouds. Ascetic meditates beneath bodhi tree.",
        "image": "garden_pavilion.png",
        "exits": {"east": "palace_courtyard"},
        "npcs": ["ascetic"]
    },
    "royal_stables": {
        "name": "Royal Stables",
        "description": "Earthy smell of elephants. Minister Radhagupta stands near Airavata.",
        "image": "royal_stables.png",
        "exits": {"west": "palace_courtyard", "south": "marketplace"},
        "npcs": ["radhagupta"]
    },
    "marketplace": {
        "name": "Marketplace",
        "description": "Chaos and color. Blind beggar sits with golden bowl. Vikram at his stall.",
        "image": "marketplace.png",
        "exits": {"north": "royal_stables", "west": "temple_district"},
        "npcs": ["beggar", "vikram"]
    },
    "temple_district": {
        "name": "Temple District",
        "description": "Incense smoke rises. Priest Mahinda tends the sacred fire.",
        "image": "temple_district.png",
        "exits": {"east": "marketplace"},
        "npcs": ["priest"]
    },
    "fathers_chambers": {
        "name": "Throne Room",
        "description": "Stern room with black stone throne. Your father reviews reports.",
        "image": "fathers_chambers.png",
        "exits": {"south": "palace_courtyard"},
        "npcs": ["emperor"]
    }
}

# GUI Application
class AshokaGame:
    def __init__(self, root):
        self.root = root
        self.root.title("🕉️ ASHOKA'S PATH - ACT 1")
        self.root.geometry("1000x750")
        self.root.configure(bg='#2b2b2b')
        
        self.title_font = font.Font(family="Arial", size=14, weight="bold")
        self.text_font = font.Font(family="Courier", size=11)
        
        self.setup_ui()
        self.show_riddle()
        
    def setup_ui(self):
        # Image frame
        self.image_frame = tk.Frame(self.root, bg='#1a1a1a', height=220)
        self.image_frame.pack(fill=tk.X, padx=10, pady=5)
        self.image_frame.pack_propagate(False)
        self.image_label = tk.Label(self.image_frame, bg='#1a1a1a', fg='#4a9eff')
        self.image_label.pack(expand=True)
        
        # Location label
        self.location_label = tk.Label(self.root, text="", font=self.title_font, bg='#2b2b2b', fg='#4a9eff', pady=5)
        self.location_label.pack()
        
        # Text output
        self.output = scrolledtext.ScrolledText(self.root, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', 
                                                insertbackground='#4a9eff', wrap=tk.WORD, height=18)
        self.output.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Tags
        self.output.tag_configure("green", foreground="#4ade80")
        self.output.tag_configure("yellow", foreground="#fbbf24")
        self.output.tag_configure("red", foreground="#f87171")
        self.output.tag_configure("cyan", foreground="#22d3ee")
        self.output.tag_configure("magenta", foreground="#e879f9")
        self.output.tag_configure("gold", foreground="#fbbf24")
        self.output.tag_configure("bold", font=self.title_font)
        
        # Input
        input_frame = tk.Frame(self.root, bg='#2b2b2b')
        input_frame.pack(fill=tk.X, padx=10, pady=5)
        tk.Label(input_frame, text=">", font=self.text_font, bg='#2b2b2b', fg='#4ade80').pack(side=tk.LEFT, padx=(0, 5))
        self.input_entry = tk.Entry(input_frame, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', insertbackground='#4a9eff')
        self.input_entry.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.input_entry.bind('<Return>', self.process_cmd)
        
    def p(self, text, tag=None):
        self.output.insert(tk.END, text + "\n", tag)
        self.output.see(tk.END)
        
    def clear(self):
        self.output.delete(1.0, tk.END)
        
    def load_img(self, img):
        if not img:
            return
        path = f"images/{img}"
        if os.path.exists(path):
            try:
                im = Image.open(path)
                im.thumbnail((900, 210), Image.Resampling.LANCZOS)
                photo = ImageTk.PhotoImage(im)
                self.image_label.configure(image=photo, text="")
                self.image_label.image = photo
            except:
                self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
        else:
            self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
    
    def show_riddle(self):
        self.input_entry.config(state='disabled')
        self.p("="*80, "bold")
        self.p("🕉️  ASHOKA'S PATH - ACT 1  🕉️", "bold")
        self.p("="*80, "bold")
        self.p("")
        self.p("An ancient ascetic appears...", "gold")
        self.p("")
        self.p("'Before you walk Ashoka's path, answer this:'")
        self.p("  I am won through blood, yet bring no peace")
        self.p("  I am sought by many, yet poison few")
        self.p("  I sit on heads, yet crush many souls")
        self.p("")
        self.p("  [1] The Crown", "cyan")
        self.p("  [2] A Sword", "cyan")
        self.p("  [3] Gold", "cyan")
        self.p("  [4] Power", "cyan")
        self.p("")
        self.input_entry.config(state='normal')
        self.input_entry.focus()
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_riddle)
    
    def check_riddle(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        if ans in ["1", "4"]:
            self.p(f"> {ans}", "cyan")
            self.p("")
            self.p("'Wisdom beyond your years. You may proceed...'", "green")
            self.p("")
            game.riddle_solved = True
            game.game_started = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.start_game()
        else:
            self.p(f"> {ans}", "cyan")
            self.p("'Think deeper...'", "red")
            self.p("")
    
    def start_game(self):
        self.p("="*80)
        self.p("Find FIVE sacred items:", "yellow")
        self.p("  • Vessel of compassion", "magenta")
        self.p("  • Mount of strength", "magenta")
        self.p("  • Drink of gods", "magenta")
        self.p("  • Food with humility", "magenta")
        self.p("  • Seat through courage", "magenta")
        self.p("")
        self.p("Type 'help' for commands, 'map' to see the palace", "cyan")
        self.p("Type 'quest' to track progress", "cyan")
        self.p("="*80)
        self.show_loc()
    
    def show_loc(self):
        loc = LOCATIONS[game.current_location]
        self.load_img(loc["image"])
        self.location_label.configure(text=loc["name"])
        self.p("")
        self.p(loc["description"])
        self.p("")
        exits = ", ".join([d.upper() for d in loc["exits"]])
        self.p(f"Exits: {exits}", "yellow")
        self.p("")
        if loc["npcs"]:
            self.p(f"People: {', '.join(loc['npcs'])}", "green")
            self.p("")
        # Show cushion if ready
        if game.current_location == "fathers_chambers" and len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
            self.p("You see: Royal Cushion", "magenta")
            self.p("")
    
    def process_cmd(self, e=None):
        if not game.game_started:
            return
        cmd = self.input_entry.get().lower().strip()
        self.input_entry.delete(0, tk.END)
        if not cmd:
            return
        
        # Combat mode - different command handling
        if game.in_combat:
            self.combat_action(cmd)
            return
        
        self.p(f"> {cmd}", "cyan")
        self.p("")
        
        if cmd in ["n","north"]: self.move("north")
        elif cmd in ["s","south"]: self.move("south")
        elif cmd in ["e","east"]: self.move("east")
        elif cmd in ["w","west"]: self.move("west")
        elif cmd in ["l","look"]: self.clear(); self.show_loc()
        elif cmd in ["i","inv","inventory"]: self.show_inv()
        elif cmd in ["q","quest","quests"]: self.show_quest()
        elif cmd in ["h","health"]: self.p(f"❤️ {game.health}/100", "green")
        elif cmd in ["?","help"]: self.show_help()
        elif cmd in ["m","map"]: self.show_map()
        elif cmd in ["quit","exit"]: self.p("Thanks for playing!","yellow"); self.root.after(2000, self.root.quit)
        elif cmd.startswith("talk "): self.talk(cmd.replace("talk to ","").replace("talk ",""))
        elif cmd == "help vikram": self.help_vikram()
        elif cmd == "test elephant": self.test_elephant()
        elif cmd.startswith("feed "): self.feed(cmd.split()[1])
        elif cmd == "remove thorn": self.remove_thorn()
        elif cmd == "comfort elephant": self.comfort()
        elif cmd == "ritual": self.ritual()
        elif cmd == "learn cooking": self.learn_cook()
        elif cmd == "make cakes": self.make_cakes()
        elif cmd == "train combat" or cmd == "spar" or cmd == "train": self.train_combat()
        elif cmd.startswith("pick ") or cmd.startswith("get ") or cmd.startswith("take "): self.pick(cmd.split(maxsplit=1)[1])
        else: self.p("Unknown command. Type 'help'", "red")
        
        self.p("")
        if game.check_victory() and not game.flags["victory"]:
            self.p("="*70, "gold")
            self.p("✨ ALL FIVE ITEMS COLLECTED! ✨", "gold")
            self.p("="*70, "gold")
            self.p("Return to GARDEN and talk to ascetic!", "yellow")
            self.p("")
    
    def move(self, dir):
        loc = LOCATIONS[game.current_location]
        if dir in loc["exits"]:
            next_loc = loc["exits"][dir]
            # Throne room logic
            if next_loc == "fathers_chambers":
                if len(game.inventory) < 4:
                    # Still too early - instant death
                    self.die()
                    return
                elif len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
                    # Ready for Susima battle!
                    game.current_location = next_loc
                    self.p(f"You travel {dir}...", "cyan")
                    self.p("")
                    self.start_susima_battle()
                    return
            
            game.current_location = next_loc
            self.p(f"You travel {dir}...", "cyan")
            self.p("")
            self.show_loc()
        else:
            self.p("Can't go that way!", "red")
    
    def die(self):
        self.clear()
        self.p("="*70, "red")
        self.p("DISCOVERED BY SUSIMA!", "red")
        self.p("="*70, "red")
        self.p("Your brother accuses you of treason!")
        self.p("YOU DIED", "red")
        self.p("Collect 4+ items first. Restarting...", "yellow")
        self.input_entry.config(state='disabled')
        self.root.after(5000, self.restart)
    
    def restart(self):
        global game
        game = GameState()
        self.clear()
        self.input_entry.config(state='normal')
        self.show_riddle()
    
    def pick(self, item):
        item = item.lower().replace(" ", "_")
        if item == "royal_cushion":
            if game.current_location != "fathers_chambers":
                self.p("Not here", "red")
                return
            if len(game.inventory) < 4:
                self.p("Too dangerous. Need 4+ items first", "red")
                return
            if game.has_item("royal_cushion"):
                self.p("Already have it", "yellow")
                return
            game.add_item("royal_cushion")
            game.quests["cushion"]["status"] = "completed"
            self.p("✓ Acquired: Royal Cushion (5/5)", "gold")
            return
        self.p("Items earned through quests!", "yellow")
    
    def talk(self, npc):
        loc = LOCATIONS[game.current_location]
        if npc not in loc["npcs"]:
            self.p("Nobody by that name here", "red")
            return
        
        if npc == "beggar": self.talk_beggar()
        elif npc == "vikram": self.talk_vikram()
        elif npc == "radhagupta": self.talk_radha()
        elif npc == "priest": self.talk_priest()
        elif npc == "amma": self.talk_amma()
        elif npc == "ascetic": self.talk_ascetic()
        else: self.p(f"{npc} says: Hello", "green")
    
    def talk_beggar(self):
        q = game.quests["beggar"]
        if q["stage"] == 0:
            self.p("DHRUVA:", "green")
            self.p("Vikram owes me 20 gold. Help me get it?")
            self.p("(Try: 'talk to vikram' then 'help vikram')", "yellow")
            q["status"] = "in_progress"
            q["stage"] = 1
        elif q["stage"] == 1:
            self.p("DHRUVA: Spoken to Vikram yet?")
        elif q["stage"] == 2 and game.flags["vikram_paid"]:
            self.p("DHRUVA: Thank you! Take my bowl.")
            game.add_item("golden_bowl")
            q["status"] = "completed"
            q["stage"] = 3
            self.p("✓ Quest Complete! Golden Bowl (1/5)", "gold")
        else:
            self.p("DHRUVA: Thanks again.")
    
    def talk_vikram(self):
        if game.quests["beggar"]["stage"] < 1:
            self.p("VIKRAM: I'm busy.")
            return
        self.p("VIKRAM: I owe him nothing!")
        self.p("(Try: 'help vikram')", "yellow")
    
    def help_vikram(self):
        if game.quests["beggar"]["stage"] != 1:
            return
        self.p("You check his ledgers... Found it!")
        self.p("VIKRAM: Fine, take it!")
        game.flags["vikram_paid"] = True
        game.quests["beggar"]["stage"] = 2
        self.p("Return to beggar", "yellow")
    
    def talk_radha(self):
        self.p("RADHAGUPTA:", "green")
        self.p("Test yourself with Airavata?")
        if not game.combat_training_complete:
            self.p("")
            self.p("Or would you like combat training?", "yellow")
            self.p("(Try: 'test elephant' or 'train combat')", "yellow")
        else:
            self.p("(Try: 'test elephant')", "yellow")
    
    def test_elephant(self):
        if game.current_location != "royal_stables":
            return
        if game.quests["elephant"]["status"] == "completed":
            self.p("Already done", "yellow")
            return
        game.quests["elephant"]["status"] = "in_progress"
        game.flags["elephant_started"] = True
        self.p("PART 1: Feed Airavata")
        self.p("Options: hay, mangoes, sugarcane")
        self.p("(Try: 'feed hay')", "yellow")
    
    def feed(self, food):
        if not game.flags["elephant_started"] or game.flags["fed_elephant"]:
            return
        if food == "hay":
            self.p("Airavata accepts gratefully")
            game.flags["fed_elephant"] = True
            self.p("PART 2: Remove thorn", "yellow")
            self.p("(Try: 'remove thorn')", "yellow")
        else:
            self.p("Wrong choice", "red")
    
    def remove_thorn(self):
        if not game.flags["fed_elephant"] or game.flags["removed_thorn"]:
            return
        self.p("You carefully remove the thorn...")
        game.flags["removed_thorn"] = True
        self.p("PART 3: Comfort elephant", "yellow")
        self.p("(Try: 'comfort elephant')", "yellow")
    
    def comfort(self):
        if not game.flags["removed_thorn"]:
            return
        self.p("You speak of loneliness...")
        self.p("Airavata touches your face")
        game.flags["comforted_elephant"] = True
        game.add_item("imperial_elephant")
        game.quests["elephant"]["status"] = "completed"
        self.p("✓ Quest Complete! Imperial Elephant (2/5)", "gold")
    
    def talk_priest(self):
        self.p("MAHINDA:", "green")
        if not game.flags["has_golden_bowl"]:
            self.p("Need golden bowl first")
        else:
            self.p("Ready for ritual?")
            self.p("(Try: 'ritual')", "yellow")
    
    def ritual(self):
        if game.current_location != "temple_district":
            return
        if not game.flags["has_golden_bowl"]:
            return
        if game.quests["soma"]["status"] == "completed":
            return
        self.p("You complete the sacred ritual...")
        self.p("Vision: Two paths - war and peace")
        game.add_item("soma")
        game.quests["soma"]["status"] = "completed"
        self.p("✓ Quest Complete! Soma (3/5)", "gold")
    
    def talk_amma(self):
        self.p("AMMA:", "green")
        self.p("Want to learn cooking?")
        self.p("(Try: 'learn cooking')", "yellow")
    
    def learn_cook(self):
        if game.current_location != "palace_kitchen":
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        game.quests["cooking"]["status"] = "in_progress"
        game.flags["learned_steps"] = True
        self.p("AMMA: 5 steps:")
        self.p("1. Mix flour and ghee")
        self.p("2. Add honey slowly")
        self.p("3. Fold in cardamom")
        self.p("4. Shape into rounds")
        self.p("5. Bake until golden")
        self.p("")
        self.p("Repeat the order. Type: 1 2 3 4 5")
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_steps)
    
    def check_steps(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        correct = ["1 2 3 4 5", "12345", "1,2,3,4,5", "1, 2, 3, 4, 5"]
        if ans in correct:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Perfect!")
            game.flags["steps_correct"] = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.p("(Try: 'make cakes')", "yellow")
            self.p("")
        else:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Wrong! Come back.", "red")
            game.quests["cooking"]["status"] = "not_started"
            game.flags["learned_steps"] = False
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
    
    def make_cakes(self):
        if not game.flags["steps_correct"]:
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        self.p("You make honey cakes with care...")
        self.p("AMMA: Remember this humility")
        game.add_item("honey_cakes")
        game.quests["cooking"]["status"] = "completed"
        self.p("✓ Quest Complete! Honey Cakes (4/5)", "gold")
    
    def talk_ascetic(self):
        if not game.check_victory():
            self.p("ASCETIC: Return with all five items")
            return
        if game.flags["victory"]:
            return
        self.victory()
    
    def victory(self):
        game.flags["victory"] = True
        self.clear()
        self.load_img("coronation.png")
        self.p("")
        self.p("="*70, "gold")
        self.p("ACT 1 COMPLETE", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("You are crowned EMPEROR ASHOKA")
        self.p("")
        self.p("="*70, "gold")
        self.p("TO BE CONTINUED IN ACT 2...", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("Thank you for playing!", "cyan")
    
    # ========== COMBAT SYSTEM ==========
    
    def train_combat(self):
        """Start optional training battle"""
        if game.current_location != "royal_stables":
            self.p("Must be at Royal Stables to train", "red")
            return
        
        if game.combat_training_complete:
            self.p("RADHAGUPTA: You've already mastered combat training.", "green")
            return
        
        self.p("RADHAGUPTA: You wish to learn combat?", "green")
        self.p("RADHAGUPTA: Very well. Defend yourself!")
        self.p("")
        self.init_combat("Training Dummy", 60, is_training=True)
    
    def start_susima_battle(self):
        """Start required Susima boss battle"""
        self.clear()
        self.load_img("fathers_chambers.png")
        self.p("")
        self.p("="*70, "red")
        self.p("⚔️  BROTHER SUSIMA BLOCKS YOUR PATH!", "red")
        self.p("="*70, "red")
        self.p("")
        self.p("SUSIMA: 'Ashoka! You dare approach the throne?'")
        self.p("SUSIMA: 'I see you've been collecting... trinkets.'")
        self.p("SUSIMA: 'Let's see if you can fight as well as you scheme!'")
        self.p("")
        self.p("Susima draws his sword!")
        self.p("")
        self.init_combat("Brother Susima", 100, is_training=False)
    
    def init_combat(self, enemy_name, enemy_hp, is_training=False):
        """Initialize combat state"""
        import random
        game.in_combat = True
        game.combat_enemy = enemy_name
        game.combat_enemy_hp = enemy_hp
        game.combat_enemy_max_hp = enemy_hp
        game.combat_heals_remaining = 3
        game.combat_defending = False
        game.combat_is_training = is_training
        
        self.show_combat_status()
    
    def show_combat_status(self):
        """Display combat UI"""
        import random
        
        # Player health bar
        player_bar_filled = int((game.health / game.max_health) * 10)
        player_bar = "█" * player_bar_filled + "░" * (10 - player_bar_filled)
        
        # Enemy health bar
        enemy_bar_filled = int((game.combat_enemy_hp / game.combat_enemy_max_hp) * 10)
        enemy_bar = "█" * enemy_bar_filled + "░" * (10 - enemy_bar_filled)
        
        self.p("")
        self.p("="*60, "bold")
        self.p(f"⚔️  BATTLE: You vs {game.combat_enemy}", "bold")
        self.p("="*60, "bold")
        self.p("")
        self.p(f"YOU:    {player_bar} {game.health}/{game.max_health} HP | Heals: {game.combat_heals_remaining}/3", "green")
        self.p(f"{game.combat_enemy.upper()}: {enemy_bar} {game.combat_enemy_hp}/{game.combat_enemy_max_hp} HP", "red")
        self.p("")
        self.p("Your turn! What will you do?", "yellow")
        self.p("  [1] ATTACK      - Strike (15-25 damage)", "cyan")
        self.p("  [2] DEFEND      - Guard (50% damage reduction next turn)", "cyan")
        self.p("  [3] POWER ATK   - Heavy strike (25-35 dmg, 30% miss)", "cyan")
        self.p("  [4] HEAL        - Restore 25 HP", "cyan")
        self.p("")
    
    def combat_action(self, cmd):
        """Process combat commands"""
        import random
        
        if cmd in ["1", "attack", "atk"]:
            self.player_attack()
        elif cmd in ["2", "defend", "def", "guard"]:
            self.player_defend()
        elif cmd in ["3", "power", "power attack", "power atk"]:
            self.player_power_attack()
        elif cmd in ["4", "heal"]:
            self.player_heal()
        else:
            self.p("Invalid action! Choose 1-4", "red")
            self.p("")
            return
        
        # Check if enemy dead
        if self.check_combat_end():
            return
        
        # Enemy turn
        self.enemy_turn()
        
        # Check if player dead
        if self.check_combat_end():
            return
        
        # Next round
        self.show_combat_status()
    
    def player_attack(self):
        """Player normal attack"""
        import random
        damage = random.randint(15, 25)
        game.combat_enemy_hp -= damage
        game.combat_defending = False
        
        self.p(f"> ATTACK", "cyan")
        self.p("")
        self.p(f"You strike {game.combat_enemy}!", "green")
        self.p(f"💥 Dealt {damage} damage!", "green")
        self.p("")
    
    def player_defend(self):
        """Player defend"""
        game.combat_defending = True
        
        self.p(f"> DEFEND", "cyan")
        self.p("")
        self.p("You raise your guard defensively!", "green")
        self.p("🛡️  Next attack will be reduced by 50%!", "green")
        self.p("")
    
    def player_power_attack(self):
        """Player power attack with miss chance"""
        import random
        game.combat_defending = False
        
        self.p(f"> POWER ATTACK", "cyan")
        self.p("")
        
        if random.random() < 0.3:  # 30% miss chance
            self.p("Your heavy strike misses!", "red")
            self.p("You're off balance!", "red")
            self.p("")
        else:
            damage = random.randint(25, 35)
            game.combat_enemy_hp -= damage
            self.p(f"CRITICAL HIT!", "gold")
            self.p(f"💥💥 Dealt {damage} damage!", "gold")
            self.p("")
    
    def player_heal(self):
        """Player heal"""
        game.combat_defending = False
        
        self.p(f"> HEAL", "cyan")
        self.p("")
        
        if game.combat_heals_remaining <= 0:
            self.p("No heals remaining!", "red")
            self.p("You waste your turn!", "red")
            self.p("")
            return
        
        heal_amount = 25
        old_health = game.health
        game.health = min(game.health + heal_amount, game.max_health)
        actual_heal = game.health - old_health
        game.combat_heals_remaining -= 1
        
        self.p(f"You drink a healing potion!", "green")
        self.p(f"❤️  Restored {actual_heal} HP!", "green")
        self.p("")
    
    def enemy_turn(self):
        """Enemy AI turn"""
        import random
        
        # Simple AI: Random attack or power attack
        action = random.choice(["attack", "attack", "power"])  # 66% attack, 33% power
        
        if action == "attack":
            damage = random.randint(10, 20) if game.combat_is_training else random.randint(15, 25)
            
            if game.combat_defending:
                damage = damage // 2
                self.p(f"{game.combat_enemy} attacks!", "yellow")
                self.p(f"🛡️  Your guard blocks half the damage!", "green")
                self.p(f"💔 You take {damage} damage!", "red")
            else:
                self.p(f"{game.combat_enemy} attacks!", "yellow")
                self.p(f"💔 You take {damage} damage!", "red")
            
            game.health -= damage
            game.combat_defending = False
            
        elif action == "power":
            if random.random() < 0.4:  # 40% miss for enemy power
                self.p(f"{game.combat_enemy} attempts a heavy strike but misses!", "green")
            else:
                damage = random.randint(20, 30) if game.combat_is_training else random.randint(25, 35)
                
                if game.combat_defending:
                    damage = damage // 2
                    self.p(f"{game.combat_enemy} unleashes a POWER ATTACK!", "red")
                    self.p(f"🛡️  Your guard reduces the damage!", "green")
                    self.p(f"💔💔 You take {damage} damage!", "red")
                else:
                    self.p(f"{game.combat_enemy} unleashes a POWER ATTACK!", "red")
                    self.p(f"💔💔 You take {damage} damage!", "red")
                
                game.health -= damage
            
            game.combat_defending = False
        
        self.p("")
    
    def check_combat_end(self):
        """Check if combat is over"""
        if game.combat_enemy_hp <= 0:
            # Player wins!
            self.p("="*60, "gold")
            self.p("⭐ VICTORY! ⭐", "gold")
            self.p("="*60, "gold")
            self.p("")
            
            if game.combat_is_training:
                self.p("RADHAGUPTA: 'Excellent! You have mastered combat!'", "green")
                self.p("RADHAGUPTA: 'You are ready for real battles now.'", "green")
                game.combat_training_complete = True
            else:
                self.p("Susima falls to his knees, defeated!", "green")
                self.p("")
                self.p("SUSIMA: 'Impossible... how did you...'", "yellow")
                self.p("")
                self.p("You stand victorious. The throne room is yours.", "green")
                self.p("")
                self.p("The royal cushion sits on the throne, waiting...", "magenta")
                self.p("")
            
            game.in_combat = False
            self.p("")
            
            if not game.combat_is_training:
                self.show_loc()
            
            return True
        
        if game.health <= 0:
            # Player dies!
            self.p("="*60, "red")
            self.p("💀 DEFEAT 💀", "red")
            self.p("="*60, "red")
            self.p("")
            
            if game.combat_is_training:
                self.p("RADHAGUPTA: 'You fought well, but need more training.'", "yellow")
                self.p("RADHAGUPTA: 'Rest and try again when ready.'", "yellow")
                game.health = game.max_health  # Restore health after training
                game.in_combat = False
            else:
                self.p("Susima strikes you down!", "red")
                self.p("")
                self.p("YOU DIED", "red")
                self.p("")
                self.p("Restarting in 5 seconds...", "yellow")
                game.in_combat = False
                self.input_entry.config(state='disabled')
                self.root.after(5000, self.restart)
            
            self.p("")
            return True
        
        return False
    
    # ========== END COMBAT SYSTEM ==========
    
    def show_inv(self):
        if not game.inventory:
            self.p("Empty", "yellow")
        else:
            self.p("📦 INVENTORY:", "bold")
            for i, item in enumerate(game.inventory, 1):
                self.p(f"  {i}. {item.replace('_', ' ').title()}", "green")
            self.p(f"{len(game.inventory)}/5 items", "yellow")
    
    def show_quest(self):
        self.p("📜 QUESTS:", "bold")
        for name, q in game.quests.items():
            s = "✓" if q["status"] == "completed" else "◆" if q["status"] == "in_progress" else "○"
            c = "green" if q["status"] == "completed" else "yellow" if q["status"] == "in_progress" else "cyan"
            self.p(f"{s} {name}", c)
    
    def show_map(self):
        """Display game map with current location highlighted"""
        # Load map image if available
        self.load_img("map.png")
        
        self.p("")
        self.p("="*60, "bold")
        self.p("🗺️  PATALIPUTRA PALACE MAP", "bold")
        self.p("="*60, "bold")
        self.p("")
        
        # Create map with current location highlighted
        current = game.current_location
        
        # Map structure
        locations_map = {
            "fathers_chambers": "    [Throne Room]",
            "palace_courtyard_n": "          |",
            "palace_courtyard": "  [Palace Courtyard]",
            "mothers_chambers": "          |",
            "palace_kitchen": "[Kitchen]--+",
            "garden_pavilion": "[Garden]--+--[Stables]",
            "marketplace": "              |",
            "temple_district": "     [Temple]--[Market]"
        }
        
        # Print map with highlights
        self.p("         NORTH", "cyan")
        self.p("")
        
        # Throne Room
        if current == "fathers_chambers":
            self.p("    >>> [THRONE ROOM] <<<", "gold")
        else:
            self.p("        [Throne Room]", "cyan")
        
        self.p("              |", "cyan")
        
        # Palace Courtyard (center)
        if current == "palace_courtyard":
            self.p("                            ", "cyan")
            self.p("  [Garden]------>>> [YOU ARE HERE] <<<------[Stables]", "gold")
            self.p("                            ", "cyan")
        else:
            self.p("                            ", "cyan")
            self.p("  [Garden]--------[Courtyard]--------[Stables]", "cyan")
            self.p("                            ", "cyan")
        
        self.p("                      |                    |", "cyan")
        
        # Bottom row
        left = "              " if current != "garden_pavilion" else ">>> "
        middle_m = " [Mother's] " if current != "mothers_chambers" else " >>> [YOU] <<< "
        middle_k = "[Kitchen]" if current != "palace_kitchen" else ">>>[YOU]<<<"
        right_m = "[Market]" if current != "marketplace" else ">>>[YOU]<<<"
        right_t = "[Temple]" if current != "temple_district" else ">>>[YOU]<<<"
        right_s = " " if current != "royal_stables" else ""
        
        self.p(f"{left}          {middle_m}            {right_s}", "cyan" if current not in ["mothers_chambers", "royal_stables"] else "gold")
        self.p(f"                      |", "cyan")
        self.p(f"                 {middle_k}", "cyan" if current != "palace_kitchen" else "gold")
        self.p("")
        self.p(f"                                   {right_t}---{right_m}", "cyan" if current not in ["marketplace", "temple_district"] else "gold")
        
        self.p("")
        self.p(f"Current Location: {LOCATIONS[current]['name']}", "yellow")
        self.p("")
        self.p("="*60, "bold")
        self.p("")
    
    def show_help(self):
        self.p("COMMANDS:", "bold")
        self.p("Movement: n/s/e/w, look, map")
        self.p("Actions: talk to <npc>, inventory, quest")
        self.p("Combat: train combat (at stables)")
        self.p("Quests: help vikram, test elephant, feed <food>,")
        self.p("        remove thorn, comfort elephant, ritual,")
        self.p("        learn cooking, make cakes, pick cushion")
        self.p("Other: help, quit")

def main():
    root = tk.Tk()
    app = AshokaGame(root)
    root.mainloop()

if __name__ == "__main__":
    main()

# version5 with music


In [1]:
"""
ASHOKA'S PATH - COMPLETE ACT 1
All quests, riddles, and adventures fully implemented
University of Aberdeen GameJam 2026
"""

import tkinter as tk
from tkinter import scrolledtext, font
from PIL import Image, ImageTk
import os
import pygame

# Game State with ALL quest tracking
class GameState:
    def __init__(self):
        self.current_location = "palace_courtyard"
        self.inventory = []
        self.health = 100
        self.max_health = 100
        self.riddle_solved = False
        self.game_started = False
        
        # Combat state
        self.in_combat = False
        self.combat_enemy = None
        self.combat_enemy_hp = 0
        self.combat_enemy_max_hp = 0
        self.combat_heals_remaining = 3
        self.combat_defending = False
        self.combat_training_complete = False
        
        self.quests = {
            "beggar": {"status": "not_started", "stage": 0},
            "elephant": {"status": "not_started", "stage": 0},
            "soma": {"status": "not_started", "stage": 0},
            "cooking": {"status": "not_started", "stage": 0},
            "cushion": {"status": "not_started", "stage": 0}
        }
        
        self.flags = {
            "has_golden_bowl": False,
            "vikram_paid": False,
            "elephant_started": False,
            "fed_elephant": False,
            "removed_thorn": False,
            "comforted_elephant": False,
            "learned_steps": False,
            "steps_correct": False,
            "victory": False
        }
        
        self.items_needed = ["golden_bowl", "imperial_elephant", "soma", "honey_cakes", "royal_cushion"]
        
        # Music state
        self.music_enabled = True
        self.current_music = None
        
    def has_item(self, item):
        return item in self.inventory
        
    def add_item(self, item):
        if item not in self.inventory:
            self.inventory.append(item)
            if item == "golden_bowl":
                self.flags["has_golden_bowl"] = True
            return True
        return False
            
    def check_victory(self):
        return all(item in self.inventory for item in self.items_needed)

game = GameState()

# Locations
LOCATIONS = {
    "palace_courtyard": {
        "name": "Palace Courtyard",
        "description": "Grand courtyard with marble pillars. Morning sun glints off golden domes.",
        "image": "palace_courtyard.png",
        "exits": {"north": "fathers_chambers", "south": "mothers_chambers", "east": "royal_stables", "west": "garden_pavilion"},
        "npcs": []
    },
    "mothers_chambers": {
        "name": "Mother's Chambers",
        "description": "Gentle space with sandalwood incense. Your mother sits by the window.",
        "image": "mothers_chambers.png",
        "exits": {"north": "palace_courtyard", "west": "palace_kitchen"},
        "npcs": ["mother"]
    },
    "palace_kitchen": {
        "name": "Palace Kitchen",
        "description": "Busy kitchen filled with heat and noise. Amma Savitri kneads dough.",
        "image": "palace_kitchen.png",
        "exits": {"east": "mothers_chambers"},
        "npcs": ["amma"]
    },
    "garden_pavilion": {
        "name": "Garden of the Golden Pavilion",
        "description": "Otherworldly space. Golden pavilion touches clouds. Ascetic meditates beneath bodhi tree.",
        "image": "garden_pavilion.png",
        "exits": {"east": "palace_courtyard"},
        "npcs": ["ascetic"]
    },
    "royal_stables": {
        "name": "Royal Stables",
        "description": "Earthy smell of elephants. Minister Radhagupta stands near Airavata.",
        "image": "royal_stables.png",
        "exits": {"west": "palace_courtyard", "south": "marketplace"},
        "npcs": ["radhagupta"]
    },
    "marketplace": {
        "name": "Marketplace",
        "description": "Chaos and color. Blind beggar sits with golden bowl. Vikram at his stall.",
        "image": "marketplace.png",
        "exits": {"north": "royal_stables", "west": "temple_district"},
        "npcs": ["beggar", "vikram"]
    },
    "temple_district": {
        "name": "Temple District",
        "description": "Incense smoke rises. Priest Mahinda tends the sacred fire.",
        "image": "temple_district.png",
        "exits": {"east": "marketplace"},
        "npcs": ["priest"]
    },
    "fathers_chambers": {
        "name": "Throne Room",
        "description": "Stern room with black stone throne. Your father reviews reports.",
        "image": "fathers_chambers.png",
        "exits": {"south": "palace_courtyard"},
        "npcs": ["emperor"]
    }
}

# GUI Application
class AshokaGame:
    def __init__(self, root):
        self.root = root
        self.root.title("🕉️ ASHOKA'S PATH - ACT 1")
        self.root.geometry("1000x750")
        self.root.configure(bg='#2b2b2b')
        
        self.title_font = font.Font(family="Arial", size=14, weight="bold")
        self.text_font = font.Font(family="Courier", size=11)
        
        # Initialize pygame mixer for music
        try:
            pygame.mixer.init()
            self.music_available = True
        except:
            self.music_available = False
            self.p("Music system unavailable", "yellow")
        
        self.setup_ui()
        self.play_music("menu")  # Play menu music
        self.show_riddle()
        
    def setup_ui(self):
        # Image frame
        self.image_frame = tk.Frame(self.root, bg='#1a1a1a', height=220)
        self.image_frame.pack(fill=tk.X, padx=10, pady=5)
        self.image_frame.pack_propagate(False)
        self.image_label = tk.Label(self.image_frame, bg='#1a1a1a', fg='#4a9eff')
        self.image_label.pack(expand=True)
        
        # Location label
        self.location_label = tk.Label(self.root, text="", font=self.title_font, bg='#2b2b2b', fg='#4a9eff', pady=5)
        self.location_label.pack()
        
        # Text output
        self.output = scrolledtext.ScrolledText(self.root, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', 
                                                insertbackground='#4a9eff', wrap=tk.WORD, height=18)
        self.output.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Tags
        self.output.tag_configure("green", foreground="#4ade80")
        self.output.tag_configure("yellow", foreground="#fbbf24")
        self.output.tag_configure("red", foreground="#f87171")
        self.output.tag_configure("cyan", foreground="#22d3ee")
        self.output.tag_configure("magenta", foreground="#e879f9")
        self.output.tag_configure("gold", foreground="#fbbf24")
        self.output.tag_configure("bold", font=self.title_font)
        
        # Input
        input_frame = tk.Frame(self.root, bg='#2b2b2b')
        input_frame.pack(fill=tk.X, padx=10, pady=5)
        tk.Label(input_frame, text=">", font=self.text_font, bg='#2b2b2b', fg='#4ade80').pack(side=tk.LEFT, padx=(0, 5))
        self.input_entry = tk.Entry(input_frame, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', insertbackground='#4a9eff')
        self.input_entry.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.input_entry.bind('<Return>', self.process_cmd)
        
    def p(self, text, tag=None):
        self.output.insert(tk.END, text + "\n", tag)
        self.output.see(tk.END)
        
    def clear(self):
        self.output.delete(1.0, tk.END)
        
    def load_img(self, img):
        if not img:
            return
        path = f"images/{img}"
        if os.path.exists(path):
            try:
                im = Image.open(path)
                im.thumbnail((900, 210), Image.Resampling.LANCZOS)
                photo = ImageTk.PhotoImage(im)
                self.image_label.configure(image=photo, text="")
                self.image_label.image = photo
            except:
                self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
        else:
            self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
    
    # ========== MUSIC SYSTEM ==========
    
    def play_music(self, track_name):
        """Play background music"""
        if not self.music_available or not game.music_enabled:
            return
        
        music_files = {
            "menu": "music/menu.mp3",
            "exploration": "music/exploration.mp3",
            "battle": "music/battle.mp3",
            "victory": "music/victory.mp3"
        }
        
        if track_name not in music_files:
            return
        
        music_path = music_files[track_name]
        
        # Only change if different track
        if game.current_music == track_name:
            return
        
        try:
            if os.path.exists(music_path):
                pygame.mixer.music.load(music_path)
                pygame.mixer.music.set_volume(0.5)  # 50% volume
                
                # Loop for exploration/battle, play once for victory
                if track_name in ["exploration", "battle"]:
                    pygame.mixer.music.play(-1)  # Loop indefinitely
                else:
                    pygame.mixer.music.play(0)   # Play once
                
                game.current_music = track_name
        except Exception as e:
            pass  # Silently fail if music file missing
    
    def stop_music(self):
        """Stop all music"""
        if self.music_available:
            try:
                pygame.mixer.music.stop()
                game.current_music = None
            except:
                pass
    
    def toggle_music(self):
        """Toggle music on/off"""
        game.music_enabled = not game.music_enabled
        
        if game.music_enabled:
            self.p("🎵 Music: ON", "green")
            # Resume appropriate music
            if game.in_combat:
                self.play_music("battle")
            else:
                self.play_music("exploration")
        else:
            self.p("🔇 Music: OFF", "yellow")
            self.stop_music()
    
    def set_volume(self, volume):
        """Set music volume (0.0 to 1.0)"""
        if self.music_available:
            try:
                pygame.mixer.music.set_volume(max(0.0, min(1.0, volume)))
                self.p(f"🔊 Volume: {int(volume*100)}%", "cyan")
            except:
                pass
    
    # ========== END MUSIC SYSTEM ==========
    
    def show_riddle(self):
        self.input_entry.config(state='disabled')
        self.p("="*80, "bold")
        self.p("🕉️  ASHOKA'S PATH - ACT 1  🕉️", "bold")
        self.p("="*80, "bold")
        self.p("")
        self.p("An ancient ascetic appears...", "gold")
        self.p("")
        self.p("'Before you walk Ashoka's path, answer this:'")
        self.p("  I am won through blood, yet bring no peace")
        self.p("  I am sought by many, yet poison few")
        self.p("  I sit on heads, yet crush many souls")
        self.p("")
        self.p("  [1] The Crown", "cyan")
        self.p("  [2] A Sword", "cyan")
        self.p("  [3] Gold", "cyan")
        self.p("  [4] Power", "cyan")
        self.p("")
        self.input_entry.config(state='normal')
        self.input_entry.focus()
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_riddle)
    
    def check_riddle(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        if ans in ["1", "4"]:
            self.p(f"> {ans}", "cyan")
            self.p("")
            self.p("'Wisdom beyond your years. You may proceed...'", "green")
            self.p("")
            game.riddle_solved = True
            game.game_started = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.start_game()
        else:
            self.p(f"> {ans}", "cyan")
            self.p("'Think deeper...'", "red")
            self.p("")
    
    def start_game(self):
        self.play_music("exploration")  # Start exploration music
        self.p("="*80)
        self.p("Find FIVE sacred items:", "yellow")
        self.p("  • Vessel of compassion", "magenta")
        self.p("  • Mount of strength", "magenta")
        self.p("  • Drink of gods", "magenta")
        self.p("  • Food with humility", "magenta")
        self.p("  • Seat through courage", "magenta")
        self.p("")
        self.p("Type 'help' for commands, 'map' to see the palace", "cyan")
        self.p("Type 'quest' to track progress", "cyan")
        self.p("="*80)
        self.show_loc()
    
    def show_loc(self):
        loc = LOCATIONS[game.current_location]
        self.load_img(loc["image"])
        self.location_label.configure(text=loc["name"])
        self.p("")
        self.p(loc["description"])
        self.p("")
        exits = ", ".join([d.upper() for d in loc["exits"]])
        self.p(f"Exits: {exits}", "yellow")
        self.p("")
        if loc["npcs"]:
            self.p(f"People: {', '.join(loc['npcs'])}", "green")
            self.p("")
        # Show cushion if ready
        if game.current_location == "fathers_chambers" and len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
            self.p("You see: Royal Cushion", "magenta")
            self.p("")
    
    def process_cmd(self, e=None):
        if not game.game_started:
            return
        cmd = self.input_entry.get().lower().strip()
        self.input_entry.delete(0, tk.END)
        if not cmd:
            return
        
        # Combat mode - different command handling
        if game.in_combat:
            self.combat_action(cmd)
            return
        
        self.p(f"> {cmd}", "cyan")
        self.p("")
        
        if cmd in ["n","north"]: self.move("north")
        elif cmd in ["s","south"]: self.move("south")
        elif cmd in ["e","east"]: self.move("east")
        elif cmd in ["w","west"]: self.move("west")
        elif cmd in ["l","look"]: self.clear(); self.show_loc()
        elif cmd in ["i","inv","inventory"]: self.show_inv()
        elif cmd in ["q","quest","quests"]: self.show_quest()
        elif cmd in ["h","health"]: self.p(f"❤️ {game.health}/100", "green")
        elif cmd in ["?","help"]: self.show_help()
        elif cmd in ["m","map"]: self.show_map()
        elif cmd in ["music","toggle music"]: self.toggle_music()
        elif cmd.startswith("volume "):
            try:
                vol = float(cmd.split()[1]) / 100.0
                self.set_volume(vol)
            except:
                self.p("Usage: volume <0-100>", "red")
        elif cmd in ["quit","exit"]: self.p("Thanks for playing!","yellow"); self.root.after(2000, self.root.quit)
        elif cmd.startswith("talk "): self.talk(cmd.replace("talk to ","").replace("talk ",""))
        elif cmd == "help vikram": self.help_vikram()
        elif cmd == "test elephant": self.test_elephant()
        elif cmd.startswith("feed "): self.feed(cmd.split()[1])
        elif cmd == "remove thorn": self.remove_thorn()
        elif cmd == "comfort elephant": self.comfort()
        elif cmd == "ritual": self.ritual()
        elif cmd == "learn cooking": self.learn_cook()
        elif cmd == "make cakes": self.make_cakes()
        elif cmd == "train combat" or cmd == "spar" or cmd == "train": self.train_combat()
        elif cmd.startswith("pick ") or cmd.startswith("get ") or cmd.startswith("take "): self.pick(cmd.split(maxsplit=1)[1])
        else: self.p("Unknown command. Type 'help'", "red")
        
        self.p("")
        if game.check_victory() and not game.flags["victory"]:
            self.p("="*70, "gold")
            self.p("✨ ALL FIVE ITEMS COLLECTED! ✨", "gold")
            self.p("="*70, "gold")
            self.p("Return to GARDEN and talk to ascetic!", "yellow")
            self.p("")
    
    def move(self, dir):
        loc = LOCATIONS[game.current_location]
        if dir in loc["exits"]:
            next_loc = loc["exits"][dir]
            # Throne room logic
            if next_loc == "fathers_chambers":
                if len(game.inventory) < 4:
                    # Still too early - instant death
                    self.die()
                    return
                elif len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
                    # Ready for Susima battle!
                    game.current_location = next_loc
                    self.p(f"You travel {dir}...", "cyan")
                    self.p("")
                    self.start_susima_battle()
                    return
            
            game.current_location = next_loc
            self.p(f"You travel {dir}...", "cyan")
            self.p("")
            self.show_loc()
        else:
            self.p("Can't go that way!", "red")
    
    def die(self):
        self.clear()
        self.p("="*70, "red")
        self.p("DISCOVERED BY SUSIMA!", "red")
        self.p("="*70, "red")
        self.p("Your brother accuses you of treason!")
        self.p("YOU DIED", "red")
        self.p("Collect 4+ items first. Restarting...", "yellow")
        self.input_entry.config(state='disabled')
        self.root.after(5000, self.restart)
    
    def restart(self):
        global game
        game = GameState()
        self.clear()
        self.input_entry.config(state='normal')
        self.show_riddle()
    
    def pick(self, item):
        item = item.lower().replace(" ", "_")
        if item == "royal_cushion":
            if game.current_location != "fathers_chambers":
                self.p("Not here", "red")
                return
            if len(game.inventory) < 4:
                self.p("Too dangerous. Need 4+ items first", "red")
                return
            if game.has_item("royal_cushion"):
                self.p("Already have it", "yellow")
                return
            game.add_item("royal_cushion")
            game.quests["cushion"]["status"] = "completed"
            self.p("✓ Acquired: Royal Cushion (5/5)", "gold")
            return
        self.p("Items earned through quests!", "yellow")
    
    def talk(self, npc):
        loc = LOCATIONS[game.current_location]
        if npc not in loc["npcs"]:
            self.p("Nobody by that name here", "red")
            return
        
        if npc == "beggar": self.talk_beggar()
        elif npc == "vikram": self.talk_vikram()
        elif npc == "radhagupta": self.talk_radha()
        elif npc == "priest": self.talk_priest()
        elif npc == "amma": self.talk_amma()
        elif npc == "ascetic": self.talk_ascetic()
        else: self.p(f"{npc} says: Hello", "green")
    
    def talk_beggar(self):
        q = game.quests["beggar"]
        if q["stage"] == 0:
            self.p("DHRUVA:", "green")
            self.p("Vikram owes me 20 gold. Help me get it?")
            self.p("(Try: 'talk to vikram' then 'help vikram')", "yellow")
            q["status"] = "in_progress"
            q["stage"] = 1
        elif q["stage"] == 1:
            self.p("DHRUVA: Spoken to Vikram yet?")
        elif q["stage"] == 2 and game.flags["vikram_paid"]:
            self.p("DHRUVA: Thank you! Take my bowl.")
            game.add_item("golden_bowl")
            q["status"] = "completed"
            q["stage"] = 3
            self.p("✓ Quest Complete! Golden Bowl (1/5)", "gold")
        else:
            self.p("DHRUVA: Thanks again.")
    
    def talk_vikram(self):
        if game.quests["beggar"]["stage"] < 1:
            self.p("VIKRAM: I'm busy.")
            return
        self.p("VIKRAM: I owe him nothing!")
        self.p("(Try: 'help vikram')", "yellow")
    
    def help_vikram(self):
        if game.quests["beggar"]["stage"] != 1:
            return
        self.p("You check his ledgers... Found it!")
        self.p("VIKRAM: Fine, take it!")
        game.flags["vikram_paid"] = True
        game.quests["beggar"]["stage"] = 2
        self.p("Return to beggar", "yellow")
    
    def talk_radha(self):
        self.p("RADHAGUPTA:", "green")
        self.p("Test yourself with Airavata?")
        if not game.combat_training_complete:
            self.p("")
            self.p("Or would you like combat training?", "yellow")
            self.p("(Try: 'test elephant' or 'train combat')", "yellow")
        else:
            self.p("(Try: 'test elephant')", "yellow")
    
    def test_elephant(self):
        if game.current_location != "royal_stables":
            return
        if game.quests["elephant"]["status"] == "completed":
            self.p("Already done", "yellow")
            return
        game.quests["elephant"]["status"] = "in_progress"
        game.flags["elephant_started"] = True
        self.p("PART 1: Feed Airavata")
        self.p("Options: hay, mangoes, sugarcane")
        self.p("(Try: 'feed hay')", "yellow")
    
    def feed(self, food):
        if not game.flags["elephant_started"] or game.flags["fed_elephant"]:
            return
        if food == "hay":
            self.p("Airavata accepts gratefully")
            game.flags["fed_elephant"] = True
            self.p("PART 2: Remove thorn", "yellow")
            self.p("(Try: 'remove thorn')", "yellow")
        else:
            self.p("Wrong choice", "red")
    
    def remove_thorn(self):
        if not game.flags["fed_elephant"] or game.flags["removed_thorn"]:
            return
        self.p("You carefully remove the thorn...")
        game.flags["removed_thorn"] = True
        self.p("PART 3: Comfort elephant", "yellow")
        self.p("(Try: 'comfort elephant')", "yellow")
    
    def comfort(self):
        if not game.flags["removed_thorn"]:
            return
        self.p("You speak of loneliness...")
        self.p("Airavata touches your face")
        game.flags["comforted_elephant"] = True
        game.add_item("imperial_elephant")
        game.quests["elephant"]["status"] = "completed"
        self.p("✓ Quest Complete! Imperial Elephant (2/5)", "gold")
    
    def talk_priest(self):
        self.p("MAHINDA:", "green")
        if not game.flags["has_golden_bowl"]:
            self.p("Need golden bowl first")
        else:
            self.p("Ready for ritual?")
            self.p("(Try: 'ritual')", "yellow")
    
    def ritual(self):
        if game.current_location != "temple_district":
            return
        if not game.flags["has_golden_bowl"]:
            return
        if game.quests["soma"]["status"] == "completed":
            return
        self.p("You complete the sacred ritual...")
        self.p("Vision: Two paths - war and peace")
        game.add_item("soma")
        game.quests["soma"]["status"] = "completed"
        self.p("✓ Quest Complete! Soma (3/5)", "gold")
    
    def talk_amma(self):
        self.p("AMMA:", "green")
        self.p("Want to learn cooking?")
        self.p("(Try: 'learn cooking')", "yellow")
    
    def learn_cook(self):
        if game.current_location != "palace_kitchen":
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        game.quests["cooking"]["status"] = "in_progress"
        game.flags["learned_steps"] = True
        self.p("AMMA: 5 steps:")
        self.p("1. Mix flour and ghee")
        self.p("2. Add honey slowly")
        self.p("3. Fold in cardamom")
        self.p("4. Shape into rounds")
        self.p("5. Bake until golden")
        self.p("")
        self.p("Repeat the order. Type: 1 2 3 4 5")
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_steps)
    
    def check_steps(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        correct = ["1 2 3 4 5", "12345", "1,2,3,4,5", "1, 2, 3, 4, 5"]
        if ans in correct:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Perfect!")
            game.flags["steps_correct"] = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.p("(Try: 'make cakes')", "yellow")
            self.p("")
        else:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Wrong! Come back.", "red")
            game.quests["cooking"]["status"] = "not_started"
            game.flags["learned_steps"] = False
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
    
    def make_cakes(self):
        if not game.flags["steps_correct"]:
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        self.p("You make honey cakes with care...")
        self.p("AMMA: Remember this humility")
        game.add_item("honey_cakes")
        game.quests["cooking"]["status"] = "completed"
        self.p("✓ Quest Complete! Honey Cakes (4/5)", "gold")
    
    def talk_ascetic(self):
        if not game.check_victory():
            self.p("ASCETIC: Return with all five items")
            return
        if game.flags["victory"]:
            return
        self.victory()
    
    def victory(self):
        game.flags["victory"] = True
        self.play_music("victory")  # Play victory music
        self.clear()
        self.load_img("coronation.png")
        self.p("")
        self.p("="*70, "gold")
        self.p("ACT 1 COMPLETE", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("You are crowned EMPEROR ASHOKA")
        self.p("")
        self.p("="*70, "gold")
        self.p("TO BE CONTINUED IN ACT 2...", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("Thank you for playing!", "cyan")
    
    # ========== COMBAT SYSTEM ==========
    
    def train_combat(self):
        """Start optional training battle"""
        if game.current_location != "royal_stables":
            self.p("Must be at Royal Stables to train", "red")
            return
        
        if game.combat_training_complete:
            self.p("RADHAGUPTA: You've already mastered combat training.", "green")
            return
        
        self.p("RADHAGUPTA: You wish to learn combat?", "green")
        self.p("RADHAGUPTA: Very well. Defend yourself!")
        self.p("")
        self.init_combat("Training Dummy", 60, is_training=True)
    
    def start_susima_battle(self):
        """Start required Susima boss battle"""
        self.clear()
        self.load_img("fathers_chambers.png")
        self.p("")
        self.p("="*70, "red")
        self.p("⚔️  BROTHER SUSIMA BLOCKS YOUR PATH!", "red")
        self.p("="*70, "red")
        self.p("")
        self.p("SUSIMA: 'Ashoka! You dare approach the throne?'")
        self.p("SUSIMA: 'I see you've been collecting... trinkets.'")
        self.p("SUSIMA: 'Let's see if you can fight as well as you scheme!'")
        self.p("")
        self.p("Susima draws his sword!")
        self.p("")
        self.init_combat("Brother Susima", 100, is_training=False)
    
    def init_combat(self, enemy_name, enemy_hp, is_training=False):
        """Initialize combat state"""
        import random
        self.play_music("battle")  # Play battle music
        game.in_combat = True
        game.combat_enemy = enemy_name
        game.combat_enemy_hp = enemy_hp
        game.combat_enemy_max_hp = enemy_hp
        game.combat_heals_remaining = 3
        game.combat_defending = False
        game.combat_is_training = is_training
        
        self.show_combat_status()
    
    def show_combat_status(self):
        """Display combat UI"""
        import random
        
        # Player health bar
        player_bar_filled = int((game.health / game.max_health) * 10)
        player_bar = "█" * player_bar_filled + "░" * (10 - player_bar_filled)
        
        # Enemy health bar
        enemy_bar_filled = int((game.combat_enemy_hp / game.combat_enemy_max_hp) * 10)
        enemy_bar = "█" * enemy_bar_filled + "░" * (10 - enemy_bar_filled)
        
        self.p("")
        self.p("="*60, "bold")
        self.p(f"⚔️  BATTLE: You vs {game.combat_enemy}", "bold")
        self.p("="*60, "bold")
        self.p("")
        self.p(f"YOU:    {player_bar} {game.health}/{game.max_health} HP | Heals: {game.combat_heals_remaining}/3", "green")
        self.p(f"{game.combat_enemy.upper()}: {enemy_bar} {game.combat_enemy_hp}/{game.combat_enemy_max_hp} HP", "red")
        self.p("")
        self.p("Your turn! What will you do?", "yellow")
        self.p("  [1] ATTACK      - Strike (15-25 damage)", "cyan")
        self.p("  [2] DEFEND      - Guard (50% damage reduction next turn)", "cyan")
        self.p("  [3] POWER ATK   - Heavy strike (25-35 dmg, 30% miss)", "cyan")
        self.p("  [4] HEAL        - Restore 25 HP", "cyan")
        self.p("")
    
    def combat_action(self, cmd):
        """Process combat commands"""
        import random
        
        if cmd in ["1", "attack", "atk"]:
            self.player_attack()
        elif cmd in ["2", "defend", "def", "guard"]:
            self.player_defend()
        elif cmd in ["3", "power", "power attack", "power atk"]:
            self.player_power_attack()
        elif cmd in ["4", "heal"]:
            self.player_heal()
        else:
            self.p("Invalid action! Choose 1-4", "red")
            self.p("")
            return
        
        # Check if enemy dead
        if self.check_combat_end():
            return
        
        # Enemy turn
        self.enemy_turn()
        
        # Check if player dead
        if self.check_combat_end():
            return
        
        # Next round
        self.show_combat_status()
    
    def player_attack(self):
        """Player normal attack"""
        import random
        damage = random.randint(15, 25)
        game.combat_enemy_hp -= damage
        game.combat_defending = False
        
        self.p(f"> ATTACK", "cyan")
        self.p("")
        self.p(f"You strike {game.combat_enemy}!", "green")
        self.p(f"💥 Dealt {damage} damage!", "green")
        self.p("")
    
    def player_defend(self):
        """Player defend"""
        game.combat_defending = True
        
        self.p(f"> DEFEND", "cyan")
        self.p("")
        self.p("You raise your guard defensively!", "green")
        self.p("🛡️  Next attack will be reduced by 50%!", "green")
        self.p("")
    
    def player_power_attack(self):
        """Player power attack with miss chance"""
        import random
        game.combat_defending = False
        
        self.p(f"> POWER ATTACK", "cyan")
        self.p("")
        
        if random.random() < 0.3:  # 30% miss chance
            self.p("Your heavy strike misses!", "red")
            self.p("You're off balance!", "red")
            self.p("")
        else:
            damage = random.randint(25, 35)
            game.combat_enemy_hp -= damage
            self.p(f"CRITICAL HIT!", "gold")
            self.p(f"💥💥 Dealt {damage} damage!", "gold")
            self.p("")
    
    def player_heal(self):
        """Player heal"""
        game.combat_defending = False
        
        self.p(f"> HEAL", "cyan")
        self.p("")
        
        if game.combat_heals_remaining <= 0:
            self.p("No heals remaining!", "red")
            self.p("You waste your turn!", "red")
            self.p("")
            return
        
        heal_amount = 25
        old_health = game.health
        game.health = min(game.health + heal_amount, game.max_health)
        actual_heal = game.health - old_health
        game.combat_heals_remaining -= 1
        
        self.p(f"You drink a healing potion!", "green")
        self.p(f"❤️  Restored {actual_heal} HP!", "green")
        self.p("")
    
    def enemy_turn(self):
        """Enemy AI turn"""
        import random
        
        # Simple AI: Random attack or power attack
        action = random.choice(["attack", "attack", "power"])  # 66% attack, 33% power
        
        if action == "attack":
            damage = random.randint(10, 20) if game.combat_is_training else random.randint(15, 25)
            
            if game.combat_defending:
                damage = damage // 2
                self.p(f"{game.combat_enemy} attacks!", "yellow")
                self.p(f"🛡️  Your guard blocks half the damage!", "green")
                self.p(f"💔 You take {damage} damage!", "red")
            else:
                self.p(f"{game.combat_enemy} attacks!", "yellow")
                self.p(f"💔 You take {damage} damage!", "red")
            
            game.health -= damage
            game.combat_defending = False
            
        elif action == "power":
            if random.random() < 0.4:  # 40% miss for enemy power
                self.p(f"{game.combat_enemy} attempts a heavy strike but misses!", "green")
            else:
                damage = random.randint(20, 30) if game.combat_is_training else random.randint(25, 35)
                
                if game.combat_defending:
                    damage = damage // 2
                    self.p(f"{game.combat_enemy} unleashes a POWER ATTACK!", "red")
                    self.p(f"🛡️  Your guard reduces the damage!", "green")
                    self.p(f"💔💔 You take {damage} damage!", "red")
                else:
                    self.p(f"{game.combat_enemy} unleashes a POWER ATTACK!", "red")
                    self.p(f"💔💔 You take {damage} damage!", "red")
                
                game.health -= damage
            
            game.combat_defending = False
        
        self.p("")
    
    def check_combat_end(self):
        """Check if combat is over"""
        if game.combat_enemy_hp <= 0:
            # Player wins!
            self.play_music("victory")  # Victory music
            self.p("="*60, "gold")
            self.p("⭐ VICTORY! ⭐", "gold")
            self.p("="*60, "gold")
            self.p("")
            
            if game.combat_is_training:
                self.p("RADHAGUPTA: 'Excellent! You have mastered combat!'", "green")
                self.p("RADHAGUPTA: 'You are ready for real battles now.'", "green")
                game.combat_training_complete = True
            else:
                self.p("Susima falls to his knees, defeated!", "green")
                self.p("")
                self.p("SUSIMA: 'Impossible... how did you...'", "yellow")
                self.p("")
                self.p("You stand victorious. The throne room is yours.", "green")
                self.p("")
                self.p("The royal cushion sits on the throne, waiting...", "magenta")
                self.p("")
            
            game.in_combat = False
            self.p("")
            
            # Return to exploration music after a delay
            self.root.after(3000, lambda: self.play_music("exploration"))
            
            if not game.combat_is_training:
                self.show_loc()
            
            return True
        
        if game.health <= 0:
            # Player dies!
            self.stop_music()  # Stop music on death
            self.p("="*60, "red")
            self.p("💀 DEFEAT 💀", "red")
            self.p("="*60, "red")
            self.p("")
            
            if game.combat_is_training:
                self.p("RADHAGUPTA: 'You fought well, but need more training.'", "yellow")
                self.p("RADHAGUPTA: 'Rest and try again when ready.'", "yellow")
                game.health = game.max_health  # Restore health after training
                game.in_combat = False
                self.play_music("exploration")  # Back to exploration
            else:
                self.p("Susima strikes you down!", "red")
                self.p("")
                self.p("YOU DIED", "red")
                self.p("")
                self.p("Restarting in 5 seconds...", "yellow")
                game.in_combat = False
                self.input_entry.config(state='disabled')
                self.root.after(5000, self.restart)
            
            self.p("")
            return True
        
        return False
    
    # ========== END COMBAT SYSTEM ==========
    
    def show_inv(self):
        if not game.inventory:
            self.p("Empty", "yellow")
        else:
            self.p("📦 INVENTORY:", "bold")
            for i, item in enumerate(game.inventory, 1):
                self.p(f"  {i}. {item.replace('_', ' ').title()}", "green")
            self.p(f"{len(game.inventory)}/5 items", "yellow")
    
    def show_quest(self):
        self.p("📜 QUESTS:", "bold")
        for name, q in game.quests.items():
            s = "✓" if q["status"] == "completed" else "◆" if q["status"] == "in_progress" else "○"
            c = "green" if q["status"] == "completed" else "yellow" if q["status"] == "in_progress" else "cyan"
            self.p(f"{s} {name}", c)
    
    def show_map(self):
        """Display game map with current location highlighted"""
        # Load map image if available
        self.load_img("map.png")
        
        self.p("")
        self.p("="*60, "bold")
        self.p("🗺️  PATALIPUTRA PALACE MAP", "bold")
        self.p("="*60, "bold")
        self.p("")
        
        # Create map with current location highlighted
        current = game.current_location
        
        # Map structure
        locations_map = {
            "fathers_chambers": "    [Throne Room]",
            "palace_courtyard_n": "          |",
            "palace_courtyard": "  [Palace Courtyard]",
            "mothers_chambers": "          |",
            "palace_kitchen": "[Kitchen]--+",
            "garden_pavilion": "[Garden]--+--[Stables]",
            "marketplace": "              |",
            "temple_district": "     [Temple]--[Market]"
        }
        
        # Print map with highlights
        self.p("         NORTH", "cyan")
        self.p("")
        
        # Throne Room
        if current == "fathers_chambers":
            self.p("    >>> [THRONE ROOM] <<<", "gold")
        else:
            self.p("        [Throne Room]", "cyan")
        
        self.p("              |", "cyan")
        
        # Palace Courtyard (center)
        if current == "palace_courtyard":
            self.p("                            ", "cyan")
            self.p("  [Garden]------>>> [YOU ARE HERE] <<<------[Stables]", "gold")
            self.p("                            ", "cyan")
        else:
            self.p("                            ", "cyan")
            self.p("  [Garden]--------[Courtyard]--------[Stables]", "cyan")
            self.p("                            ", "cyan")
        
        self.p("                      |                    |", "cyan")
        
        # Bottom row
        left = "              " if current != "garden_pavilion" else ">>> "
        middle_m = " [Mother's] " if current != "mothers_chambers" else " >>> [YOU] <<< "
        middle_k = "[Kitchen]" if current != "palace_kitchen" else ">>>[YOU]<<<"
        right_m = "[Market]" if current != "marketplace" else ">>>[YOU]<<<"
        right_t = "[Temple]" if current != "temple_district" else ">>>[YOU]<<<"
        right_s = " " if current != "royal_stables" else ""
        
        self.p(f"{left}          {middle_m}            {right_s}", "cyan" if current not in ["mothers_chambers", "royal_stables"] else "gold")
        self.p(f"                      |", "cyan")
        self.p(f"                 {middle_k}", "cyan" if current != "palace_kitchen" else "gold")
        self.p("")
        self.p(f"                                   {right_t}---{right_m}", "cyan" if current not in ["marketplace", "temple_district"] else "gold")
        
        self.p("")
        self.p(f"Current Location: {LOCATIONS[current]['name']}", "yellow")
        self.p("")
        self.p("="*60, "bold")
        self.p("")
    
    def show_help(self):
        self.p("COMMANDS:", "bold")
        self.p("Movement: n/s/e/w, look, map")
        self.p("Actions: talk to <npc>, inventory, quest")
        self.p("Combat: train combat (at stables)")
        self.p("Audio: music (toggle), volume <0-100>")
        self.p("Quests: help vikram, test elephant, feed <food>,")
        self.p("        remove thorn, comfort elephant, ritual,")
        self.p("        learn cooking, make cakes, pick cushion")
        self.p("Other: help, quit")

def main():
    root = tk.Tk()
    app = AshokaGame(root)
    root.mainloop()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


## version 7

In [3]:
"""
ASHOKA'S PATH - COMPLETE ACT 1
All quests, riddles, and adventures fully implemented
University of Aberdeen GameJam 2026
"""

import tkinter as tk
from tkinter import scrolledtext, font
from PIL import Image, ImageTk
import os
import pygame

# Game State with ALL quest tracking
class GameState:
    def __init__(self):
        self.current_location = "palace_courtyard"
        self.inventory = []
        self.health = 100
        self.max_health = 100
        self.riddle_solved = False
        self.game_started = False
        
        # Combat state
        self.in_combat = False
        self.combat_enemy = None
        self.combat_enemy_hp = 0
        self.combat_enemy_max_hp = 0
        self.combat_heals_remaining = 3
        self.combat_defending = False
        self.combat_training_complete = False
        
        self.quests = {
            "beggar": {"status": "not_started", "stage": 0},
            "elephant": {"status": "not_started", "stage": 0},
            "soma": {"status": "not_started", "stage": 0},
            "cooking": {"status": "not_started", "stage": 0},
            "cushion": {"status": "not_started", "stage": 0}
        }
        
        self.flags = {
            "has_golden_bowl": False,
            "vikram_paid": False,
            "elephant_started": False,
            "fed_elephant": False,
            "removed_thorn": False,
            "comforted_elephant": False,
            "learned_steps": False,
            "steps_correct": False,
            "victory": False
        }
        
        self.items_needed = ["golden_bowl", "imperial_elephant", "soma", "honey_cakes", "royal_cushion"]
        
        # Music state
        self.music_enabled = True
        self.current_music = None
        
        # Tutorial state
        self.tutorial_shown = False
        
    def has_item(self, item):
        return item in self.inventory
        
    def add_item(self, item):
        if item not in self.inventory:
            self.inventory.append(item)
            if item == "golden_bowl":
                self.flags["has_golden_bowl"] = True
            return True
        return False
            
    def check_victory(self):
        return all(item in self.inventory for item in self.items_needed)

game = GameState()

# Locations
LOCATIONS = {
    "palace_courtyard": {
        "name": "Palace Courtyard",
        "description": "Grand courtyard with marble pillars. Morning sun glints off golden domes.",
        "image": "palace_courtyard.png",
        "exits": {"north": "fathers_chambers", "south": "mothers_chambers", "east": "royal_stables", "west": "garden_pavilion"},
        "npcs": [],
        "lore": """HISTORICAL NOTE: Pataliputra (modern-day Patna)

You stand in what was once the capital of the mighty Mauryan Empire, founded by 
Chandragupta Maurya around 322 BCE. Pataliputra was one of the largest cities in 
the ancient world, rivaling Rome and Alexandria.

The city was strategically located at the confluence of the Ganges and Son rivers, 
making it a hub of trade and military power. Greek ambassador Megasthenes described 
it as having massive wooden walls stretching 9 miles long.

This palace, where young Ashoka walks, witnessed the machinations of power that 
would shape the future of South Asia. Here, succession battles were fought not with 
swords alone, but with political intrigue and divine legitimacy.

The Mauryan Empire at its height controlled nearly the entire Indian subcontinent - 
an achievement not matched until the British Raj 2,000 years later."""
    },
    "mothers_chambers": {
        "name": "Mother's Chambers",
        "description": "Gentle space with sandalwood incense. Your mother sits by the window.",
        "image": "mothers_chambers.png",
        "exits": {"north": "palace_courtyard", "west": "palace_kitchen"},
        "npcs": ["mother"],
        "lore": """HISTORICAL NOTE: Ashoka's Mother - Subhadrangi (Dharma)

Ancient texts refer to Ashoka's mother as Dharma or Subhadrangi. Unlike the chief 
queens, she held a lower rank in the royal household, which complicated Ashoka's 
claim to the throne.

In the rigidly hierarchical Mauryan court, birth order and maternal lineage mattered 
immensely. Ashoka's rough appearance and lower maternal status initially excluded 
him from consideration as heir.

According to the Ashokavadana, his mother was devoted to Buddhism even before Ashoka's 
conversion. Her influence may have planted the seeds of compassion that would later 
transform him from conqueror to peacemaker.

The relationship between royal mothers and their sons often determined the fate of 
empires. A supportive mother could mean the difference between obscurity and greatness."""
    },
    "palace_kitchen": {
        "name": "Palace Kitchen",
        "description": "Busy kitchen filled with heat and noise. Amma Savitri kneads dough.",
        "image": "palace_kitchen.png",
        "exits": {"east": "mothers_chambers"},
        "npcs": ["amma"],
        "lore": """HISTORICAL NOTE: Food and Hierarchy in Ancient India

The palace kitchen was a world unto itself, feeding hundreds daily. Ancient texts 
describe elaborate feasts with dozens of dishes - rice, lentils, vegetables, sweets 
made with honey and ghee.

Mauryan society was highly stratified, yet the kitchen represented a space where 
practical skill sometimes transcended social rank. Cooks held specialized knowledge 
passed down through generations.

Honey cakes, mentioned in ancient recipes, combined wheat flour, honey, ghee, and 
spices like cardamom. Food was not merely sustenance but carried religious and social 
significance - what you ate, with whom, and how determined your status.

Ashoka's willingness to learn cooking would have been revolutionary - a prince 
humbling himself before a cook violated every social norm of the time."""
    },
    "garden_pavilion": {
        "name": "Garden of the Golden Pavilion",
        "description": "Otherworldly space. Golden pavilion touches clouds. Ascetic meditates beneath bodhi tree.",
        "image": "garden_pavilion.png",
        "exits": {"east": "palace_courtyard"},
        "npcs": ["ascetic"],
        "lore": """HISTORICAL NOTE: The Ashokavadana and Buddhist Texts

This scene draws from the Ashokavadana, a 2nd-century CE Sanskrit text that 
chronicles Ashoka's life through a Buddhist lens. Though written centuries after 
Ashoka's death, it preserves important historical traditions.

According to the text, ascetic Pingala-vatsajiva played a crucial role in Ashoka's 
selection as heir. The ascetic's prophecy - delivered through cryptic tests rather 
than direct naming - protected Ashoka from his brothers' jealousy.

The "five sacred items" quest in this game is inspired by the Ashokavadana's 
succession story, where objects symbolized different aspects of kingship: justice 
(bowl), strength (elephant), divine favor (soma), humility (food), and authority 
(throne).

These Buddhist texts shaped how Ashoka was remembered - not just as emperor, but as 
dharmaraja (righteous king), exemplifying Buddhist ideals of compassionate governance."""
    },
    "royal_stables": {
        "name": "Royal Stables",
        "description": "Earthy smell of elephants. Minister Radhagupta stands near Airavata.",
        "image": "royal_stables.png",
        "exits": {"west": "palace_courtyard", "south": "marketplace"},
        "npcs": ["radhagupta"],
        "lore": """HISTORICAL NOTE: War Elephants and Mauryan Military Power

Elephants were the tanks of ancient warfare. The Mauryan military maintained 
thousands - Megasthenes claimed 9,000 war elephants in Chandragupta's army.

These weren't just beasts of burden. A trained war elephant, armored and carrying 
archers, could break enemy formations, trample cavalry, and inspire terror. 
Controlling an elephant required deep understanding - force alone would fail.

Minister Radhagupta, mentioned in both Buddhist and Jain texts, was Chandragupta's 
advisor and likely served Bindusara as well. His historical counterpart, Chanakya 
(also called Kautilya), wrote the Arthashastra - an ancient treatise on statecraft, 
economics, and military strategy that remains studied today.

The relationship between Ashoka and his advisors would prove crucial. Unlike his 
brothers who ruled through fear, Ashoka would learn to balance power with wisdom."""
    },
    "marketplace": {
        "name": "Marketplace",
        "description": "Chaos and color. Blind beggar sits with golden bowl. Vikram at his stall.",
        "image": "marketplace.png",
        "exits": {"north": "royal_stables", "west": "temple_district"},
        "npcs": ["beggar", "vikram"],
        "lore": """HISTORICAL NOTE: Trade and Economy in Mauryan India

Pataliputra's marketplace was a crossroads of the ancient world. Merchants traded 
silk from China, spices from South India, gems from Sri Lanka, and goods from as 
far as Greece and Persia.

The Mauryan state controlled trade through elaborate taxation and regulation. 
The Arthashastra describes market superintendents, weights and measures standards, 
and penalties for fraud - showing sophisticated economic management.

Coins bore royal insignias, establishing centralized currency. Trade routes connected 
the empire, with the Grand Trunk Road linking Pataliputra to the northwest frontier.

Social inequality was stark. Merchants could grow wealthy, but rigid caste hierarchy 
limited mobility. A blind beggar, as in our story, would have few options beyond 
charity - though Buddhist and Jain teachings increasingly emphasized compassion for 
all beings.

Ashoka's later edicts would mandate the state's responsibility to care for the poor, 
sick, and disabled - revolutionary for its time."""
    },
    "temple_district": {
        "name": "Temple District",
        "description": "Incense smoke rises. Priest Mahinda tends the sacred fire.",
        "image": "temple_district.png",
        "exits": {"east": "marketplace"},
        "npcs": ["priest"],
        "lore": """HISTORICAL NOTE: Religion in Pre-Buddhist Mauryan India

Before Ashoka's conversion, the Mauryan court practiced Brahmanism (early Hinduism). 
Rituals, sacrifices, and the Vedic tradition dominated religious life. Priests 
(Brahmins) held immense social and spiritual authority.

However, this was an era of religious ferment. Buddhism and Jainism challenged 
Brahmanical orthodoxy, offering paths to enlightenment without caste hierarchy or 
animal sacrifice. These "heterodox" movements gained followers among merchants and 
even some nobility.

Soma, mentioned in ancient texts, was a ritual drink in Vedic ceremonies - possibly 
an intoxicating plant extract. Consuming soma was believed to grant communion with 
the gods and insight into cosmic truths.

Young Ashoka, raised in orthodox traditions, would later undergo a profound spiritual 
transformation. After the bloody Kalinga War, he embraced Buddhism (dhamma) and 
dedicated his reign to moral governance, non-violence, and religious tolerance.

His son Mahinda (possibly named after this priest in our story) would become a 
Buddhist monk who spread Buddhism to Sri Lanka."""
    },
    "fathers_chambers": {
        "name": "Throne Room",
        "description": "Stern room with black stone throne. Your father reviews reports.",
        "image": "fathers_chambers.png",
        "exits": {"south": "palace_courtyard"},
        "npcs": ["emperor"],
        "lore": """HISTORICAL NOTE: Bindusara and the Succession Crisis

Emperor Bindusara (ruled 298-272 BCE), Ashoka's father, expanded the empire south. 
Known as Amitraghata ("Slayer of Enemies"), he continued Chandragupta's conquests.

Ancient sources suggest Bindusara favored Ashoka's elder brother Susima as heir. 
Ashoka's path to power was not straightforward. According to tradition, Bindusara 
sent Ashoka to suppress a rebellion in Taxila, perhaps hoping distance would remove 
him from succession politics.

When Bindusara fell ill around 272 BCE, a succession crisis erupted. The 
Ashokavadana claims Ashoka killed 99 brothers to claim the throne, though this is 
likely exaggerated. Historical evidence suggests violent conflict with at least 
Susima and possibly others.

The throne Ashoka fought for would become a platform for one of history's most 
remarkable transformations. The man who waded through blood to claim power would 
later renounce violence and spread a message of peace across Asia.

His rock edicts - carved declarations of policy and philosophy - remain visible 
today, 2,300 years later."""
    }
}

# GUI Application
class AshokaGame:
    def __init__(self, root):
        self.root = root
        self.root.title("🕉️ ASHOKA'S PATH - ACT 1")
        self.root.geometry("1000x750")
        self.root.configure(bg='#2b2b2b')
        
        self.title_font = font.Font(family="Arial", size=14, weight="bold")
        self.text_font = font.Font(family="Courier", size=11)
        
        # Initialize pygame mixer for music
        try:
            pygame.mixer.init()
            self.music_available = True
        except:
            self.music_available = False
            self.p("Music system unavailable", "yellow")
        
        self.setup_ui()
        self.play_music("menu")  # Play menu music
        self.show_riddle()
        
    def setup_ui(self):
        # Image frame
        self.image_frame = tk.Frame(self.root, bg='#1a1a1a', height=220)
        self.image_frame.pack(fill=tk.X, padx=10, pady=5)
        self.image_frame.pack_propagate(False)
        self.image_label = tk.Label(self.image_frame, bg='#1a1a1a', fg='#4a9eff')
        self.image_label.pack(expand=True)
        
        # Location label
        self.location_label = tk.Label(self.root, text="", font=self.title_font, bg='#2b2b2b', fg='#4a9eff', pady=5)
        self.location_label.pack()
        
        # Text output
        self.output = scrolledtext.ScrolledText(self.root, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', 
                                                insertbackground='#4a9eff', wrap=tk.WORD, height=18)
        self.output.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Tags
        self.output.tag_configure("green", foreground="#4ade80")
        self.output.tag_configure("yellow", foreground="#fbbf24")
        self.output.tag_configure("red", foreground="#f87171")
        self.output.tag_configure("cyan", foreground="#22d3ee")
        self.output.tag_configure("magenta", foreground="#e879f9")
        self.output.tag_configure("gold", foreground="#fbbf24")
        self.output.tag_configure("bold", font=self.title_font)
        
        # Input
        input_frame = tk.Frame(self.root, bg='#2b2b2b')
        input_frame.pack(fill=tk.X, padx=10, pady=5)
        tk.Label(input_frame, text=">", font=self.text_font, bg='#2b2b2b', fg='#4ade80').pack(side=tk.LEFT, padx=(0, 5))
        self.input_entry = tk.Entry(input_frame, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', insertbackground='#4a9eff')
        self.input_entry.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.input_entry.bind('<Return>', self.process_cmd)
        
    def p(self, text, tag=None):
        self.output.insert(tk.END, text + "\n", tag)
        self.output.see(tk.END)
        
    def clear(self):
        self.output.delete(1.0, tk.END)
        
    def load_img(self, img):
        if not img:
            return
        path = f"images/{img}"
        if os.path.exists(path):
            try:
                im = Image.open(path)
                im.thumbnail((900, 210), Image.Resampling.LANCZOS)
                photo = ImageTk.PhotoImage(im)
                self.image_label.configure(image=photo, text="")
                self.image_label.image = photo
            except:
                self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
        else:
            self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
    
    # ========== MUSIC SYSTEM ==========
    
    def play_music(self, track_name):
        """Play background music"""
        if not self.music_available or not game.music_enabled:
            return
        
        music_files = {
            "menu": "music/menu.mp3",
            "exploration": "music/exploration.mp3",
            "battle": "music/battle.mp3",
            "victory": "music/victory.mp3"
        }
        
        if track_name not in music_files:
            return
        
        music_path = music_files[track_name]
        
        # Only change if different track
        if game.current_music == track_name:
            return
        
        try:
            if os.path.exists(music_path):
                pygame.mixer.music.load(music_path)
                pygame.mixer.music.set_volume(0.5)  # 50% volume
                
                # Loop for exploration/battle, play once for victory
                if track_name in ["exploration", "battle"]:
                    pygame.mixer.music.play(-1)  # Loop indefinitely
                else:
                    pygame.mixer.music.play(0)   # Play once
                
                game.current_music = track_name
        except Exception as e:
            pass  # Silently fail if music file missing
    
    def stop_music(self):
        """Stop all music"""
        if self.music_available:
            try:
                pygame.mixer.music.stop()
                game.current_music = None
            except:
                pass
    
    def toggle_music(self):
        """Toggle music on/off"""
        game.music_enabled = not game.music_enabled
        
        if game.music_enabled:
            self.p("🎵 Music: ON", "green")
            # Resume appropriate music
            if game.in_combat:
                self.play_music("battle")
            else:
                self.play_music("exploration")
        else:
            self.p("🔇 Music: OFF", "yellow")
            self.stop_music()
    
    def set_volume(self, volume):
        """Set music volume (0.0 to 1.0)"""
        if self.music_available:
            try:
                pygame.mixer.music.set_volume(max(0.0, min(1.0, volume)))
                self.p(f"🔊 Volume: {int(volume*100)}%", "cyan")
            except:
                pass
    
    # ========== END MUSIC SYSTEM ==========
    
    def show_riddle(self):
        self.input_entry.config(state='disabled')
        self.p("="*80, "bold")
        self.p("🕉️  ASHOKA'S PATH - ACT 1  🕉️", "bold")
        self.p("="*80, "bold")
        self.p("")
        self.p("An ancient ascetic appears...", "gold")
        self.p("")
        self.p("'Before you walk Ashoka's path, answer this:'")
        self.p("  I am won through blood, yet bring no peace")
        self.p("  I am sought by many, yet poison few")
        self.p("  I sit on heads, yet crush many souls")
        self.p("")
        self.p("  [1] The Crown", "cyan")
        self.p("  [2] A Sword", "cyan")
        self.p("  [3] Gold", "cyan")
        self.p("  [4] Power", "cyan")
        self.p("")
        self.input_entry.config(state='normal')
        self.input_entry.focus()
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_riddle)
    
    def check_riddle(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        if ans in ["1", "4"]:
            self.p(f"> {ans}", "cyan")
            self.p("")
            self.p("'Wisdom beyond your years. You may proceed...'", "green")
            self.p("")
            game.riddle_solved = True
            game.game_started = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.start_game()
        else:
            self.p(f"> {ans}", "cyan")
            self.p("'Think deeper...'", "red")
            self.p("")
    
    def start_game(self):
        self.play_music("exploration")  # Start exploration music
        self.p("="*80)
        self.p("Find FIVE sacred items:", "yellow")
        self.p("  • Vessel of compassion", "magenta")
        self.p("  • Mount of strength", "magenta")
        self.p("  • Drink of gods", "magenta")
        self.p("  • Food with humility", "magenta")
        self.p("  • Seat through courage", "magenta")
        self.p("")
        self.p("⭐ NEW PLAYER? Type 'tutorial' for detailed guide!", "gold")
        self.p("📜 Type 'lore' at any location for historical facts!", "gold")
        self.p("")
        self.p("Quick start: 'help' for commands | 'map' to see palace", "cyan")
        self.p("="*80)
        self.show_loc()
    
    def show_loc(self):
        loc = LOCATIONS[game.current_location]
        self.load_img(loc["image"])
        self.location_label.configure(text=loc["name"])
        self.p("")
        self.p(loc["description"])
        self.p("")
        exits = ", ".join([d.upper() for d in loc["exits"]])
        self.p(f"Exits: {exits}", "yellow")
        self.p("")
        if loc["npcs"]:
            self.p(f"People: {', '.join(loc['npcs'])}", "green")
            self.p("")
        # Show cushion if ready
        if game.current_location == "fathers_chambers" and len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
            self.p("You see: Royal Cushion", "magenta")
            self.p("")
    
    def process_cmd(self, e=None):
        if not game.game_started:
            return
        cmd = self.input_entry.get().lower().strip()
        self.input_entry.delete(0, tk.END)
        if not cmd:
            return
        
        # Combat mode - different command handling
        if game.in_combat:
            self.combat_action(cmd)
            return
        
        self.p(f"> {cmd}", "cyan")
        self.p("")
        
        if cmd in ["n","north"]: self.move("north")
        elif cmd in ["s","south"]: self.move("south")
        elif cmd in ["e","east"]: self.move("east")
        elif cmd in ["w","west"]: self.move("west")
        elif cmd in ["l","look"]: self.clear(); self.show_loc()
        elif cmd in ["i","inv","inventory"]: self.show_inv()
        elif cmd in ["q","quest","quests"]: self.show_quest()
        elif cmd in ["h","health"]: self.p(f"❤️ {game.health}/100", "green")
        elif cmd in ["?","help"]: self.show_help()
        elif cmd in ["m","map"]: self.show_map()
        elif cmd in ["music","toggle music"]: self.toggle_music()
        elif cmd in ["tutorial","tut"]: self.show_tutorial()
        elif cmd in ["lore","history","info"]: self.show_lore()
        elif cmd.startswith("volume "):
            try:
                vol = float(cmd.split()[1]) / 100.0
                self.set_volume(vol)
            except:
                self.p("Usage: volume <0-100>", "red")
        elif cmd in ["quit","exit"]: self.p("Thanks for playing!","yellow"); self.root.after(2000, self.root.quit)
        elif cmd.startswith("talk "): self.talk(cmd.replace("talk to ","").replace("talk ",""))
        elif cmd == "help vikram": self.help_vikram()
        elif cmd == "test elephant": self.test_elephant()
        elif cmd.startswith("feed "): self.feed(cmd.split()[1])
        elif cmd == "remove thorn": self.remove_thorn()
        elif cmd == "comfort elephant": self.comfort()
        elif cmd == "ritual": self.ritual()
        elif cmd == "learn cooking": self.learn_cook()
        elif cmd == "make cakes": self.make_cakes()
        elif cmd == "train combat" or cmd == "spar" or cmd == "train": self.train_combat()
        elif cmd.startswith("pick ") or cmd.startswith("get ") or cmd.startswith("take "): self.pick(cmd.split(maxsplit=1)[1])
        else: self.p("Unknown command. Type 'help'", "red")
        
        self.p("")
        if game.check_victory() and not game.flags["victory"]:
            self.p("="*70, "gold")
            self.p("✨ ALL FIVE ITEMS COLLECTED! ✨", "gold")
            self.p("="*70, "gold")
            self.p("Return to GARDEN and talk to ascetic!", "yellow")
            self.p("")
    
    def move(self, dir):
        loc = LOCATIONS[game.current_location]
        if dir in loc["exits"]:
            next_loc = loc["exits"][dir]
            # Throne room logic
            if next_loc == "fathers_chambers":
                if len(game.inventory) < 4:
                    # Still too early - instant death
                    self.die()
                    return
                elif len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
                    # Ready for Susima battle!
                    game.current_location = next_loc
                    self.p(f"You travel {dir}...", "cyan")
                    self.p("")
                    self.start_susima_battle()
                    return
            
            game.current_location = next_loc
            self.p(f"You travel {dir}...", "cyan")
            self.p("")
            self.show_loc()
        else:
            self.p("Can't go that way!", "red")
    
    def die(self):
        self.clear()
        self.p("="*70, "red")
        self.p("DISCOVERED BY SUSIMA!", "red")
        self.p("="*70, "red")
        self.p("Your brother accuses you of treason!")
        self.p("YOU DIED", "red")
        self.p("Collect 4+ items first. Restarting...", "yellow")
        self.input_entry.config(state='disabled')
        self.root.after(5000, self.restart)
    
    def restart(self):
        global game
        game = GameState()
        self.clear()
        self.input_entry.config(state='normal')
        self.show_riddle()
    
    def pick(self, item):
        item = item.lower().replace(" ", "_")
        if item == "royal_cushion":
            if game.current_location != "fathers_chambers":
                self.p("Not here", "red")
                return
            if len(game.inventory) < 4:
                self.p("Too dangerous. Need 4+ items first", "red")
                return
            if game.has_item("royal_cushion"):
                self.p("Already have it", "yellow")
                return
            game.add_item("royal_cushion")
            game.quests["cushion"]["status"] = "completed"
            self.p("✓ Acquired: Royal Cushion (5/5)", "gold")
            return
        self.p("Items earned through quests!", "yellow")
    
    def talk(self, npc):
        loc = LOCATIONS[game.current_location]
        if npc not in loc["npcs"]:
            self.p("Nobody by that name here", "red")
            return
        
        if npc == "beggar": self.talk_beggar()
        elif npc == "vikram": self.talk_vikram()
        elif npc == "radhagupta": self.talk_radha()
        elif npc == "priest": self.talk_priest()
        elif npc == "amma": self.talk_amma()
        elif npc == "ascetic": self.talk_ascetic()
        else: self.p(f"{npc} says: Hello", "green")
    
    def talk_beggar(self):
        q = game.quests["beggar"]
        if q["stage"] == 0:
            self.p("DHRUVA:", "green")
            self.p("Vikram owes me 20 gold. Help me get it?")
            self.p("(Try: 'talk to vikram' then 'help vikram')", "yellow")
            q["status"] = "in_progress"
            q["stage"] = 1
        elif q["stage"] == 1:
            self.p("DHRUVA: Spoken to Vikram yet?")
        elif q["stage"] == 2 and game.flags["vikram_paid"]:
            self.p("DHRUVA: Thank you! Take my bowl.")
            game.add_item("golden_bowl")
            q["status"] = "completed"
            q["stage"] = 3
            self.p("✓ Quest Complete! Golden Bowl (1/5)", "gold")
        else:
            self.p("DHRUVA: Thanks again.")
    
    def talk_vikram(self):
        if game.quests["beggar"]["stage"] < 1:
            self.p("VIKRAM: I'm busy.")
            return
        self.p("VIKRAM: I owe him nothing!")
        self.p("(Try: 'help vikram')", "yellow")
    
    def help_vikram(self):
        if game.quests["beggar"]["stage"] != 1:
            return
        self.p("You check his ledgers... Found it!")
        self.p("VIKRAM: Fine, take it!")
        game.flags["vikram_paid"] = True
        game.quests["beggar"]["stage"] = 2
        self.p("Return to beggar", "yellow")
    
    def talk_radha(self):
        self.p("RADHAGUPTA:", "green")
        self.p("Test yourself with Airavata?")
        if not game.combat_training_complete:
            self.p("")
            self.p("Or would you like combat training?", "yellow")
            self.p("(Try: 'test elephant' or 'train combat')", "yellow")
        else:
            self.p("(Try: 'test elephant')", "yellow")
    
    def test_elephant(self):
        if game.current_location != "royal_stables":
            return
        if game.quests["elephant"]["status"] == "completed":
            self.p("Already done", "yellow")
            return
        game.quests["elephant"]["status"] = "in_progress"
        game.flags["elephant_started"] = True
        self.p("PART 1: Feed Airavata")
        self.p("Options: hay, mangoes, sugarcane")
        self.p("(Try: 'feed hay')", "yellow")
    
    def feed(self, food):
        if not game.flags["elephant_started"] or game.flags["fed_elephant"]:
            return
        if food == "hay":
            self.p("Airavata accepts gratefully")
            game.flags["fed_elephant"] = True
            self.p("PART 2: Remove thorn", "yellow")
            self.p("(Try: 'remove thorn')", "yellow")
        else:
            self.p("Wrong choice", "red")
    
    def remove_thorn(self):
        if not game.flags["fed_elephant"] or game.flags["removed_thorn"]:
            return
        self.p("You carefully remove the thorn...")
        game.flags["removed_thorn"] = True
        self.p("PART 3: Comfort elephant", "yellow")
        self.p("(Try: 'comfort elephant')", "yellow")
    
    def comfort(self):
        if not game.flags["removed_thorn"]:
            return
        self.p("You speak of loneliness...")
        self.p("Airavata touches your face")
        game.flags["comforted_elephant"] = True
        game.add_item("imperial_elephant")
        game.quests["elephant"]["status"] = "completed"
        self.p("✓ Quest Complete! Imperial Elephant (2/5)", "gold")
    
    def talk_priest(self):
        self.p("MAHINDA:", "green")
        if not game.flags["has_golden_bowl"]:
            self.p("Need golden bowl first")
        else:
            self.p("Ready for ritual?")
            self.p("(Try: 'ritual')", "yellow")
    
    def ritual(self):
        if game.current_location != "temple_district":
            return
        if not game.flags["has_golden_bowl"]:
            return
        if game.quests["soma"]["status"] == "completed":
            return
        self.p("You complete the sacred ritual...")
        self.p("Vision: Two paths - war and peace")
        game.add_item("soma")
        game.quests["soma"]["status"] = "completed"
        self.p("✓ Quest Complete! Soma (3/5)", "gold")
    
    def talk_amma(self):
        self.p("AMMA:", "green")
        self.p("Want to learn cooking?")
        self.p("(Try: 'learn cooking')", "yellow")
    
    def learn_cook(self):
        if game.current_location != "palace_kitchen":
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        game.quests["cooking"]["status"] = "in_progress"
        game.flags["learned_steps"] = True
        self.p("AMMA: 5 steps:")
        self.p("1. Mix flour and ghee")
        self.p("2. Add honey slowly")
        self.p("3. Fold in cardamom")
        self.p("4. Shape into rounds")
        self.p("5. Bake until golden")
        self.p("")
        self.p("Repeat the order. Type: 1 2 3 4 5")
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_steps)
    
    def check_steps(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        correct = ["1 2 3 4 5", "12345", "1,2,3,4,5", "1, 2, 3, 4, 5"]
        if ans in correct:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Perfect!")
            game.flags["steps_correct"] = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.p("(Try: 'make cakes')", "yellow")
            self.p("")
        else:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Wrong! Come back.", "red")
            game.quests["cooking"]["status"] = "not_started"
            game.flags["learned_steps"] = False
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
    
    def make_cakes(self):
        if not game.flags["steps_correct"]:
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        self.p("You make honey cakes with care...")
        self.p("AMMA: Remember this humility")
        game.add_item("honey_cakes")
        game.quests["cooking"]["status"] = "completed"
        self.p("✓ Quest Complete! Honey Cakes (4/5)", "gold")
    
    def talk_ascetic(self):
        if not game.check_victory():
            self.p("ASCETIC: Return with all five items")
            return
        if game.flags["victory"]:
            return
        self.victory()
    
    def victory(self):
        game.flags["victory"] = True
        self.play_music("victory")  # Play victory music
        self.clear()
        self.load_img("coronation.png")
        self.p("")
        self.p("="*70, "gold")
        self.p("ACT 1 COMPLETE", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("You are crowned EMPEROR ASHOKA")
        self.p("")
        self.p("="*70, "gold")
        self.p("TO BE CONTINUED IN ACT 2...", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("Thank you for playing!", "cyan")
    
    # ========== COMBAT SYSTEM ==========
    
    def train_combat(self):
        """Start optional training battle"""
        if game.current_location != "royal_stables":
            self.p("Must be at Royal Stables to train", "red")
            return
        
        if game.combat_training_complete:
            self.p("RADHAGUPTA: You've already mastered combat training.", "green")
            return
        
        self.p("RADHAGUPTA: You wish to learn combat?", "green")
        self.p("RADHAGUPTA: Very well. Defend yourself!")
        self.p("")
        self.init_combat("Training Dummy", 60, is_training=True)
    
    def start_susima_battle(self):
        """Start required Susima boss battle"""
        self.clear()
        self.load_img("fathers_chambers.png")
        self.p("")
        self.p("="*70, "red")
        self.p("⚔️  BROTHER SUSIMA BLOCKS YOUR PATH!", "red")
        self.p("="*70, "red")
        self.p("")
        self.p("SUSIMA: 'Ashoka! You dare approach the throne?'")
        self.p("SUSIMA: 'I see you've been collecting... trinkets.'")
        self.p("SUSIMA: 'Let's see if you can fight as well as you scheme!'")
        self.p("")
        self.p("Susima draws his sword!")
        self.p("")
        self.init_combat("Brother Susima", 100, is_training=False)
    
    def init_combat(self, enemy_name, enemy_hp, is_training=False):
        """Initialize combat state"""
        import random
        self.play_music("battle")  # Play battle music
        game.in_combat = True
        game.combat_enemy = enemy_name
        game.combat_enemy_hp = enemy_hp
        game.combat_enemy_max_hp = enemy_hp
        game.combat_heals_remaining = 3
        game.combat_defending = False
        game.combat_is_training = is_training
        
        self.show_combat_status()
    
    def show_combat_status(self):
        """Display combat UI"""
        import random
        
        # Player health bar
        player_bar_filled = int((game.health / game.max_health) * 10)
        player_bar = "█" * player_bar_filled + "░" * (10 - player_bar_filled)
        
        # Enemy health bar
        enemy_bar_filled = int((game.combat_enemy_hp / game.combat_enemy_max_hp) * 10)
        enemy_bar = "█" * enemy_bar_filled + "░" * (10 - enemy_bar_filled)
        
        self.p("")
        self.p("="*60, "bold")
        self.p(f"⚔️  BATTLE: You vs {game.combat_enemy}", "bold")
        self.p("="*60, "bold")
        self.p("")
        self.p(f"YOU:    {player_bar} {game.health}/{game.max_health} HP | Heals: {game.combat_heals_remaining}/3", "green")
        self.p(f"{game.combat_enemy.upper()}: {enemy_bar} {game.combat_enemy_hp}/{game.combat_enemy_max_hp} HP", "red")
        self.p("")
        self.p("Your turn! What will you do?", "yellow")
        self.p("  [1] ATTACK      - Strike (15-25 damage)", "cyan")
        self.p("  [2] DEFEND      - Guard (50% damage reduction next turn)", "cyan")
        self.p("  [3] POWER ATK   - Heavy strike (25-35 dmg, 30% miss)", "cyan")
        self.p("  [4] HEAL        - Restore 25 HP", "cyan")
        self.p("")
    
    def combat_action(self, cmd):
        """Process combat commands"""
        import random
        
        if cmd in ["1", "attack", "atk"]:
            self.player_attack()
        elif cmd in ["2", "defend", "def", "guard"]:
            self.player_defend()
        elif cmd in ["3", "power", "power attack", "power atk"]:
            self.player_power_attack()
        elif cmd in ["4", "heal"]:
            self.player_heal()
        else:
            self.p("Invalid action! Choose 1-4", "red")
            self.p("")
            return
        
        # Check if enemy dead
        if self.check_combat_end():
            return
        
        # Enemy turn
        self.enemy_turn()
        
        # Check if player dead
        if self.check_combat_end():
            return
        
        # Next round
        self.show_combat_status()
    
    def player_attack(self):
        """Player normal attack"""
        import random
        damage = random.randint(15, 25)
        game.combat_enemy_hp -= damage
        game.combat_defending = False
        
        self.p(f"> ATTACK", "cyan")
        self.p("")
        self.p(f"You strike {game.combat_enemy}!", "green")
        self.p(f"💥 Dealt {damage} damage!", "green")
        self.p("")
    
    def player_defend(self):
        """Player defend"""
        game.combat_defending = True
        
        self.p(f"> DEFEND", "cyan")
        self.p("")
        self.p("You raise your guard defensively!", "green")
        self.p("🛡️  Next attack will be reduced by 50%!", "green")
        self.p("")
    
    def player_power_attack(self):
        """Player power attack with miss chance"""
        import random
        game.combat_defending = False
        
        self.p(f"> POWER ATTACK", "cyan")
        self.p("")
        
        if random.random() < 0.3:  # 30% miss chance
            self.p("Your heavy strike misses!", "red")
            self.p("You're off balance!", "red")
            self.p("")
        else:
            damage = random.randint(25, 35)
            game.combat_enemy_hp -= damage
            self.p(f"CRITICAL HIT!", "gold")
            self.p(f"💥💥 Dealt {damage} damage!", "gold")
            self.p("")
    
    def player_heal(self):
        """Player heal"""
        game.combat_defending = False
        
        self.p(f"> HEAL", "cyan")
        self.p("")
        
        if game.combat_heals_remaining <= 0:
            self.p("No heals remaining!", "red")
            self.p("You waste your turn!", "red")
            self.p("")
            return
        
        heal_amount = 25
        old_health = game.health
        game.health = min(game.health + heal_amount, game.max_health)
        actual_heal = game.health - old_health
        game.combat_heals_remaining -= 1
        
        self.p(f"You drink a healing potion!", "green")
        self.p(f"❤️  Restored {actual_heal} HP!", "green")
        self.p("")
    
    def enemy_turn(self):
        """Enemy AI turn"""
        import random
        
        # Simple AI: Random attack or power attack
        action = random.choice(["attack", "attack", "power"])  # 66% attack, 33% power
        
        if action == "attack":
            damage = random.randint(10, 20) if game.combat_is_training else random.randint(15, 25)
            
            if game.combat_defending:
                damage = damage // 2
                self.p(f"{game.combat_enemy} attacks!", "yellow")
                self.p(f"🛡️  Your guard blocks half the damage!", "green")
                self.p(f"💔 You take {damage} damage!", "red")
            else:
                self.p(f"{game.combat_enemy} attacks!", "yellow")
                self.p(f"💔 You take {damage} damage!", "red")
            
            game.health -= damage
            game.combat_defending = False
            
        elif action == "power":
            if random.random() < 0.4:  # 40% miss for enemy power
                self.p(f"{game.combat_enemy} attempts a heavy strike but misses!", "green")
            else:
                damage = random.randint(20, 30) if game.combat_is_training else random.randint(25, 35)
                
                if game.combat_defending:
                    damage = damage // 2
                    self.p(f"{game.combat_enemy} unleashes a POWER ATTACK!", "red")
                    self.p(f"🛡️  Your guard reduces the damage!", "green")
                    self.p(f"💔💔 You take {damage} damage!", "red")
                else:
                    self.p(f"{game.combat_enemy} unleashes a POWER ATTACK!", "red")
                    self.p(f"💔💔 You take {damage} damage!", "red")
                
                game.health -= damage
            
            game.combat_defending = False
        
        self.p("")
    
    def check_combat_end(self):
        """Check if combat is over"""
        if game.combat_enemy_hp <= 0:
            # Player wins!
            self.play_music("victory")  # Victory music
            self.p("="*60, "gold")
            self.p("⭐ VICTORY! ⭐", "gold")
            self.p("="*60, "gold")
            self.p("")
            
            if game.combat_is_training:
                self.p("RADHAGUPTA: 'Excellent! You have mastered combat!'", "green")
                self.p("RADHAGUPTA: 'You are ready for real battles now.'", "green")
                game.combat_training_complete = True
            else:
                self.p("Susima falls to his knees, defeated!", "green")
                self.p("")
                self.p("SUSIMA: 'Impossible... how did you...'", "yellow")
                self.p("")
                self.p("You stand victorious. The throne room is yours.", "green")
                self.p("")
                self.p("The royal cushion sits on the throne, waiting...", "magenta")
                self.p("")
            
            game.in_combat = False
            self.p("")
            
            # Return to exploration music after a delay
            self.root.after(3000, lambda: self.play_music("exploration"))
            
            if not game.combat_is_training:
                self.show_loc()
            
            return True
        
        if game.health <= 0:
            # Player dies!
            self.stop_music()  # Stop music on death
            self.p("="*60, "red")
            self.p("💀 DEFEAT 💀", "red")
            self.p("="*60, "red")
            self.p("")
            
            if game.combat_is_training:
                self.p("RADHAGUPTA: 'You fought well, but need more training.'", "yellow")
                self.p("RADHAGUPTA: 'Rest and try again when ready.'", "yellow")
                game.health = game.max_health  # Restore health after training
                game.in_combat = False
                self.play_music("exploration")  # Back to exploration
            else:
                self.p("Susima strikes you down!", "red")
                self.p("")
                self.p("YOU DIED", "red")
                self.p("")
                self.p("Restarting in 5 seconds...", "yellow")
                game.in_combat = False
                self.input_entry.config(state='disabled')
                self.root.after(5000, self.restart)
            
            self.p("")
            return True
        
        return False
    
    # ========== END COMBAT SYSTEM ==========
    
    # ========== TUTORIAL SYSTEM ==========
    
    def show_tutorial(self):
        """Display comprehensive tutorial"""
        self.clear()
        self.p("="*70, "bold")
        self.p("📖 ASHOKA'S PATH - COMPLETE TUTORIAL", "bold")
        self.p("="*70, "bold")
        self.p("")
        
        self.p("🎯 OBJECTIVE:", "gold")
        self.p("Collect FIVE sacred items to prove your worth as heir to the")
        self.p("Mauryan throne. Each item tests a different virtue.")
        self.p("")
        
        self.p("="*70, "cyan")
        self.p("MOVEMENT COMMANDS", "cyan")
        self.p("="*70, "cyan")
        self.p("  north, n       - Move north")
        self.p("  south, s       - Move south")
        self.p("  east, e        - Move east")
        self.p("  west, w        - Move west")
        self.p("")
        self.p("  look, l        - Re-examine current location")
        self.p("  map, m         - View palace map with your position")
        self.p("")
        
        self.p("="*70, "green")
        self.p("INFORMATION COMMANDS", "green")
        self.p("="*70, "green")
        self.p("  inventory, i   - Check items you've collected (X/5)")
        self.p("  quest, q       - View quest progress and active quests")
        self.p("  health, h      - Check your current health (combat)")
        self.p("  lore, history  - Read historical facts about location")
        self.p("  tutorial, tut  - Show this tutorial anytime")
        self.p("  help, ?        - Quick command reference")
        self.p("")
        
        self.p("="*70, "yellow")
        self.p("INTERACTION COMMANDS", "yellow")
        self.p("="*70, "yellow")
        self.p("  talk to <npc>  - Speak with characters")
        self.p("                   Example: 'talk to beggar'")
        self.p("")
        self.p("  pick <item>    - Pick up items (when available)")
        self.p("  get <item>     - Same as pick")
        self.p("  take <item>    - Same as pick")
        self.p("")
        
        self.p("="*70, "magenta")
        self.p("QUEST-SPECIFIC COMMANDS", "magenta")
        self.p("="*70, "magenta")
        self.p("Golden Bowl Quest (Marketplace):")
        self.p("  help vikram    - Investigate merchant's ledgers")
        self.p("")
        self.p("Elephant Quest (Royal Stables):")
        self.p("  test elephant  - Begin Airavata's worthiness test")
        self.p("  feed <food>    - Feed elephant (hay/mangoes/sugarcane)")
        self.p("  remove thorn   - Help elephant with thorn")
        self.p("  comfort elephant - Show compassion to Airavata")
        self.p("")
        self.p("Soma Quest (Temple District):")
        self.p("  ritual         - Begin sacred soma ritual")
        self.p("                   (Requires golden bowl first!)")
        self.p("")
        self.p("Cooking Quest (Palace Kitchen):")
        self.p("  learn cooking  - Learn honey cake recipe")
        self.p("  make cakes     - Cook the honey cakes")
        self.p("")
        self.p("Cushion Quest (Throne Room):")
        self.p("  pick cushion   - Take royal cushion")
        self.p("                   (Need 4+ items or death!)")
        self.p("")
        
        self.p("="*70, "red")
        self.p("COMBAT COMMANDS", "red")
        self.p("="*70, "red")
        self.p("  train combat   - Optional training at Royal Stables")
        self.p("                   (No death penalty, learn safely)")
        self.p("")
        self.p("During Battle (choose 1-4):")
        self.p("  1, attack      - Normal attack (15-25 damage)")
        self.p("  2, defend      - Guard (50% damage reduction)")
        self.p("  3, power       - Power attack (25-35 dmg, 30% miss)")
        self.p("  4, heal        - Restore 25 HP (3 uses per battle)")
        self.p("")
        
        self.p("="*70, "cyan")
        self.p("AUDIO COMMANDS", "cyan")
        self.p("="*70, "cyan")
        self.p("  music          - Toggle music on/off")
        self.p("  volume <0-100> - Set volume (0=mute, 100=max)")
        self.p("                   Example: 'volume 50'")
        self.p("")
        
        self.p("="*70, "gold")
        self.p("💡 IMPORTANT TIPS", "gold")
        self.p("="*70, "gold")
        self.p("")
        self.p("📍 QUEST ORDER:", "yellow")
        self.p("  1. Golden Bowl (beggar) - Unlocks soma ritual")
        self.p("  2. Imperial Elephant (stables) - Test of compassion")
        self.p("  3. Honey Cakes (kitchen) - Test of humility")
        self.p("  4. Soma (temple) - Needs bowl, vision quest")
        self.p("  5. Royal Cushion (throne) - Needs 4+ items!")
        self.p("")
        
        self.p("⚠️  DANGER ZONES:", "red")
        self.p("  • Throne Room with <4 items = INSTANT DEATH")
        self.p("  • Throne Room with 4+ items = BOSS BATTLE")
        self.p("  • Losing to Susima = GAME RESTART")
        self.p("  • Training battle = Safe practice (no penalty)")
        self.p("")
        
        self.p("✨ STRATEGY:", "green")
        self.p("  • Talk to everyone - NPCs give quest hints")
        self.p("  • Read lore at locations for immersion")
        self.p("  • Do training battle before Susima fight")
        self.p("  • Collect 4 items before throne room")
        self.p("  • Save heals for emergencies in combat")
        self.p("  • Return to Garden after all 5 items")
        self.p("")
        
        self.p("="*70, "bold")
        self.p("Press any key to continue...", "yellow")
        self.p("="*70, "bold")
        
        game.tutorial_shown = True
    
    def show_lore(self):
        """Display historical lore for current location"""
        location = LOCATIONS[game.current_location]
        
        if "lore" not in location:
            self.p("No historical information available here.", "yellow")
            return
        
        self.p("")
        self.p("="*70, "gold")
        self.p("📜 HISTORICAL LORE", "gold")
        self.p("="*70, "gold")
        self.p("")
        
        # Print lore with proper formatting
        lore_lines = location["lore"].split("\n")
        for line in lore_lines:
            if line.strip():
                self.p(line, "cyan" if line.startswith("HISTORICAL") else None)
            else:
                self.p("")
        
        self.p("")
        self.p("="*70, "gold")
    
    # ========== END TUTORIAL SYSTEM ==========
    
    def show_inv(self):
        if not game.inventory:
            self.p("Empty", "yellow")
        else:
            self.p("📦 INVENTORY:", "bold")
            for i, item in enumerate(game.inventory, 1):
                self.p(f"  {i}. {item.replace('_', ' ').title()}", "green")
            self.p(f"{len(game.inventory)}/5 items", "yellow")
    
    def show_quest(self):
        self.p("📜 QUESTS:", "bold")
        for name, q in game.quests.items():
            s = "✓" if q["status"] == "completed" else "◆" if q["status"] == "in_progress" else "○"
            c = "green" if q["status"] == "completed" else "yellow" if q["status"] == "in_progress" else "cyan"
            self.p(f"{s} {name}", c)
    
    def show_map(self):
        """Display game map with current location highlighted"""
        # Load map image if available
        self.load_img("map.png")
        
        self.p("")
        self.p("="*60, "bold")
        self.p("🗺️  PATALIPUTRA PALACE MAP", "bold")
        self.p("="*60, "bold")
        self.p("")
        
        # Create map with current location highlighted
        current = game.current_location
        
        # Map structure
        locations_map = {
            "fathers_chambers": "    [Throne Room]",
            "palace_courtyard_n": "          |",
            "palace_courtyard": "  [Palace Courtyard]",
            "mothers_chambers": "          |",
            "palace_kitchen": "[Kitchen]--+",
            "garden_pavilion": "[Garden]--+--[Stables]",
            "marketplace": "              |",
            "temple_district": "     [Temple]--[Market]"
        }
        
        # Print map with highlights
        self.p("         NORTH", "cyan")
        self.p("")
        
        # Throne Room
        if current == "fathers_chambers":
            self.p("    >>> [THRONE ROOM] <<<", "gold")
        else:
            self.p("        [Throne Room]", "cyan")
        
        self.p("              |", "cyan")
        
        # Palace Courtyard (center)
        if current == "palace_courtyard":
            self.p("                            ", "cyan")
            self.p("  [Garden]------>>> [YOU ARE HERE] <<<------[Stables]", "gold")
            self.p("                            ", "cyan")
        else:
            self.p("                            ", "cyan")
            self.p("  [Garden]--------[Courtyard]--------[Stables]", "cyan")
            self.p("                            ", "cyan")
        
        self.p("                      |                    |", "cyan")
        
        # Bottom row
        left = "              " if current != "garden_pavilion" else ">>> "
        middle_m = " [Mother's] " if current != "mothers_chambers" else " >>> [YOU] <<< "
        middle_k = "[Kitchen]" if current != "palace_kitchen" else ">>>[YOU]<<<"
        right_m = "[Market]" if current != "marketplace" else ">>>[YOU]<<<"
        right_t = "[Temple]" if current != "temple_district" else ">>>[YOU]<<<"
        right_s = " " if current != "royal_stables" else ""
        
        self.p(f"{left}          {middle_m}            {right_s}", "cyan" if current not in ["mothers_chambers", "royal_stables"] else "gold")
        self.p(f"                      |", "cyan")
        self.p(f"                 {middle_k}", "cyan" if current != "palace_kitchen" else "gold")
        self.p("")
        self.p(f"                                   {right_t}---{right_m}", "cyan" if current not in ["marketplace", "temple_district"] else "gold")
        
        self.p("")
        self.p(f"Current Location: {LOCATIONS[current]['name']}", "yellow")
        self.p("")
        self.p("="*60, "bold")
        self.p("")
    
    def show_help(self):
        self.p("COMMANDS:", "bold")
        self.p("Movement: n/s/e/w, look, map")
        self.p("Info: inventory, quest, lore, tutorial")
        self.p("Actions: talk to <npc>, pick <item>")
        self.p("Combat: train combat (at stables)")
        self.p("Audio: music (toggle), volume <0-100>")
        self.p("Quests: help vikram, test elephant, feed <food>,")
        self.p("        remove thorn, comfort elephant, ritual,")
        self.p("        learn cooking, make cakes")
        self.p("")
        self.p("Type 'tutorial' for detailed guide!", "yellow")
        self.p("Type 'lore' for historical facts!", "yellow")

def main():
    root = tk.Tk()
    app = AshokaGame(root)
    root.mainloop()

if __name__ == "__main__":
    main()

# version 8 

In [2]:
"""
ASHOKA'S PATH - COMPLETE ACT 1
All quests, riddles, and adventures fully implemented
University of Aberdeen GameJam 2026
"""

import tkinter as tk
from tkinter import scrolledtext, font
from PIL import Image, ImageTk
import os
import pygame

# Game State with ALL quest tracking
class GameState:
    def __init__(self):
        self.current_location = "palace_courtyard"
        self.inventory = []
        self.health = 100
        self.max_health = 100
        self.riddle_solved = False
        self.game_started = False
        
        # Combat state
        self.in_combat = False
        self.combat_enemy = None
        self.combat_enemy_hp = 0
        self.combat_enemy_max_hp = 0
        self.combat_heals_remaining = 3
        self.combat_defending = False
        self.combat_training_complete = False
        
        self.quests = {
            "beggar": {"status": "not_started", "stage": 0},
            "elephant": {"status": "not_started", "stage": 0},
            "soma": {"status": "not_started", "stage": 0},
            "cooking": {"status": "not_started", "stage": 0},
            "cushion": {"status": "not_started", "stage": 0}
        }
        
        self.flags = {
            "has_golden_bowl": False,
            "vikram_paid": False,
            "elephant_started": False,
            "fed_elephant": False,
            "removed_thorn": False,
            "comforted_elephant": False,
            "learned_steps": False,
            "steps_correct": False,
            "victory": False
        }
        
        self.items_needed = ["golden_bowl", "imperial_elephant", "soma", "honey_cakes", "royal_cushion"]
        
        # Music state
        self.music_enabled = True
        self.current_music = None
        
        # Tutorial state
        self.tutorial_shown = False
        
    def has_item(self, item):
        return item in self.inventory
        
    def add_item(self, item):
        if item not in self.inventory:
            self.inventory.append(item)
            if item == "golden_bowl":
                self.flags["has_golden_bowl"] = True
            return True
        return False
            
    def check_victory(self):
        return all(item in self.inventory for item in self.items_needed)

game = GameState()

# Locations
LOCATIONS = {
    "palace_courtyard": {
        "name": "Palace Courtyard",
        "description": "Grand courtyard with marble pillars. Morning sun glints off golden domes.",
        "image": "palace_courtyard.png",
        "exits": {"north": "fathers_chambers", "south": "mothers_chambers", "east": "royal_stables", "west": "garden_pavilion"},
        "npcs": [],
        "lore": """HISTORICAL NOTE: Pataliputra (modern-day Patna)

You stand in what was once the capital of the mighty Mauryan Empire, founded by 
Chandragupta Maurya around 322 BCE. Pataliputra was one of the largest cities in 
the ancient world, rivaling Rome and Alexandria.

The city was strategically located at the confluence of the Ganges and Son rivers, 
making it a hub of trade and military power. Greek ambassador Megasthenes described 
it as having massive wooden walls stretching 9 miles long.

This palace, where young Ashoka walks, witnessed the machinations of power that 
would shape the future of South Asia. Here, succession battles were fought not with 
swords alone, but with political intrigue and divine legitimacy.

The Mauryan Empire at its height controlled nearly the entire Indian subcontinent - 
an achievement not matched until the British Raj 2,000 years later."""
    },
    "mothers_chambers": {
        "name": "Mother's Chambers",
        "description": "Gentle space with sandalwood incense. Your mother sits by the window.",
        "image": "mothers_chambers.png",
        "exits": {"north": "palace_courtyard", "west": "palace_kitchen"},
        "npcs": ["mother"],
        "lore": """HISTORICAL NOTE: Ashoka's Mother - Subhadrangi (Dharma)

Ancient texts refer to Ashoka's mother as Dharma or Subhadrangi. Unlike the chief 
queens, she held a lower rank in the royal household, which complicated Ashoka's 
claim to the throne.

In the rigidly hierarchical Mauryan court, birth order and maternal lineage mattered 
immensely. Ashoka's rough appearance and lower maternal status initially excluded 
him from consideration as heir.

According to the Ashokavadana, his mother was devoted to Buddhism even before Ashoka's 
conversion. Her influence may have planted the seeds of compassion that would later 
transform him from conqueror to peacemaker.

The relationship between royal mothers and their sons often determined the fate of 
empires. A supportive mother could mean the difference between obscurity and greatness."""
    },
    "palace_kitchen": {
        "name": "Palace Kitchen",
        "description": "Busy kitchen filled with heat and noise. Amma Savitri kneads dough.",
        "image": "palace_kitchen.png",
        "exits": {"east": "mothers_chambers"},
        "npcs": ["amma"],
        "lore": """HISTORICAL NOTE: Food and Hierarchy in Ancient India

The palace kitchen was a world unto itself, feeding hundreds daily. Ancient texts 
describe elaborate feasts with dozens of dishes - rice, lentils, vegetables, sweets 
made with honey and ghee.

Mauryan society was highly stratified, yet the kitchen represented a space where 
practical skill sometimes transcended social rank. Cooks held specialized knowledge 
passed down through generations.

Honey cakes, mentioned in ancient recipes, combined wheat flour, honey, ghee, and 
spices like cardamom. Food was not merely sustenance but carried religious and social 
significance - what you ate, with whom, and how determined your status.

Ashoka's willingness to learn cooking would have been revolutionary - a prince 
humbling himself before a cook violated every social norm of the time."""
    },
    "garden_pavilion": {
        "name": "Garden of the Golden Pavilion",
        "description": "Otherworldly space. Golden pavilion touches clouds. Ascetic meditates beneath bodhi tree.",
        "image": "garden_pavilion.png",
        "exits": {"east": "palace_courtyard"},
        "npcs": ["ascetic"],
        "lore": """HISTORICAL NOTE: The Ashokavadana and Buddhist Texts

This scene draws from the Ashokavadana, a 2nd-century CE Sanskrit text that 
chronicles Ashoka's life through a Buddhist lens. Though written centuries after 
Ashoka's death, it preserves important historical traditions.

According to the text, ascetic Pingala-vatsajiva played a crucial role in Ashoka's 
selection as heir. The ascetic's prophecy - delivered through cryptic tests rather 
than direct naming - protected Ashoka from his brothers' jealousy.

The "five sacred items" quest in this game is inspired by the Ashokavadana's 
succession story, where objects symbolized different aspects of kingship: justice 
(bowl), strength (elephant), divine favor (soma), humility (food), and authority 
(throne).

These Buddhist texts shaped how Ashoka was remembered - not just as emperor, but as 
dharmaraja (righteous king), exemplifying Buddhist ideals of compassionate governance."""
    },
    "royal_stables": {
        "name": "Royal Stables",
        "description": "Earthy smell of elephants. Minister Radhagupta stands near Airavata.",
        "image": "royal_stables.png",
        "exits": {"west": "palace_courtyard", "south": "marketplace"},
        "npcs": ["radhagupta"],
        "lore": """HISTORICAL NOTE: War Elephants and Mauryan Military Power

Elephants were the tanks of ancient warfare. The Mauryan military maintained 
thousands - Megasthenes claimed 9,000 war elephants in Chandragupta's army.

These weren't just beasts of burden. A trained war elephant, armored and carrying 
archers, could break enemy formations, trample cavalry, and inspire terror. 
Controlling an elephant required deep understanding - force alone would fail.

Minister Radhagupta, mentioned in both Buddhist and Jain texts, was Chandragupta's 
advisor and likely served Bindusara as well. His historical counterpart, Chanakya 
(also called Kautilya), wrote the Arthashastra - an ancient treatise on statecraft, 
economics, and military strategy that remains studied today.

The relationship between Ashoka and his advisors would prove crucial. Unlike his 
brothers who ruled through fear, Ashoka would learn to balance power with wisdom."""
    },
    "marketplace": {
        "name": "Marketplace",
        "description": "Chaos and color. Blind beggar sits with golden bowl. Vikram at his stall.",
        "image": "marketplace.png",
        "exits": {"north": "royal_stables", "west": "temple_district"},
        "npcs": ["beggar", "vikram"],
        "lore": """HISTORICAL NOTE: Trade and Economy in Mauryan India

Pataliputra's marketplace was a crossroads of the ancient world. Merchants traded 
silk from China, spices from South India, gems from Sri Lanka, and goods from as 
far as Greece and Persia.

The Mauryan state controlled trade through elaborate taxation and regulation. 
The Arthashastra describes market superintendents, weights and measures standards, 
and penalties for fraud - showing sophisticated economic management.

Coins bore royal insignias, establishing centralized currency. Trade routes connected 
the empire, with the Grand Trunk Road linking Pataliputra to the northwest frontier.

Social inequality was stark. Merchants could grow wealthy, but rigid caste hierarchy 
limited mobility. A blind beggar, as in our story, would have few options beyond 
charity - though Buddhist and Jain teachings increasingly emphasized compassion for 
all beings.

Ashoka's later edicts would mandate the state's responsibility to care for the poor, 
sick, and disabled - revolutionary for its time."""
    },
    "temple_district": {
        "name": "Temple District",
        "description": "Incense smoke rises. Priest Mahinda tends the sacred fire.",
        "image": "temple_district.png",
        "exits": {"east": "marketplace"},
        "npcs": ["priest"],
        "lore": """HISTORICAL NOTE: Religion in Pre-Buddhist Mauryan India

Before Ashoka's conversion, the Mauryan court practiced Brahmanism (early Hinduism). 
Rituals, sacrifices, and the Vedic tradition dominated religious life. Priests 
(Brahmins) held immense social and spiritual authority.

However, this was an era of religious ferment. Buddhism and Jainism challenged 
Brahmanical orthodoxy, offering paths to enlightenment without caste hierarchy or 
animal sacrifice. These "heterodox" movements gained followers among merchants and 
even some nobility.

Soma, mentioned in ancient texts, was a ritual drink in Vedic ceremonies - possibly 
an intoxicating plant extract. Consuming soma was believed to grant communion with 
the gods and insight into cosmic truths.

Young Ashoka, raised in orthodox traditions, would later undergo a profound spiritual 
transformation. After the bloody Kalinga War, he embraced Buddhism (dhamma) and 
dedicated his reign to moral governance, non-violence, and religious tolerance.

His son Mahinda (possibly named after this priest in our story) would become a 
Buddhist monk who spread Buddhism to Sri Lanka."""
    },
    "fathers_chambers": {
        "name": "Throne Room",
        "description": "Stern room with black stone throne. Your father reviews reports.",
        "image": "fathers_chambers.png",
        "exits": {"south": "palace_courtyard"},
        "npcs": ["emperor"],
        "lore": """HISTORICAL NOTE: Bindusara and the Succession Crisis

Emperor Bindusara (ruled 298-272 BCE), Ashoka's father, expanded the empire south. 
Known as Amitraghata ("Slayer of Enemies"), he continued Chandragupta's conquests.

Ancient sources suggest Bindusara favored Ashoka's elder brother Susima as heir. 
Ashoka's path to power was not straightforward. According to tradition, Bindusara 
sent Ashoka to suppress a rebellion in Taxila, perhaps hoping distance would remove 
him from succession politics.

When Bindusara fell ill around 272 BCE, a succession crisis erupted. The 
Ashokavadana claims Ashoka killed 99 brothers to claim the throne, though this is 
likely exaggerated. Historical evidence suggests violent conflict with at least 
Susima and possibly others.

The throne Ashoka fought for would become a platform for one of history's most 
remarkable transformations. The man who waded through blood to claim power would 
later renounce violence and spread a message of peace across Asia.

His rock edicts - carved declarations of policy and philosophy - remain visible 
today, 2,300 years later."""
    }
}

# GUI Application
class AshokaGame:
    def __init__(self, root):
        self.root = root
        self.root.title("🕉️ ASHOKA'S PATH - ACT 1")
        self.root.geometry("1000x750")
        self.root.configure(bg='#2b2b2b')
        
        self.title_font = font.Font(family="Arial", size=14, weight="bold")
        self.text_font = font.Font(family="Courier", size=11)
        
        # Initialize pygame mixer for music
        try:
            pygame.mixer.init()
            self.music_available = True
        except:
            self.music_available = False
            self.p("Music system unavailable", "yellow")
        
        self.setup_ui()
        self.play_music("menu")  # Play menu music
        self.show_riddle()
        
    def setup_ui(self):
        # Image frame
        self.image_frame = tk.Frame(self.root, bg='#1a1a1a', height=220)
        self.image_frame.pack(fill=tk.X, padx=10, pady=5)
        self.image_frame.pack_propagate(False)
        self.image_label = tk.Label(self.image_frame, bg='#1a1a1a', fg='#4a9eff')
        self.image_label.pack(expand=True)
        
        # Location label
        self.location_label = tk.Label(self.root, text="", font=self.title_font, bg='#2b2b2b', fg='#4a9eff', pady=5)
        self.location_label.pack()
        
        # Text output
        self.output = scrolledtext.ScrolledText(self.root, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', 
                                                insertbackground='#4a9eff', wrap=tk.WORD, height=18)
        self.output.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Tags
        self.output.tag_configure("green", foreground="#4ade80")
        self.output.tag_configure("yellow", foreground="#fbbf24")
        self.output.tag_configure("red", foreground="#f87171")
        self.output.tag_configure("cyan", foreground="#22d3ee")
        self.output.tag_configure("magenta", foreground="#e879f9")
        self.output.tag_configure("gold", foreground="#fbbf24")
        self.output.tag_configure("bold", font=self.title_font)
        
        # Input
        input_frame = tk.Frame(self.root, bg='#2b2b2b')
        input_frame.pack(fill=tk.X, padx=10, pady=5)
        tk.Label(input_frame, text=">", font=self.text_font, bg='#2b2b2b', fg='#4ade80').pack(side=tk.LEFT, padx=(0, 5))
        self.input_entry = tk.Entry(input_frame, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', insertbackground='#4a9eff')
        self.input_entry.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.input_entry.bind('<Return>', self.process_cmd)
        
    def p(self, text, tag=None):
        self.output.insert(tk.END, text + "\n", tag)
        self.output.see(tk.END)
        
    def clear(self):
        self.output.delete(1.0, tk.END)
        
    def load_img(self, img):
        if not img:
            return
        path = f"images/{img}"
        if os.path.exists(path):
            try:
                im = Image.open(path)
                im.thumbnail((900, 210), Image.Resampling.LANCZOS)
                photo = ImageTk.PhotoImage(im)
                self.image_label.configure(image=photo, text="")
                self.image_label.image = photo
            except:
                self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
        else:
            self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
    
    # ========== MUSIC SYSTEM ==========
    
    def play_music(self, track_name):
        """Play background music"""
        if not self.music_available or not game.music_enabled:
            return
        
        music_files = {
            "menu": "music/menu.mp3",
            "exploration": "music/exploration.mp3",
            "battle": "music/battle.mp3",
            "victory": "music/victory.mp3"
        }
        
        if track_name not in music_files:
            return
        
        music_path = music_files[track_name]
        
        # Only change if different track
        if game.current_music == track_name:
            return
        
        try:
            if os.path.exists(music_path):
                pygame.mixer.music.load(music_path)
                pygame.mixer.music.set_volume(0.5)  # 50% volume
                
                # Loop for exploration/battle, play once for victory
                if track_name in ["exploration", "battle"]:
                    pygame.mixer.music.play(-1)  # Loop indefinitely
                else:
                    pygame.mixer.music.play(0)   # Play once
                
                game.current_music = track_name
        except Exception as e:
            pass  # Silently fail if music file missing
    
    def stop_music(self):
        """Stop all music"""
        if self.music_available:
            try:
                pygame.mixer.music.stop()
                game.current_music = None
            except:
                pass
    
    def toggle_music(self):
        """Toggle music on/off"""
        game.music_enabled = not game.music_enabled
        
        if game.music_enabled:
            self.p("🎵 Music: ON", "green")
            # Resume appropriate music
            if game.in_combat:
                self.play_music("battle")
            else:
                self.play_music("exploration")
        else:
            self.p("🔇 Music: OFF", "yellow")
            self.stop_music()
    
    def set_volume(self, volume):
        """Set music volume (0.0 to 1.0)"""
        if self.music_available:
            try:
                pygame.mixer.music.set_volume(max(0.0, min(1.0, volume)))
                self.p(f"🔊 Volume: {int(volume*100)}%", "cyan")
            except:
                pass
    
    # ========== END MUSIC SYSTEM ==========
    
    def show_riddle(self):
        self.input_entry.config(state='disabled')
        self.p("="*80, "bold")
        self.p("🕉️  ASHOKA'S PATH - ACT 1  🕉️", "bold")
        self.p("="*80, "bold")
        self.p("")
        self.p("An ancient ascetic appears...", "gold")
        self.p("")
        self.p("'Before you walk Ashoka's path, answer this:'")
        self.p("  I am won through blood, yet bring no peace")
        self.p("  I am sought by many, yet poison few")
        self.p("  I sit on heads, yet crush many souls")
        self.p("")
        self.p("  [1] The Crown", "cyan")
        self.p("  [2] A Sword", "cyan")
        self.p("  [3] Gold", "cyan")
        self.p("  [4] Power", "cyan")
        self.p("")
        self.input_entry.config(state='normal')
        self.input_entry.focus()
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_riddle)
    
    def check_riddle(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        if ans in ["1", "4"]:
            self.p(f"> {ans}", "cyan")
            self.p("")
            self.p("'Wisdom beyond your years. You may proceed...'", "green")
            self.p("")
            game.riddle_solved = True
            game.game_started = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.start_game()
        else:
            self.p(f"> {ans}", "cyan")
            self.p("'Think deeper...'", "red")
            self.p("")
    
    def start_game(self):
        self.play_music("exploration")  # Start exploration music
        self.p("="*80)
        self.p("Find FIVE sacred items:", "yellow")
        self.p("  • Vessel of compassion", "magenta")
        self.p("  • Mount of strength", "magenta")
        self.p("  • Drink of gods", "magenta")
        self.p("  • Food with humility", "magenta")
        self.p("  • Seat through courage", "magenta")
        self.p("")
        self.p("⭐ NEW PLAYER? Type 'tutorial' for detailed guide!", "gold")
        self.p("📜 Type 'lore' at any location for historical facts!", "gold")
        self.p("")
        self.p("Quick start: 'help' for commands | 'map' to see palace", "cyan")
        self.p("="*80)
        self.show_loc()
    
    def show_loc(self):
        loc = LOCATIONS[game.current_location]
        self.load_img(loc["image"])
        self.location_label.configure(text=loc["name"])
        self.p("")
        self.p(loc["description"])
        self.p("")
        exits = ", ".join([d.upper() for d in loc["exits"]])
        self.p(f"Exits: {exits}", "yellow")
        self.p("")
        if loc["npcs"]:
            self.p(f"People: {', '.join(loc['npcs'])}", "green")
            self.p("")
        # Show cushion if ready
        if game.current_location == "fathers_chambers" and len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
            self.p("You see: Royal Cushion", "magenta")
            self.p("")
    
    def process_cmd(self, e=None):
        if not game.game_started:
            return
        cmd = self.input_entry.get().lower().strip()
        self.input_entry.delete(0, tk.END)
        if not cmd:
            return
        
        # Combat mode - different command handling
        if game.in_combat:
            self.combat_action(cmd)
            return
        
        self.p(f"> {cmd}", "cyan")
        self.p("")
        
        if cmd in ["n","north"]: self.move("north")
        elif cmd in ["s","south"]: self.move("south")
        elif cmd in ["e","east"]: self.move("east")
        elif cmd in ["w","west"]: self.move("west")
        elif cmd in ["l","look"]: self.clear(); self.show_loc()
        elif cmd in ["i","inv","inventory"]: self.show_inv()
        elif cmd in ["q","quest","quests"]: self.show_quest()
        elif cmd in ["h","health"]: self.p(f"❤️ {game.health}/100", "green")
        elif cmd in ["?","help"]: self.show_help()
        elif cmd in ["m","map"]: self.show_map()
        elif cmd in ["music","toggle music"]: self.toggle_music()
        elif cmd in ["tutorial","tut"]: self.show_tutorial()
        elif cmd in ["lore","history","info"]: self.show_lore()
        elif cmd.startswith("volume "):
            try:
                vol = float(cmd.split()[1]) / 100.0
                self.set_volume(vol)
            except:
                self.p("Usage: volume <0-100>", "red")
        elif cmd in ["quit","exit"]: self.p("Thanks for playing!","yellow"); self.root.after(2000, self.root.quit)
        elif cmd.startswith("talk "): self.talk(cmd.replace("talk to ","").replace("talk ",""))
        elif cmd == "help vikram": self.help_vikram()
        elif cmd == "test elephant": self.test_elephant()
        elif cmd.startswith("feed "): self.feed(cmd.split()[1])
        elif cmd == "remove thorn": self.remove_thorn()
        elif cmd == "comfort elephant": self.comfort()
        elif cmd == "ritual": self.ritual()
        elif cmd == "learn cooking": self.learn_cook()
        elif cmd == "make cakes": self.make_cakes()
        elif cmd == "train combat" or cmd == "spar" or cmd == "train": self.train_combat()
        elif cmd.startswith("pick ") or cmd.startswith("get ") or cmd.startswith("take "): self.pick(cmd.split(maxsplit=1)[1])
        else: self.p("Unknown command. Type 'help'", "red")
        
        self.p("")
        if game.check_victory() and not game.flags["victory"]:
            self.p("="*70, "gold")
            self.p("✨ ALL FIVE ITEMS COLLECTED! ✨", "gold")
            self.p("="*70, "gold")
            self.p("Return to GARDEN and talk to ascetic!", "yellow")
            self.p("")
    
    def move(self, dir):
        loc = LOCATIONS[game.current_location]
        if dir in loc["exits"]:
            next_loc = loc["exits"][dir]
            # Throne room logic
            if next_loc == "fathers_chambers":
                if len(game.inventory) < 4:
                    # Still too early - instant death
                    self.die()
                    return
                elif len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
                    # Ready for Susima battle!
                    game.current_location = next_loc
                    self.p(f"You travel {dir}...", "cyan")
                    self.p("")
                    self.start_susima_battle()
                    return
            
            game.current_location = next_loc
            self.p(f"You travel {dir}...", "cyan")
            self.p("")
            self.show_loc()
        else:
            self.p("Can't go that way!", "red")
    
    def die(self):
        self.clear()
        self.p("="*70, "red")
        self.p("DISCOVERED BY SUSIMA!", "red")
        self.p("="*70, "red")
        self.p("Your brother accuses you of treason!")
        self.p("YOU DIED", "red")
        self.p("Collect 4+ items first. Restarting...", "yellow")
        self.input_entry.config(state='disabled')
        self.root.after(5000, self.restart)
    
    def restart(self):
        global game
        game = GameState()
        self.clear()
        self.input_entry.config(state='normal')
        self.show_riddle()
    
    def pick(self, item):
        item = item.lower().replace(" ", "_")
        if item == "royal_cushion":
            if game.current_location != "fathers_chambers":
                self.p("Not here", "red")
                return
            if len(game.inventory) < 4:
                self.p("Too dangerous. Need 4+ items first", "red")
                return
            if game.has_item("royal_cushion"):
                self.p("Already have it", "yellow")
                return
            game.add_item("royal_cushion")
            game.quests["cushion"]["status"] = "completed"
            self.p("✓ Acquired: Royal Cushion (5/5)", "gold")
            return
        self.p("Items earned through quests!", "yellow")
    
    def talk(self, npc):
        loc = LOCATIONS[game.current_location]
        if npc not in loc["npcs"]:
            self.p("Nobody by that name here", "red")
            return
        
        if npc == "mother": self.talk_mother()
        elif npc == "beggar": self.talk_beggar()
        elif npc == "vikram": self.talk_vikram()
        elif npc == "radhagupta": self.talk_radha()
        elif npc == "priest": self.talk_priest()
        elif npc == "amma": self.talk_amma()
        elif npc == "ascetic": self.talk_ascetic()
        else: self.p(f"{npc} says: Hello", "green")
    
    def talk_mother(self):
        """Mother Dharma's motivational dialogue"""
        self.p("")
        self.p("-" * 70, "green")
        self.p("MOTHER DHARMA speaks softly:", "green")
        self.p("-" * 70, "green")
        self.p("")
        
        self.p("My beloved son... come, sit with me.")
        self.p("")
        self.p("I know what troubles your heart. Your brothers mock you,")
        self.p("your father ignores you, the court whispers behind your")
        self.p("back. They see only your rough skin and think you unworthy")
        self.p("of the throne.")
        self.p("")
        self.p("But I have seen your destiny, Ashoka. In my dreams, I see", "gold")
        self.p("you not just as emperor, but as something far greater -", "gold")
        self.p("a light that will illuminate all of Jambudvipa.", "gold")
        self.p("")
        self.p("The ascetic Pingala-vatsajiva has been summoned by your")
        self.p("father to choose the next heir. He will test all of you.")
        self.p("This is your chance, my son.")
        self.p("")
        self.p("Listen carefully - the test will not be one of strength")
        self.p("or royal bearing. It will test the qualities that truly")
        self.p("make a great ruler:")
        self.p("")
        self.p("  • COMPASSION for the suffering", "yellow")
        self.p("  • WISDOM to see truth", "yellow")
        self.p("  • HUMILITY to learn from others", "yellow")
        self.p("  • COURAGE to face danger", "yellow")
        self.p("  • STRENGTH tempered with understanding", "yellow")
        self.p("")
        self.p("Go to the Garden of the Golden Pavilion. Face whatever")
        self.p("test the ascetic gives you. Show them all what I have")
        self.p("always known - that you have the heart of a true king.")
        self.p("")
        self.p("Remember what I've taught you:", "magenta")
        self.p("")
        self.p("  True power does not crush the weak - it protects them.", "cyan")
        self.p("")
        self.p("  A king who rules through fear is forgotten.", "cyan")
        self.p("  A king who rules with dharma is remembered forever.", "cyan")
        self.p("")
        self.p("Your father may never see it. Your brothers may never")
        self.p("accept it. But I believe in you, Ashoka.")
        self.p("")
        self.p("Now go. Make me proud.", "gold")
        self.p("")
        self.p("-" * 70, "green")
        self.p("")
        
        # Add a hint about where to go
        if not game.riddle_solved:
            self.p("💡 TIP: The ascetic awaits in the GARDEN (west)", "yellow")
        elif len(game.inventory) == 0:
            self.p("💡 TIP: Begin your quest - explore the palace!", "yellow")
        elif len(game.inventory) < 5:
            self.p(f"💡 TIP: You've collected {len(game.inventory)}/5 items. Keep going!", "yellow")
        else:
            self.p("💡 TIP: Return to the Garden - you're ready!", "yellow")
    
    def talk_beggar(self):
        q = game.quests["beggar"]
        if q["stage"] == 0:
            self.p("DHRUVA:", "green")
            self.p("Vikram owes me 20 gold. Help me get it?")
            self.p("(Try: 'talk to vikram' then 'help vikram')", "yellow")
            q["status"] = "in_progress"
            q["stage"] = 1
        elif q["stage"] == 1:
            self.p("DHRUVA: Spoken to Vikram yet?")
        elif q["stage"] == 2 and game.flags["vikram_paid"]:
            self.p("DHRUVA: Thank you! Take my bowl.")
            game.add_item("golden_bowl")
            q["status"] = "completed"
            q["stage"] = 3
            self.p("✓ Quest Complete! Golden Bowl (1/5)", "gold")
        else:
            self.p("DHRUVA: Thanks again.")
    
    def talk_vikram(self):
        if game.quests["beggar"]["stage"] < 1:
            self.p("VIKRAM: I'm busy.")
            return
        self.p("VIKRAM: I owe him nothing!")
        self.p("(Try: 'help vikram')", "yellow")
    
    def help_vikram(self):
        if game.quests["beggar"]["stage"] != 1:
            return
        self.p("You check his ledgers... Found it!")
        self.p("VIKRAM: Fine, take it!")
        game.flags["vikram_paid"] = True
        game.quests["beggar"]["stage"] = 2
        self.p("Return to beggar", "yellow")
    
    def talk_radha(self):
        self.p("RADHAGUPTA:", "green")
        self.p("Test yourself with Airavata?")
        if not game.combat_training_complete:
            self.p("")
            self.p("Or would you like combat training?", "yellow")
            self.p("(Try: 'test elephant' or 'train combat')", "yellow")
        else:
            self.p("(Try: 'test elephant')", "yellow")
    
    def test_elephant(self):
        if game.current_location != "royal_stables":
            return
        if game.quests["elephant"]["status"] == "completed":
            self.p("Already done", "yellow")
            return
        game.quests["elephant"]["status"] = "in_progress"
        game.flags["elephant_started"] = True
        self.p("PART 1: Feed Airavata")
        self.p("Options: hay, mangoes, sugarcane")
        self.p("(Try: 'feed hay')", "yellow")
    
    def feed(self, food):
        if not game.flags["elephant_started"] or game.flags["fed_elephant"]:
            return
        if food == "hay":
            self.p("Airavata accepts gratefully")
            game.flags["fed_elephant"] = True
            self.p("PART 2: Remove thorn", "yellow")
            self.p("(Try: 'remove thorn')", "yellow")
        else:
            self.p("Wrong choice", "red")
    
    def remove_thorn(self):
        if not game.flags["fed_elephant"] or game.flags["removed_thorn"]:
            return
        self.p("You carefully remove the thorn...")
        game.flags["removed_thorn"] = True
        self.p("PART 3: Comfort elephant", "yellow")
        self.p("(Try: 'comfort elephant')", "yellow")
    
    def comfort(self):
        if not game.flags["removed_thorn"]:
            return
        self.p("You speak of loneliness...")
        self.p("Airavata touches your face")
        game.flags["comforted_elephant"] = True
        game.add_item("imperial_elephant")
        game.quests["elephant"]["status"] = "completed"
        self.p("✓ Quest Complete! Imperial Elephant (2/5)", "gold")
    
    def talk_priest(self):
        self.p("MAHINDA:", "green")
        if not game.flags["has_golden_bowl"]:
            self.p("Need golden bowl first")
        else:
            self.p("Ready for ritual?")
            self.p("(Try: 'ritual')", "yellow")
    
    def ritual(self):
        if game.current_location != "temple_district":
            return
        if not game.flags["has_golden_bowl"]:
            return
        if game.quests["soma"]["status"] == "completed":
            return
        self.p("You complete the sacred ritual...")
        self.p("Vision: Two paths - war and peace")
        game.add_item("soma")
        game.quests["soma"]["status"] = "completed"
        self.p("✓ Quest Complete! Soma (3/5)", "gold")
    
    def talk_amma(self):
        self.p("AMMA:", "green")
        self.p("Want to learn cooking?")
        self.p("(Try: 'learn cooking')", "yellow")
    
    def learn_cook(self):
        if game.current_location != "palace_kitchen":
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        game.quests["cooking"]["status"] = "in_progress"
        game.flags["learned_steps"] = True
        self.p("AMMA: 5 steps:")
        self.p("1. Mix flour and ghee")
        self.p("2. Add honey slowly")
        self.p("3. Fold in cardamom")
        self.p("4. Shape into rounds")
        self.p("5. Bake until golden")
        self.p("")
        self.p("Repeat the order. Type: 1 2 3 4 5")
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_steps)
    
    def check_steps(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        correct = ["1 2 3 4 5", "12345", "1,2,3,4,5", "1, 2, 3, 4, 5"]
        if ans in correct:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Perfect!")
            game.flags["steps_correct"] = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.p("(Try: 'make cakes')", "yellow")
            self.p("")
        else:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Wrong! Come back.", "red")
            game.quests["cooking"]["status"] = "not_started"
            game.flags["learned_steps"] = False
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
    
    def make_cakes(self):
        if not game.flags["steps_correct"]:
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        self.p("You make honey cakes with care...")
        self.p("AMMA: Remember this humility")
        game.add_item("honey_cakes")
        game.quests["cooking"]["status"] = "completed"
        self.p("✓ Quest Complete! Honey Cakes (4/5)", "gold")
    
    def talk_ascetic(self):
        if not game.check_victory():
            self.p("ASCETIC: Return with all five items")
            return
        if game.flags["victory"]:
            return
        self.victory()
    
    def victory(self):
        game.flags["victory"] = True
        self.play_music("victory")  # Play victory music
        self.clear()
        self.load_img("coronation.png")
        self.p("")
        self.p("="*70, "gold")
        self.p("ACT 1 COMPLETE", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("You are crowned EMPEROR ASHOKA")
        self.p("")
        self.p("="*70, "gold")
        self.p("TO BE CONTINUED IN ACT 2...", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("Thank you for playing!", "cyan")
    
    # ========== COMBAT SYSTEM ==========
    
    def train_combat(self):
        """Start optional training battle"""
        if game.current_location != "royal_stables":
            self.p("Must be at Royal Stables to train", "red")
            return
        
        if game.combat_training_complete:
            self.p("RADHAGUPTA: You've already mastered combat training.", "green")
            return
        
        self.p("RADHAGUPTA: You wish to learn combat?", "green")
        self.p("RADHAGUPTA: Very well. Defend yourself!")
        self.p("")
        self.init_combat("Training Dummy", 60, is_training=True)
    
    def start_susima_battle(self):
        """Start required Susima boss battle"""
        self.clear()
        self.load_img("fathers_chambers.png")
        self.p("")
        self.p("="*70, "red")
        self.p("⚔️  BROTHER SUSIMA BLOCKS YOUR PATH!", "red")
        self.p("="*70, "red")
        self.p("")
        self.p("SUSIMA: 'Ashoka! You dare approach the throne?'")
        self.p("SUSIMA: 'I see you've been collecting... trinkets.'")
        self.p("SUSIMA: 'Let's see if you can fight as well as you scheme!'")
        self.p("")
        self.p("Susima draws his sword!")
        self.p("")
        self.init_combat("Brother Susima", 100, is_training=False)
    
    def init_combat(self, enemy_name, enemy_hp, is_training=False):
        """Initialize combat state"""
        import random
        self.play_music("battle")  # Play battle music
        game.in_combat = True
        game.combat_enemy = enemy_name
        game.combat_enemy_hp = enemy_hp
        game.combat_enemy_max_hp = enemy_hp
        game.combat_heals_remaining = 3
        game.combat_defending = False
        game.combat_is_training = is_training
        
        self.show_combat_status()
    
    def show_combat_status(self):
        """Display combat UI"""
        import random
        
        # Player health bar
        player_bar_filled = int((game.health / game.max_health) * 10)
        player_bar = "█" * player_bar_filled + "░" * (10 - player_bar_filled)
        
        # Enemy health bar
        enemy_bar_filled = int((game.combat_enemy_hp / game.combat_enemy_max_hp) * 10)
        enemy_bar = "█" * enemy_bar_filled + "░" * (10 - enemy_bar_filled)
        
        self.p("")
        self.p("="*60, "bold")
        self.p(f"⚔️  BATTLE: You vs {game.combat_enemy}", "bold")
        self.p("="*60, "bold")
        self.p("")
        self.p(f"YOU:    {player_bar} {game.health}/{game.max_health} HP | Heals: {game.combat_heals_remaining}/3", "green")
        self.p(f"{game.combat_enemy.upper()}: {enemy_bar} {game.combat_enemy_hp}/{game.combat_enemy_max_hp} HP", "red")
        self.p("")
        self.p("Your turn! What will you do?", "yellow")
        self.p("  [1] ATTACK      - Strike (15-25 damage)", "cyan")
        self.p("  [2] DEFEND      - Guard (50% damage reduction next turn)", "cyan")
        self.p("  [3] POWER ATK   - Heavy strike (25-35 dmg, 30% miss)", "cyan")
        self.p("  [4] HEAL        - Restore 25 HP", "cyan")
        self.p("")
    
    def combat_action(self, cmd):
        """Process combat commands"""
        import random
        
        if cmd in ["1", "attack", "atk"]:
            self.player_attack()
        elif cmd in ["2", "defend", "def", "guard"]:
            self.player_defend()
        elif cmd in ["3", "power", "power attack", "power atk"]:
            self.player_power_attack()
        elif cmd in ["4", "heal"]:
            self.player_heal()
        else:
            self.p("Invalid action! Choose 1-4", "red")
            self.p("")
            return
        
        # Check if enemy dead
        if self.check_combat_end():
            return
        
        # Enemy turn
        self.enemy_turn()
        
        # Check if player dead
        if self.check_combat_end():
            return
        
        # Next round
        self.show_combat_status()
    
    def player_attack(self):
        """Player normal attack"""
        import random
        damage = random.randint(15, 25)
        game.combat_enemy_hp -= damage
        game.combat_defending = False
        
        self.p(f"> ATTACK", "cyan")
        self.p("")
        self.p(f"You strike {game.combat_enemy}!", "green")
        self.p(f"💥 Dealt {damage} damage!", "green")
        self.p("")
    
    def player_defend(self):
        """Player defend"""
        game.combat_defending = True
        
        self.p(f"> DEFEND", "cyan")
        self.p("")
        self.p("You raise your guard defensively!", "green")
        self.p("🛡️  Next attack will be reduced by 50%!", "green")
        self.p("")
    
    def player_power_attack(self):
        """Player power attack with miss chance"""
        import random
        game.combat_defending = False
        
        self.p(f"> POWER ATTACK", "cyan")
        self.p("")
        
        if random.random() < 0.3:  # 30% miss chance
            self.p("Your heavy strike misses!", "red")
            self.p("You're off balance!", "red")
            self.p("")
        else:
            damage = random.randint(25, 35)
            game.combat_enemy_hp -= damage
            self.p(f"CRITICAL HIT!", "gold")
            self.p(f"💥💥 Dealt {damage} damage!", "gold")
            self.p("")
    
    def player_heal(self):
        """Player heal"""
        game.combat_defending = False
        
        self.p(f"> HEAL", "cyan")
        self.p("")
        
        if game.combat_heals_remaining <= 0:
            self.p("No heals remaining!", "red")
            self.p("You waste your turn!", "red")
            self.p("")
            return
        
        heal_amount = 25
        old_health = game.health
        game.health = min(game.health + heal_amount, game.max_health)
        actual_heal = game.health - old_health
        game.combat_heals_remaining -= 1
        
        self.p(f"You drink a healing potion!", "green")
        self.p(f"❤️  Restored {actual_heal} HP!", "green")
        self.p("")
    
    def enemy_turn(self):
        """Enemy AI turn"""
        import random
        
        # Simple AI: Random attack or power attack
        action = random.choice(["attack", "attack", "power"])  # 66% attack, 33% power
        
        if action == "attack":
            damage = random.randint(10, 20) if game.combat_is_training else random.randint(15, 25)
            
            if game.combat_defending:
                damage = damage // 2
                self.p(f"{game.combat_enemy} attacks!", "yellow")
                self.p(f"🛡️  Your guard blocks half the damage!", "green")
                self.p(f"💔 You take {damage} damage!", "red")
            else:
                self.p(f"{game.combat_enemy} attacks!", "yellow")
                self.p(f"💔 You take {damage} damage!", "red")
            
            game.health -= damage
            game.combat_defending = False
            
        elif action == "power":
            if random.random() < 0.4:  # 40% miss for enemy power
                self.p(f"{game.combat_enemy} attempts a heavy strike but misses!", "green")
            else:
                damage = random.randint(20, 30) if game.combat_is_training else random.randint(25, 35)
                
                if game.combat_defending:
                    damage = damage // 2
                    self.p(f"{game.combat_enemy} unleashes a POWER ATTACK!", "red")
                    self.p(f"🛡️  Your guard reduces the damage!", "green")
                    self.p(f"💔💔 You take {damage} damage!", "red")
                else:
                    self.p(f"{game.combat_enemy} unleashes a POWER ATTACK!", "red")
                    self.p(f"💔💔 You take {damage} damage!", "red")
                
                game.health -= damage
            
            game.combat_defending = False
        
        self.p("")
    
    def check_combat_end(self):
        """Check if combat is over"""
        if game.combat_enemy_hp <= 0:
            # Player wins!
            self.play_music("victory")  # Victory music
            self.p("="*60, "gold")
            self.p("⭐ VICTORY! ⭐", "gold")
            self.p("="*60, "gold")
            self.p("")
            
            if game.combat_is_training:
                self.p("RADHAGUPTA: 'Excellent! You have mastered combat!'", "green")
                self.p("RADHAGUPTA: 'You are ready for real battles now.'", "green")
                game.combat_training_complete = True
            else:
                self.p("Susima falls to his knees, defeated!", "green")
                self.p("")
                self.p("SUSIMA: 'Impossible... how did you...'", "yellow")
                self.p("")
                self.p("You stand victorious. The throne room is yours.", "green")
                self.p("")
                self.p("The royal cushion sits on the throne, waiting...", "magenta")
                self.p("")
            
            game.in_combat = False
            self.p("")
            
            # Return to exploration music after a delay
            self.root.after(3000, lambda: self.play_music("exploration"))
            
            if not game.combat_is_training:
                self.show_loc()
            
            return True
        
        if game.health <= 0:
            # Player dies!
            self.stop_music()  # Stop music on death
            self.p("="*60, "red")
            self.p("💀 DEFEAT 💀", "red")
            self.p("="*60, "red")
            self.p("")
            
            if game.combat_is_training:
                self.p("RADHAGUPTA: 'You fought well, but need more training.'", "yellow")
                self.p("RADHAGUPTA: 'Rest and try again when ready.'", "yellow")
                game.health = game.max_health  # Restore health after training
                game.in_combat = False
                self.play_music("exploration")  # Back to exploration
            else:
                self.p("Susima strikes you down!", "red")
                self.p("")
                self.p("YOU DIED", "red")
                self.p("")
                self.p("Restarting in 5 seconds...", "yellow")
                game.in_combat = False
                self.input_entry.config(state='disabled')
                self.root.after(5000, self.restart)
            
            self.p("")
            return True
        
        return False
    
    # ========== END COMBAT SYSTEM ==========
    
    # ========== TUTORIAL SYSTEM ==========
    
    def show_tutorial(self):
        """Display comprehensive tutorial"""
        self.clear()
        self.p("="*70, "bold")
        self.p("📖 ASHOKA'S PATH - COMPLETE TUTORIAL", "bold")
        self.p("="*70, "bold")
        self.p("")
        
        self.p("🎯 OBJECTIVE:", "gold")
        self.p("Collect FIVE sacred items to prove your worth as heir to the")
        self.p("Mauryan throne. Each item tests a different virtue.")
        self.p("")
        
        self.p("="*70, "cyan")
        self.p("MOVEMENT COMMANDS", "cyan")
        self.p("="*70, "cyan")
        self.p("  north, n       - Move north")
        self.p("  south, s       - Move south")
        self.p("  east, e        - Move east")
        self.p("  west, w        - Move west")
        self.p("")
        self.p("  look, l        - Re-examine current location")
        self.p("  map, m         - View palace map with your position")
        self.p("")
        
        self.p("="*70, "green")
        self.p("INFORMATION COMMANDS", "green")
        self.p("="*70, "green")
        self.p("  inventory, i   - Check items you've collected (X/5)")
        self.p("  quest, q       - View quest progress and active quests")
        self.p("  health, h      - Check your current health (combat)")
        self.p("  lore, history  - Read historical facts about location")
        self.p("  tutorial, tut  - Show this tutorial anytime")
        self.p("  help, ?        - Quick command reference")
        self.p("")
        
        self.p("="*70, "yellow")
        self.p("INTERACTION COMMANDS", "yellow")
        self.p("="*70, "yellow")
        self.p("  talk to <npc>  - Speak with characters")
        self.p("                   Example: 'talk to beggar'")
        self.p("")
        self.p("  pick <item>    - Pick up items (when available)")
        self.p("  get <item>     - Same as pick")
        self.p("  take <item>    - Same as pick")
        self.p("")
        
        self.p("="*70, "magenta")
        self.p("QUEST-SPECIFIC COMMANDS", "magenta")
        self.p("="*70, "magenta")
        self.p("Golden Bowl Quest (Marketplace):")
        self.p("  help vikram    - Investigate merchant's ledgers")
        self.p("")
        self.p("Elephant Quest (Royal Stables):")
        self.p("  test elephant  - Begin Airavata's worthiness test")
        self.p("  feed <food>    - Feed elephant (hay/mangoes/sugarcane)")
        self.p("  remove thorn   - Help elephant with thorn")
        self.p("  comfort elephant - Show compassion to Airavata")
        self.p("")
        self.p("Soma Quest (Temple District):")
        self.p("  ritual         - Begin sacred soma ritual")
        self.p("                   (Requires golden bowl first!)")
        self.p("")
        self.p("Cooking Quest (Palace Kitchen):")
        self.p("  learn cooking  - Learn honey cake recipe")
        self.p("  make cakes     - Cook the honey cakes")
        self.p("")
        self.p("Cushion Quest (Throne Room):")
        self.p("  pick cushion   - Take royal cushion")
        self.p("                   (Need 4+ items or death!)")
        self.p("")
        
        self.p("="*70, "red")
        self.p("COMBAT COMMANDS", "red")
        self.p("="*70, "red")
        self.p("  train combat   - Optional training at Royal Stables")
        self.p("                   (No death penalty, learn safely)")
        self.p("")
        self.p("During Battle (choose 1-4):")
        self.p("  1, attack      - Normal attack (15-25 damage)")
        self.p("  2, defend      - Guard (50% damage reduction)")
        self.p("  3, power       - Power attack (25-35 dmg, 30% miss)")
        self.p("  4, heal        - Restore 25 HP (3 uses per battle)")
        self.p("")
        
        self.p("="*70, "cyan")
        self.p("AUDIO COMMANDS", "cyan")
        self.p("="*70, "cyan")
        self.p("  music          - Toggle music on/off")
        self.p("  volume <0-100> - Set volume (0=mute, 100=max)")
        self.p("                   Example: 'volume 50'")
        self.p("")
        
        self.p("="*70, "gold")
        self.p("💡 IMPORTANT TIPS", "gold")
        self.p("="*70, "gold")
        self.p("")
        self.p("📍 QUEST ORDER:", "yellow")
        self.p("  1. Golden Bowl (beggar) - Unlocks soma ritual")
        self.p("  2. Imperial Elephant (stables) - Test of compassion")
        self.p("  3. Honey Cakes (kitchen) - Test of humility")
        self.p("  4. Soma (temple) - Needs bowl, vision quest")
        self.p("  5. Royal Cushion (throne) - Needs 4+ items!")
        self.p("")
        
        self.p("⚠️  DANGER ZONES:", "red")
        self.p("  • Throne Room with <4 items = INSTANT DEATH")
        self.p("  • Throne Room with 4+ items = BOSS BATTLE")
        self.p("  • Losing to Susima = GAME RESTART")
        self.p("  • Training battle = Safe practice (no penalty)")
        self.p("")
        
        self.p("✨ STRATEGY:", "green")
        self.p("  • Talk to everyone - NPCs give quest hints")
        self.p("  • Read lore at locations for immersion")
        self.p("  • Do training battle before Susima fight")
        self.p("  • Collect 4 items before throne room")
        self.p("  • Save heals for emergencies in combat")
        self.p("  • Return to Garden after all 5 items")
        self.p("")
        
        self.p("="*70, "bold")
        self.p("Press any key to continue...", "yellow")
        self.p("="*70, "bold")
        
        game.tutorial_shown = True
    
    def show_lore(self):
        """Display historical lore for current location"""
        location = LOCATIONS[game.current_location]
        
        if "lore" not in location:
            self.p("No historical information available here.", "yellow")
            return
        
        self.p("")
        self.p("="*70, "gold")
        self.p("📜 HISTORICAL LORE", "gold")
        self.p("="*70, "gold")
        self.p("")
        
        # Print lore with proper formatting
        lore_lines = location["lore"].split("\n")
        for line in lore_lines:
            if line.strip():
                self.p(line, "cyan" if line.startswith("HISTORICAL") else None)
            else:
                self.p("")
        
        self.p("")
        self.p("="*70, "gold")
    
    # ========== END TUTORIAL SYSTEM ==========
    
    def show_inv(self):
        if not game.inventory:
            self.p("Empty", "yellow")
        else:
            self.p("📦 INVENTORY:", "bold")
            for i, item in enumerate(game.inventory, 1):
                self.p(f"  {i}. {item.replace('_', ' ').title()}", "green")
            self.p(f"{len(game.inventory)}/5 items", "yellow")
    
    def show_quest(self):
        self.p("📜 QUESTS:", "bold")
        for name, q in game.quests.items():
            s = "✓" if q["status"] == "completed" else "◆" if q["status"] == "in_progress" else "○"
            c = "green" if q["status"] == "completed" else "yellow" if q["status"] == "in_progress" else "cyan"
            self.p(f"{s} {name}", c)
    
    def show_map(self):
        """Display game map with current location highlighted"""
        # Load map image if available
        self.load_img("map.png")
        
        self.p("")
        self.p("="*60, "bold")
        self.p("🗺️  PATALIPUTRA PALACE MAP", "bold")
        self.p("="*60, "bold")
        self.p("")
        
        # Create map with current location highlighted
        current = game.current_location
        
        # Map structure
        locations_map = {
            "fathers_chambers": "    [Throne Room]",
            "palace_courtyard_n": "          |",
            "palace_courtyard": "  [Palace Courtyard]",
            "mothers_chambers": "          |",
            "palace_kitchen": "[Kitchen]--+",
            "garden_pavilion": "[Garden]--+--[Stables]",
            "marketplace": "              |",
            "temple_district": "     [Temple]--[Market]"
        }
        
        # Print map with highlights
        self.p("         NORTH", "cyan")
        self.p("")
        
        # Throne Room
        if current == "fathers_chambers":
            self.p("    >>> [THRONE ROOM] <<<", "gold")
        else:
            self.p("        [Throne Room]", "cyan")
        
        self.p("              |", "cyan")
        
        # Palace Courtyard (center)
        if current == "palace_courtyard":
            self.p("                            ", "cyan")
            self.p("  [Garden]------>>> [YOU ARE HERE] <<<------[Stables]", "gold")
            self.p("                            ", "cyan")
        else:
            self.p("                            ", "cyan")
            self.p("  [Garden]--------[Courtyard]--------[Stables]", "cyan")
            self.p("                            ", "cyan")
        
        self.p("                      |                    |", "cyan")
        
        # Bottom row
        left = "              " if current != "garden_pavilion" else ">>> "
        middle_m = " [Mother's] " if current != "mothers_chambers" else " >>> [YOU] <<< "
        middle_k = "[Kitchen]" if current != "palace_kitchen" else ">>>[YOU]<<<"
        right_m = "[Market]" if current != "marketplace" else ">>>[YOU]<<<"
        right_t = "[Temple]" if current != "temple_district" else ">>>[YOU]<<<"
        right_s = " " if current != "royal_stables" else ""
        
        self.p(f"{left}          {middle_m}            {right_s}", "cyan" if current not in ["mothers_chambers", "royal_stables"] else "gold")
        self.p(f"                      |", "cyan")
        self.p(f"                 {middle_k}", "cyan" if current != "palace_kitchen" else "gold")
        self.p("")
        self.p(f"                                   {right_t}---{right_m}", "cyan" if current not in ["marketplace", "temple_district"] else "gold")
        
        self.p("")
        self.p(f"Current Location: {LOCATIONS[current]['name']}", "yellow")
        self.p("")
        self.p("="*60, "bold")
        self.p("")
    
    def show_help(self):
        self.p("COMMANDS:", "bold")
        self.p("Movement: n/s/e/w, look, map")
        self.p("Info: inventory, quest, lore, tutorial")
        self.p("Actions: talk to <npc>, pick <item>")
        self.p("Combat: train combat (at stables)")
        self.p("Audio: music (toggle), volume <0-100>")
        self.p("Quests: help vikram, test elephant, feed <food>,")
        self.p("        remove thorn, comfort elephant, ritual,")
        self.p("        learn cooking, make cakes")
        self.p("")
        self.p("Type 'tutorial' for detailed guide!", "yellow")
        self.p("Type 'lore' for historical facts!", "yellow")

def main():
    root = tk.Tk()
    app = AshokaGame(root)
    root.mainloop()

if __name__ == "__main__":
    main()

# version 9 

In [1]:
"""
ASHOKA'S PATH - COMPLETE ACT 1
All quests, riddles, and adventures fully implemented
University of Aberdeen GameJam 2026
"""

import tkinter as tk
from tkinter import scrolledtext, font
from PIL import Image, ImageTk
import os
import pygame

# Game State with ALL quest tracking
class GameState:
    def __init__(self):
        self.current_location = "palace_courtyard"
        self.inventory = []
        self.health = 100
        self.max_health = 100
        self.riddle_solved = False
        self.game_started = False
        
        # Combat state
        self.in_combat = False
        self.combat_enemy = None
        self.combat_enemy_hp = 0
        self.combat_enemy_max_hp = 0
        self.combat_heals_remaining = 3
        self.combat_defending = False
        self.combat_training_complete = False
        
        self.quests = {
            "beggar": {"status": "not_started", "stage": 0},
            "elephant": {"status": "not_started", "stage": 0},
            "soma": {"status": "not_started", "stage": 0},
            "cooking": {"status": "not_started", "stage": 0},
            "cushion": {"status": "not_started", "stage": 0}
        }
        
        self.flags = {
            "has_golden_bowl": False,
            "vikram_paid": False,
            "elephant_started": False,
            "fed_elephant": False,
            "removed_thorn": False,
            "comforted_elephant": False,
            "learned_steps": False,
            "steps_correct": False,
            "victory": False
        }
        
        self.items_needed = ["golden_bowl", "imperial_elephant", "soma", "honey_cakes", "royal_cushion"]
        
        # Music state
        self.music_enabled = True
        self.current_music = None
        
        # Tutorial state
        self.tutorial_shown = False
        
    def has_item(self, item):
        return item in self.inventory
        
    def add_item(self, item):
        if item not in self.inventory:
            self.inventory.append(item)
            if item == "golden_bowl":
                self.flags["has_golden_bowl"] = True
            return True
        return False
            
    def check_victory(self):
        return all(item in self.inventory for item in self.items_needed)

game = GameState()

# Locations
LOCATIONS = {
    "palace_courtyard": {
        "name": "Palace Courtyard",
        "description": "Grand courtyard with marble pillars. Morning sun glints off golden domes.",
        "image": "palace_courtyard.png",
        "exits": {"north": "fathers_chambers", "south": "mothers_chambers", "east": "royal_stables", "west": "garden_pavilion"},
        "npcs": [],
        "lore": """HISTORICAL NOTE: Pataliputra (modern-day Patna)

You stand in what was once the capital of the mighty Mauryan Empire, founded by 
Chandragupta Maurya around 322 BCE. Pataliputra was one of the largest cities in 
the ancient world, rivaling Rome and Alexandria.

The city was strategically located at the confluence of the Ganges and Son rivers, 
making it a hub of trade and military power. Greek ambassador Megasthenes described 
it as having massive wooden walls stretching 9 miles long.

This palace, where young Ashoka walks, witnessed the machinations of power that 
would shape the future of South Asia. Here, succession battles were fought not with 
swords alone, but with political intrigue and divine legitimacy.

The Mauryan Empire at its height controlled nearly the entire Indian subcontinent - 
an achievement not matched until the British Raj 2,000 years later."""
    },
    "mothers_chambers": {
        "name": "Mother's Chambers",
        "description": "Gentle space with sandalwood incense. Your mother sits by the window.",
        "image": "mothers_chambers.png",
        "exits": {"north": "palace_courtyard", "west": "palace_kitchen"},
        "npcs": ["mother"],
        "lore": """HISTORICAL NOTE: Ashoka's Mother - Subhadrangi (Dharma)

Ancient texts refer to Ashoka's mother as Dharma or Subhadrangi. Unlike the chief 
queens, she held a lower rank in the royal household, which complicated Ashoka's 
claim to the throne.

In the rigidly hierarchical Mauryan court, birth order and maternal lineage mattered 
immensely. Ashoka's rough appearance and lower maternal status initially excluded 
him from consideration as heir.

According to the Ashokavadana, his mother was devoted to Buddhism even before Ashoka's 
conversion. Her influence may have planted the seeds of compassion that would later 
transform him from conqueror to peacemaker.

The relationship between royal mothers and their sons often determined the fate of 
empires. A supportive mother could mean the difference between obscurity and greatness."""
    },
    "palace_kitchen": {
        "name": "Palace Kitchen",
        "description": "Busy kitchen filled with heat and noise. Amma Savitri kneads dough.",
        "image": "palace_kitchen.png",
        "exits": {"east": "mothers_chambers"},
        "npcs": ["amma"],
        "lore": """HISTORICAL NOTE: Food and Hierarchy in Ancient India

The palace kitchen was a world unto itself, feeding hundreds daily. Ancient texts 
describe elaborate feasts with dozens of dishes - rice, lentils, vegetables, sweets 
made with honey and ghee.

Mauryan society was highly stratified, yet the kitchen represented a space where 
practical skill sometimes transcended social rank. Cooks held specialized knowledge 
passed down through generations.

Honey cakes, mentioned in ancient recipes, combined wheat flour, honey, ghee, and 
spices like cardamom. Food was not merely sustenance but carried religious and social 
significance - what you ate, with whom, and how determined your status.

Ashoka's willingness to learn cooking would have been revolutionary - a prince 
humbling himself before a cook violated every social norm of the time."""
    },
    "garden_pavilion": {
        "name": "Garden of the Golden Pavilion",
        "description": "Otherworldly space. Golden pavilion touches clouds. Ascetic meditates beneath bodhi tree.",
        "image": "garden_pavilion.png",
        "exits": {"east": "palace_courtyard"},
        "npcs": ["ascetic"],
        "lore": """HISTORICAL NOTE: The Ashokavadana and Buddhist Texts

This scene draws from the Ashokavadana, a 2nd-century CE Sanskrit text that 
chronicles Ashoka's life through a Buddhist lens. Though written centuries after 
Ashoka's death, it preserves important historical traditions.

According to the text, ascetic Pingala-vatsajiva played a crucial role in Ashoka's 
selection as heir. The ascetic's prophecy - delivered through cryptic tests rather 
than direct naming - protected Ashoka from his brothers' jealousy.

The "five sacred items" quest in this game is inspired by the Ashokavadana's 
succession story, where objects symbolized different aspects of kingship: justice 
(bowl), strength (elephant), divine favor (soma), humility (food), and authority 
(throne).

These Buddhist texts shaped how Ashoka was remembered - not just as emperor, but as 
dharmaraja (righteous king), exemplifying Buddhist ideals of compassionate governance."""
    },
    "royal_stables": {
        "name": "Royal Stables",
        "description": "Earthy smell of elephants. Minister Radhagupta stands near Airavata.",
        "image": "royal_stables.png",
        "exits": {"west": "palace_courtyard", "south": "marketplace"},
        "npcs": ["radhagupta"],
        "lore": """HISTORICAL NOTE: War Elephants and Mauryan Military Power

Elephants were the tanks of ancient warfare. The Mauryan military maintained 
thousands - Megasthenes claimed 9,000 war elephants in Chandragupta's army.

These weren't just beasts of burden. A trained war elephant, armored and carrying 
archers, could break enemy formations, trample cavalry, and inspire terror. 
Controlling an elephant required deep understanding - force alone would fail.

Minister Radhagupta, mentioned in both Buddhist and Jain texts, was Chandragupta's 
advisor and likely served Bindusara as well. His historical counterpart, Chanakya 
(also called Kautilya), wrote the Arthashastra - an ancient treatise on statecraft, 
economics, and military strategy that remains studied today.

The relationship between Ashoka and his advisors would prove crucial. Unlike his 
brothers who ruled through fear, Ashoka would learn to balance power with wisdom."""
    },
    "marketplace": {
        "name": "Marketplace",
        "description": "Chaos and color. Blind beggar sits with golden bowl. Vikram at his stall.",
        "image": "marketplace.png",
        "exits": {"north": "royal_stables", "west": "temple_district"},
        "npcs": ["beggar", "vikram"],
        "lore": """HISTORICAL NOTE: Trade and Economy in Mauryan India

Pataliputra's marketplace was a crossroads of the ancient world. Merchants traded 
silk from China, spices from South India, gems from Sri Lanka, and goods from as 
far as Greece and Persia.

The Mauryan state controlled trade through elaborate taxation and regulation. 
The Arthashastra describes market superintendents, weights and measures standards, 
and penalties for fraud - showing sophisticated economic management.

Coins bore royal insignias, establishing centralized currency. Trade routes connected 
the empire, with the Grand Trunk Road linking Pataliputra to the northwest frontier.

Social inequality was stark. Merchants could grow wealthy, but rigid caste hierarchy 
limited mobility. A blind beggar, as in our story, would have few options beyond 
charity - though Buddhist and Jain teachings increasingly emphasized compassion for 
all beings.

Ashoka's later edicts would mandate the state's responsibility to care for the poor, 
sick, and disabled - revolutionary for its time."""
    },
    "temple_district": {
        "name": "Temple District",
        "description": "Incense smoke rises. Priest Mahinda tends the sacred fire.",
        "image": "temple_district.png",
        "exits": {"east": "marketplace"},
        "npcs": ["priest"],
        "lore": """HISTORICAL NOTE: Religion in Pre-Buddhist Mauryan India

Before Ashoka's conversion, the Mauryan court practiced Brahmanism (early Hinduism). 
Rituals, sacrifices, and the Vedic tradition dominated religious life. Priests 
(Brahmins) held immense social and spiritual authority.

However, this was an era of religious ferment. Buddhism and Jainism challenged 
Brahmanical orthodoxy, offering paths to enlightenment without caste hierarchy or 
animal sacrifice. These "heterodox" movements gained followers among merchants and 
even some nobility.

Soma, mentioned in ancient texts, was a ritual drink in Vedic ceremonies - possibly 
an intoxicating plant extract. Consuming soma was believed to grant communion with 
the gods and insight into cosmic truths.

Young Ashoka, raised in orthodox traditions, would later undergo a profound spiritual 
transformation. After the bloody Kalinga War, he embraced Buddhism (dhamma) and 
dedicated his reign to moral governance, non-violence, and religious tolerance.

His son Mahinda (possibly named after this priest in our story) would become a 
Buddhist monk who spread Buddhism to Sri Lanka."""
    },
    "fathers_chambers": {
        "name": "Throne Room",
        "description": "Stern room with black stone throne. Your father reviews reports.",
        "image": "fathers_chambers.png",
        "exits": {"south": "palace_courtyard"},
        "npcs": ["emperor"],
        "lore": """HISTORICAL NOTE: Bindusara and the Succession Crisis

Emperor Bindusara (ruled 298-272 BCE), Ashoka's father, expanded the empire south. 
Known as Amitraghata ("Slayer of Enemies"), he continued Chandragupta's conquests.

Ancient sources suggest Bindusara favored Ashoka's elder brother Susima as heir. 
Ashoka's path to power was not straightforward. According to tradition, Bindusara 
sent Ashoka to suppress a rebellion in Taxila, perhaps hoping distance would remove 
him from succession politics.

When Bindusara fell ill around 272 BCE, a succession crisis erupted. The 
Ashokavadana claims Ashoka killed 99 brothers to claim the throne, though this is 
likely exaggerated. Historical evidence suggests violent conflict with at least 
Susima and possibly others.

The throne Ashoka fought for would become a platform for one of history's most 
remarkable transformations. The man who waded through blood to claim power would 
later renounce violence and spread a message of peace across Asia.

His rock edicts - carved declarations of policy and philosophy - remain visible 
today, 2,300 years later."""
    }
}

# GUI Application
class AshokaGame:
    def __init__(self, root):
        self.root = root
        self.root.title("🕉️ ASHOKA'S PATH - ACT 1")
        # INCREASED HEIGHT: 900px to fit larger images
        self.root.geometry("1000x900")
        self.root.configure(bg='#2b2b2b')
        
        self.title_font = font.Font(family="Arial", size=14, weight="bold")
        self.text_font = font.Font(family="Courier", size=11)
        
        # Initialize pygame mixer for music
        try:
            pygame.mixer.init()
            self.music_available = True
        except:
            self.music_available = False
            self.p("Music system unavailable", "yellow")
        
        self.setup_ui()
        self.play_music("menu")  # Play menu music
        self.show_riddle()
        
    def setup_ui(self):
        # Image frame (INCREASED SIZE for presentation)
        self.image_frame = tk.Frame(self.root, bg='#1a1a1a', height=400)
        self.image_frame.pack(fill=tk.X, padx=10, pady=5)
        self.image_frame.pack_propagate(False)
        self.image_label = tk.Label(self.image_frame, bg='#1a1a1a', fg='#4a9eff')
        self.image_label.pack(expand=True)
        
        # Location label
        self.location_label = tk.Label(self.root, text="", font=self.title_font, bg='#2b2b2b', fg='#4a9eff', pady=5)
        self.location_label.pack()
        
        # Text output
        self.output = scrolledtext.ScrolledText(self.root, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', 
                                                insertbackground='#4a9eff', wrap=tk.WORD, height=18)
        self.output.pack(fill=tk.BOTH, expand=True, padx=10, pady=5)
        
        # Tags
        self.output.tag_configure("green", foreground="#4ade80")
        self.output.tag_configure("yellow", foreground="#fbbf24")
        self.output.tag_configure("red", foreground="#f87171")
        self.output.tag_configure("cyan", foreground="#22d3ee")
        self.output.tag_configure("magenta", foreground="#e879f9")
        self.output.tag_configure("gold", foreground="#fbbf24")
        self.output.tag_configure("bold", font=self.title_font)
        
        # Input
        input_frame = tk.Frame(self.root, bg='#2b2b2b')
        input_frame.pack(fill=tk.X, padx=10, pady=5)
        tk.Label(input_frame, text=">", font=self.text_font, bg='#2b2b2b', fg='#4ade80').pack(side=tk.LEFT, padx=(0, 5))
        self.input_entry = tk.Entry(input_frame, font=self.text_font, bg='#1a1a1a', fg='#e0e0e0', insertbackground='#4a9eff')
        self.input_entry.pack(side=tk.LEFT, fill=tk.X, expand=True)
        self.input_entry.bind('<Return>', self.process_cmd)
        
    def p(self, text, tag=None):
        self.output.insert(tk.END, text + "\n", tag)
        self.output.see(tk.END)
        
    def clear(self):
        self.output.delete(1.0, tk.END)
        
    def load_img(self, img):
        if not img:
            return
        path = f"images/{img}"
        if os.path.exists(path):
            try:
                im = Image.open(path)
                # INCREASED SIZE: 900x380 for presentation visibility
                im.thumbnail((900, 380), Image.Resampling.LANCZOS)
                photo = ImageTk.PhotoImage(im)
                self.image_label.configure(image=photo, text="")
                self.image_label.image = photo
            except:
                self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
        else:
            self.image_label.configure(text=f"🖼️ {img.replace('.png','').replace('_',' ').title()}", image="")
    
    # ========== MUSIC SYSTEM ==========
    
    def play_music(self, track_name):
        """Play background music"""
        if not self.music_available or not game.music_enabled:
            return
        
        music_files = {
            "menu": "music/menu.mp3",
            "exploration": "music/exploration.mp3",
            "battle": "music/battle.mp3",
            "victory": "music/victory.mp3"
        }
        
        if track_name not in music_files:
            return
        
        music_path = music_files[track_name]
        
        # Only change if different track
        if game.current_music == track_name:
            return
        
        try:
            if os.path.exists(music_path):
                pygame.mixer.music.load(music_path)
                pygame.mixer.music.set_volume(0.5)  # 50% volume
                
                # Loop for exploration/battle, play once for victory
                if track_name in ["exploration", "battle"]:
                    pygame.mixer.music.play(-1)  # Loop indefinitely
                else:
                    pygame.mixer.music.play(0)   # Play once
                
                game.current_music = track_name
        except Exception as e:
            pass  # Silently fail if music file missing
    
    def stop_music(self):
        """Stop all music"""
        if self.music_available:
            try:
                pygame.mixer.music.stop()
                game.current_music = None
            except:
                pass
    
    def toggle_music(self):
        """Toggle music on/off"""
        game.music_enabled = not game.music_enabled
        
        if game.music_enabled:
            self.p("🎵 Music: ON", "green")
            # Resume appropriate music
            if game.in_combat:
                self.play_music("battle")
            else:
                self.play_music("exploration")
        else:
            self.p("🔇 Music: OFF", "yellow")
            self.stop_music()
    
    def set_volume(self, volume):
        """Set music volume (0.0 to 1.0)"""
        if self.music_available:
            try:
                pygame.mixer.music.set_volume(max(0.0, min(1.0, volume)))
                self.p(f"🔊 Volume: {int(volume*100)}%", "cyan")
            except:
                pass
    
    # ========== END MUSIC SYSTEM ==========
    
    def show_riddle(self):
        self.input_entry.config(state='disabled')
        self.p("="*80, "bold")
        self.p("🕉️  ASHOKA'S PATH - ACT 1  🕉️", "bold")
        self.p("="*80, "bold")
        self.p("")
        self.p("An ancient ascetic appears...", "gold")
        self.p("")
        self.p("'Before you walk Ashoka's path, answer this:'")
        self.p("  I am won through blood, yet bring no peace")
        self.p("  I am sought by many, yet poison few")
        self.p("  I sit on heads, yet crush many souls")
        self.p("")
        self.p("  [1] The Crown", "cyan")
        self.p("  [2] A Sword", "cyan")
        self.p("  [3] Gold", "cyan")
        self.p("  [4] Power", "cyan")
        self.p("")
        self.input_entry.config(state='normal')
        self.input_entry.focus()
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_riddle)
    
    def check_riddle(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        if ans in ["1", "4"]:
            self.p(f"> {ans}", "cyan")
            self.p("")
            self.p("'Wisdom beyond your years. You may proceed...'", "green")
            self.p("")
            game.riddle_solved = True
            game.game_started = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.start_game()
        else:
            self.p(f"> {ans}", "cyan")
            self.p("'Think deeper...'", "red")
            self.p("")
    
    def start_game(self):
        self.play_music("exploration")  # Start exploration music
        self.p("="*80)
        self.p("Find FIVE sacred items:", "yellow")
        self.p("  • Vessel of compassion", "magenta")
        self.p("  • Mount of strength", "magenta")
        self.p("  • Drink of gods", "magenta")
        self.p("  • Food with humility", "magenta")
        self.p("  • Seat through courage", "magenta")
        self.p("")
        self.p("⭐ NEW PLAYER? Type 'tutorial' for detailed guide!", "gold")
        self.p("📜 Type 'lore' at any location for historical facts!", "gold")
        self.p("")
        self.p("Quick start: 'help' for commands | 'map' to see palace", "cyan")
        self.p("="*80)
        self.show_loc()
    
    def show_loc(self):
        loc = LOCATIONS[game.current_location]
        self.load_img(loc["image"])
        self.location_label.configure(text=loc["name"])
        self.p("")
        self.p(loc["description"])
        self.p("")
        exits = ", ".join([d.upper() for d in loc["exits"]])
        self.p(f"Exits: {exits}", "yellow")
        self.p("")
        if loc["npcs"]:
            self.p(f"People: {', '.join(loc['npcs'])}", "green")
            self.p("")
        # Show cushion if ready
        if game.current_location == "fathers_chambers" and len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
            self.p("You see: Royal Cushion", "magenta")
            self.p("")
    
    def process_cmd(self, e=None):
        if not game.game_started:
            return
        cmd = self.input_entry.get().lower().strip()
        self.input_entry.delete(0, tk.END)
        if not cmd:
            return
        
        # Combat mode - different command handling
        if game.in_combat:
            self.combat_action(cmd)
            return
        
        self.p(f"> {cmd}", "cyan")
        self.p("")
        
        if cmd in ["n","north"]: self.move("north")
        elif cmd in ["s","south"]: self.move("south")
        elif cmd in ["e","east"]: self.move("east")
        elif cmd in ["w","west"]: self.move("west")
        elif cmd in ["l","look"]: self.clear(); self.show_loc()
        elif cmd in ["i","inv","inventory"]: self.show_inv()
        elif cmd in ["q","quest","quests"]: self.show_quest()
        elif cmd in ["h","health"]: self.p(f"❤️ {game.health}/100", "green")
        elif cmd in ["?","help"]: self.show_help()
        elif cmd in ["m","map"]: self.show_map()
        elif cmd in ["music","toggle music"]: self.toggle_music()
        elif cmd in ["tutorial","tut"]: self.show_tutorial()
        elif cmd in ["lore","history","info"]: self.show_lore()
        elif cmd.startswith("volume "):
            try:
                vol = float(cmd.split()[1]) / 100.0
                self.set_volume(vol)
            except:
                self.p("Usage: volume <0-100>", "red")
        elif cmd in ["quit","exit"]: self.p("Thanks for playing!","yellow"); self.root.after(2000, self.root.quit)
        elif cmd.startswith("talk "): self.talk(cmd.replace("talk to ","").replace("talk ",""))
        elif cmd == "help vikram": self.help_vikram()
        elif cmd == "test elephant": self.test_elephant()
        elif cmd.startswith("feed "): self.feed(cmd.split()[1])
        elif cmd == "remove thorn": self.remove_thorn()
        elif cmd == "comfort elephant": self.comfort()
        elif cmd == "ritual": self.ritual()
        elif cmd == "learn cooking": self.learn_cook()
        elif cmd == "make cakes": self.make_cakes()
        elif cmd == "train combat" or cmd == "spar" or cmd == "train": self.train_combat()
        elif cmd.startswith("pick ") or cmd.startswith("get ") or cmd.startswith("take "): self.pick(cmd.split(maxsplit=1)[1])
        else: self.p("Unknown command. Type 'help'", "red")
        
        self.p("")
        if game.check_victory() and not game.flags["victory"]:
            self.p("="*70, "gold")
            self.p("✨ ALL FIVE ITEMS COLLECTED! ✨", "gold")
            self.p("="*70, "gold")
            self.p("Return to GARDEN and talk to ascetic!", "yellow")
            self.p("")
    
    def move(self, dir):
        loc = LOCATIONS[game.current_location]
        if dir in loc["exits"]:
            next_loc = loc["exits"][dir]
            # Throne room logic
            if next_loc == "fathers_chambers":
                if len(game.inventory) < 4:
                    # Still too early - instant death
                    self.die()
                    return
                elif len(game.inventory) >= 4 and not game.has_item("royal_cushion"):
                    # Ready for Susima battle!
                    game.current_location = next_loc
                    self.p(f"You travel {dir}...", "cyan")
                    self.p("")
                    self.start_susima_battle()
                    return
            
            game.current_location = next_loc
            self.p(f"You travel {dir}...", "cyan")
            self.p("")
            self.show_loc()
        else:
            self.p("Can't go that way!", "red")
    
    def die(self):
        self.clear()
        self.p("="*70, "red")
        self.p("DISCOVERED BY SUSIMA!", "red")
        self.p("="*70, "red")
        self.p("Your brother accuses you of treason!")
        self.p("YOU DIED", "red")
        self.p("Collect 4+ items first. Restarting...", "yellow")
        self.input_entry.config(state='disabled')
        self.root.after(5000, self.restart)
    
    def restart(self):
        global game
        game = GameState()
        self.clear()
        self.input_entry.config(state='normal')
        self.show_riddle()
    
    def pick(self, item):
        item = item.lower().replace(" ", "_")
        if item == "royal_cushion":
            if game.current_location != "fathers_chambers":
                self.p("Not here", "red")
                return
            if len(game.inventory) < 4:
                self.p("Too dangerous. Need 4+ items first", "red")
                return
            if game.has_item("royal_cushion"):
                self.p("Already have it", "yellow")
                return
            game.add_item("royal_cushion")
            game.quests["cushion"]["status"] = "completed"
            self.p("✓ Acquired: Royal Cushion (5/5)", "gold")
            return
        self.p("Items earned through quests!", "yellow")
    
    def talk(self, npc):
        loc = LOCATIONS[game.current_location]
        if npc not in loc["npcs"]:
            self.p("Nobody by that name here", "red")
            return
        
        if npc == "mother": self.talk_mother()
        elif npc == "beggar": self.talk_beggar()
        elif npc == "vikram": self.talk_vikram()
        elif npc == "radhagupta": self.talk_radha()
        elif npc == "priest": self.talk_priest()
        elif npc == "amma": self.talk_amma()
        elif npc == "ascetic": self.talk_ascetic()
        else: self.p(f"{npc} says: Hello", "green")
    
    def talk_mother(self):
        """Mother Dharma's motivational dialogue"""
        self.p("")
        self.p("-" * 70, "green")
        self.p("MOTHER DHARMA speaks softly:", "green")
        self.p("-" * 70, "green")
        self.p("")
        
        self.p("My beloved son... come, sit with me.")
        self.p("")
        self.p("I know what troubles your heart. Your brothers mock you,")
        self.p("your father ignores you, the court whispers behind your")
        self.p("back. They see only your rough skin and think you unworthy")
        self.p("of the throne.")
        self.p("")
        self.p("But I have seen your destiny, Ashoka. In my dreams, I see", "gold")
        self.p("you not just as emperor, but as something far greater -", "gold")
        self.p("a light that will illuminate all of Jambudvipa.", "gold")
        self.p("")
        self.p("The ascetic Pingala-vatsajiva has been summoned by your")
        self.p("father to choose the next heir. He will test all of you.")
        self.p("This is your chance, my son.")
        self.p("")
        self.p("Listen carefully - the test will not be one of strength")
        self.p("or royal bearing. It will test the qualities that truly")
        self.p("make a great ruler:")
        self.p("")
        self.p("  • COMPASSION for the suffering", "yellow")
        self.p("  • WISDOM to see truth", "yellow")
        self.p("  • HUMILITY to learn from others", "yellow")
        self.p("  • COURAGE to face danger", "yellow")
        self.p("  • STRENGTH tempered with understanding", "yellow")
        self.p("")
        self.p("Go to the Garden of the Golden Pavilion. Face whatever")
        self.p("test the ascetic gives you. Show them all what I have")
        self.p("always known - that you have the heart of a true king.")
        self.p("")
        self.p("Remember what I've taught you:", "magenta")
        self.p("")
        self.p("  True power does not crush the weak - it protects them.", "cyan")
        self.p("")
        self.p("  A king who rules through fear is forgotten.", "cyan")
        self.p("  A king who rules with dharma is remembered forever.", "cyan")
        self.p("")
        self.p("Your father may never see it. Your brothers may never")
        self.p("accept it. But I believe in you, Ashoka.")
        self.p("")
        self.p("Now go. Make me proud.", "gold")
        self.p("")
        self.p("-" * 70, "green")
        self.p("")
        
        # Add a hint about where to go
        if not game.riddle_solved:
            self.p("💡 TIP: The ascetic awaits in the GARDEN (west)", "yellow")
        elif len(game.inventory) == 0:
            self.p("💡 TIP: Begin your quest - explore the palace!", "yellow")
        elif len(game.inventory) < 5:
            self.p(f"💡 TIP: You've collected {len(game.inventory)}/5 items. Keep going!", "yellow")
        else:
            self.p("💡 TIP: Return to the Garden - you're ready!", "yellow")
    
    def talk_beggar(self):
        q = game.quests["beggar"]
        if q["stage"] == 0:
            self.p("DHRUVA:", "green")
            self.p("Vikram owes me 20 gold. Help me get it?")
            self.p("(Try: 'talk to vikram' then 'help vikram')", "yellow")
            q["status"] = "in_progress"
            q["stage"] = 1
        elif q["stage"] == 1:
            self.p("DHRUVA: Spoken to Vikram yet?")
        elif q["stage"] == 2 and game.flags["vikram_paid"]:
            self.p("DHRUVA: Thank you! Take my bowl.")
            game.add_item("golden_bowl")
            q["status"] = "completed"
            q["stage"] = 3
            self.p("✓ Quest Complete! Golden Bowl (1/5)", "gold")
        else:
            self.p("DHRUVA: Thanks again.")
    
    def talk_vikram(self):
        if game.quests["beggar"]["stage"] < 1:
            self.p("VIKRAM: I'm busy.")
            return
        self.p("VIKRAM: I owe him nothing!")
        self.p("(Try: 'help vikram')", "yellow")
    
    def help_vikram(self):
        if game.quests["beggar"]["stage"] != 1:
            return
        self.p("You check his ledgers... Found it!")
        self.p("VIKRAM: Fine, take it!")
        game.flags["vikram_paid"] = True
        game.quests["beggar"]["stage"] = 2
        self.p("Return to beggar", "yellow")
    
    def talk_radha(self):
        self.p("RADHAGUPTA:", "green")
        self.p("Test yourself with Airavata?")
        if not game.combat_training_complete:
            self.p("")
            self.p("Or would you like combat training?", "yellow")
            self.p("(Try: 'test elephant' or 'train combat')", "yellow")
        else:
            self.p("(Try: 'test elephant')", "yellow")
    
    def test_elephant(self):
        if game.current_location != "royal_stables":
            return
        if game.quests["elephant"]["status"] == "completed":
            self.p("Already done", "yellow")
            return
        game.quests["elephant"]["status"] = "in_progress"
        game.flags["elephant_started"] = True
        self.p("PART 1: Feed Airavata")
        self.p("Options: hay, mangoes, sugarcane")
        self.p("(Try: 'feed hay')", "yellow")
    
    def feed(self, food):
        if not game.flags["elephant_started"] or game.flags["fed_elephant"]:
            return
        if food == "hay":
            self.p("Airavata accepts gratefully")
            game.flags["fed_elephant"] = True
            self.p("PART 2: Remove thorn", "yellow")
            self.p("(Try: 'remove thorn')", "yellow")
        else:
            self.p("Wrong choice", "red")
    
    def remove_thorn(self):
        if not game.flags["fed_elephant"] or game.flags["removed_thorn"]:
            return
        self.p("You carefully remove the thorn...")
        game.flags["removed_thorn"] = True
        self.p("PART 3: Comfort elephant", "yellow")
        self.p("(Try: 'comfort elephant')", "yellow")
    
    def comfort(self):
        if not game.flags["removed_thorn"]:
            return
        self.p("You speak of loneliness...")
        self.p("Airavata touches your face")
        game.flags["comforted_elephant"] = True
        game.add_item("imperial_elephant")
        game.quests["elephant"]["status"] = "completed"
        self.p("✓ Quest Complete! Imperial Elephant (2/5)", "gold")
    
    def talk_priest(self):
        self.p("MAHINDA:", "green")
        if not game.flags["has_golden_bowl"]:
            self.p("Need golden bowl first")
        else:
            self.p("Ready for ritual?")
            self.p("(Try: 'ritual')", "yellow")
    
    def ritual(self):
        if game.current_location != "temple_district":
            return
        if not game.flags["has_golden_bowl"]:
            return
        if game.quests["soma"]["status"] == "completed":
            return
        self.p("You complete the sacred ritual...")
        self.p("Vision: Two paths - war and peace")
        game.add_item("soma")
        game.quests["soma"]["status"] = "completed"
        self.p("✓ Quest Complete! Soma (3/5)", "gold")
    
    def talk_amma(self):
        self.p("AMMA:", "green")
        self.p("Want to learn cooking?")
        self.p("(Try: 'learn cooking')", "yellow")
    
    def learn_cook(self):
        if game.current_location != "palace_kitchen":
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        game.quests["cooking"]["status"] = "in_progress"
        game.flags["learned_steps"] = True
        self.p("AMMA: 5 steps:")
        self.p("1. Mix flour and ghee")
        self.p("2. Add honey slowly")
        self.p("3. Fold in cardamom")
        self.p("4. Shape into rounds")
        self.p("5. Bake until golden")
        self.p("")
        self.p("Repeat the order. Type: 1 2 3 4 5")
        self.input_entry.unbind('<Return>')
        self.input_entry.bind('<Return>', self.check_steps)
    
    def check_steps(self, e=None):
        ans = self.input_entry.get().strip()
        self.input_entry.delete(0, tk.END)
        correct = ["1 2 3 4 5", "12345", "1,2,3,4,5", "1, 2, 3, 4, 5"]
        if ans in correct:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Perfect!")
            game.flags["steps_correct"] = True
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
            self.p("(Try: 'make cakes')", "yellow")
            self.p("")
        else:
            self.p(f"> {ans}", "cyan")
            self.p("AMMA: Wrong! Come back.", "red")
            game.quests["cooking"]["status"] = "not_started"
            game.flags["learned_steps"] = False
            self.input_entry.unbind('<Return>')
            self.input_entry.bind('<Return>', self.process_cmd)
    
    def make_cakes(self):
        if not game.flags["steps_correct"]:
            return
        if game.quests["cooking"]["status"] == "completed":
            return
        self.p("You make honey cakes with care...")
        self.p("AMMA: Remember this humility")
        game.add_item("honey_cakes")
        game.quests["cooking"]["status"] = "completed"
        self.p("✓ Quest Complete! Honey Cakes (4/5)", "gold")
    
    def talk_ascetic(self):
        if not game.check_victory():
            self.p("ASCETIC: Return with all five items")
            return
        if game.flags["victory"]:
            return
        self.victory()
    
    def victory(self):
        game.flags["victory"] = True
        self.play_music("victory")  # Play victory music
        self.clear()
        self.load_img("coronation.png")
        self.p("")
        self.p("="*70, "gold")
        self.p("ACT 1 COMPLETE", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("You are crowned EMPEROR ASHOKA")
        self.p("")
        self.p("="*70, "gold")
        self.p("TO BE CONTINUED IN ACT 2...", "bold")
        self.p("="*70, "gold")
        self.p("")
        self.p("Thank you for playing!", "cyan")
    
    # ========== COMBAT SYSTEM ==========
    
    def train_combat(self):
        """Start optional training battle"""
        if game.current_location != "royal_stables":
            self.p("Must be at Royal Stables to train", "red")
            return
        
        if game.combat_training_complete:
            self.p("RADHAGUPTA: You've already mastered combat training.", "green")
            return
        
        self.p("RADHAGUPTA: You wish to learn combat?", "green")
        self.p("RADHAGUPTA: Very well. Defend yourself!")
        self.p("")
        self.init_combat("Training Dummy", 60, is_training=True)
    
    def start_susima_battle(self):
        """Start required Susima boss battle"""
        self.clear()
        self.load_img("fathers_chambers.png")
        self.p("")
        self.p("="*70, "red")
        self.p("⚔️  BROTHER SUSIMA BLOCKS YOUR PATH!", "red")
        self.p("="*70, "red")
        self.p("")
        self.p("SUSIMA: 'Ashoka! You dare approach the throne?'")
        self.p("SUSIMA: 'I see you've been collecting... trinkets.'")
        self.p("SUSIMA: 'Let's see if you can fight as well as you scheme!'")
        self.p("")
        self.p("Susima draws his sword!")
        self.p("")
        self.init_combat("Brother Susima", 100, is_training=False)
    
    def init_combat(self, enemy_name, enemy_hp, is_training=False):
        """Initialize combat state"""
        import random
        self.play_music("battle")  # Play battle music
        game.in_combat = True
        game.combat_enemy = enemy_name
        game.combat_enemy_hp = enemy_hp
        game.combat_enemy_max_hp = enemy_hp
        game.combat_heals_remaining = 3
        game.combat_defending = False
        game.combat_is_training = is_training
        
        self.show_combat_status()
    
    def show_combat_status(self):
        """Display combat UI"""
        import random
        
        # Player health bar
        player_bar_filled = int((game.health / game.max_health) * 10)
        player_bar = "█" * player_bar_filled + "░" * (10 - player_bar_filled)
        
        # Enemy health bar
        enemy_bar_filled = int((game.combat_enemy_hp / game.combat_enemy_max_hp) * 10)
        enemy_bar = "█" * enemy_bar_filled + "░" * (10 - enemy_bar_filled)
        
        self.p("")
        self.p("="*60, "bold")
        self.p(f"⚔️  BATTLE: You vs {game.combat_enemy}", "bold")
        self.p("="*60, "bold")
        self.p("")
        self.p(f"YOU:    {player_bar} {game.health}/{game.max_health} HP | Heals: {game.combat_heals_remaining}/3", "green")
        self.p(f"{game.combat_enemy.upper()}: {enemy_bar} {game.combat_enemy_hp}/{game.combat_enemy_max_hp} HP", "red")
        self.p("")
        self.p("Your turn! What will you do?", "yellow")
        self.p("  [1] ATTACK      - Strike (15-25 damage)", "cyan")
        self.p("  [2] DEFEND      - Guard (50% damage reduction next turn)", "cyan")
        self.p("  [3] POWER ATK   - Heavy strike (25-35 dmg, 30% miss)", "cyan")
        self.p("  [4] HEAL        - Restore 25 HP", "cyan")
        self.p("")
    
    def combat_action(self, cmd):
        """Process combat commands"""
        import random
        
        if cmd in ["1", "attack", "atk"]:
            self.player_attack()
        elif cmd in ["2", "defend", "def", "guard"]:
            self.player_defend()
        elif cmd in ["3", "power", "power attack", "power atk"]:
            self.player_power_attack()
        elif cmd in ["4", "heal"]:
            self.player_heal()
        else:
            self.p("Invalid action! Choose 1-4", "red")
            self.p("")
            return
        
        # Check if enemy dead
        if self.check_combat_end():
            return
        
        # Enemy turn
        self.enemy_turn()
        
        # Check if player dead
        if self.check_combat_end():
            return
        
        # Next round
        self.show_combat_status()
    
    def player_attack(self):
        """Player normal attack"""
        import random
        damage = random.randint(15, 25)
        game.combat_enemy_hp -= damage
        game.combat_defending = False
        
        self.p(f"> ATTACK", "cyan")
        self.p("")
        self.p(f"You strike {game.combat_enemy}!", "green")
        self.p(f"💥 Dealt {damage} damage!", "green")
        self.p("")
    
    def player_defend(self):
        """Player defend"""
        game.combat_defending = True
        
        self.p(f"> DEFEND", "cyan")
        self.p("")
        self.p("You raise your guard defensively!", "green")
        self.p("🛡️  Next attack will be reduced by 50%!", "green")
        self.p("")
    
    def player_power_attack(self):
        """Player power attack with miss chance"""
        import random
        game.combat_defending = False
        
        self.p(f"> POWER ATTACK", "cyan")
        self.p("")
        
        if random.random() < 0.3:  # 30% miss chance
            self.p("Your heavy strike misses!", "red")
            self.p("You're off balance!", "red")
            self.p("")
        else:
            damage = random.randint(25, 35)
            game.combat_enemy_hp -= damage
            self.p(f"CRITICAL HIT!", "gold")
            self.p(f"💥💥 Dealt {damage} damage!", "gold")
            self.p("")
    
    def player_heal(self):
        """Player heal"""
        game.combat_defending = False
        
        self.p(f"> HEAL", "cyan")
        self.p("")
        
        if game.combat_heals_remaining <= 0:
            self.p("No heals remaining!", "red")
            self.p("You waste your turn!", "red")
            self.p("")
            return
        
        heal_amount = 25
        old_health = game.health
        game.health = min(game.health + heal_amount, game.max_health)
        actual_heal = game.health - old_health
        game.combat_heals_remaining -= 1
        
        self.p(f"You drink a healing potion!", "green")
        self.p(f"❤️  Restored {actual_heal} HP!", "green")
        self.p("")
    
    def enemy_turn(self):
        """Enemy AI turn"""
        import random
        
        # Simple AI: Random attack or power attack
        action = random.choice(["attack", "attack", "power"])  # 66% attack, 33% power
        
        if action == "attack":
            damage = random.randint(10, 20) if game.combat_is_training else random.randint(15, 25)
            
            if game.combat_defending:
                damage = damage // 2
                self.p(f"{game.combat_enemy} attacks!", "yellow")
                self.p(f"🛡️  Your guard blocks half the damage!", "green")
                self.p(f"💔 You take {damage} damage!", "red")
            else:
                self.p(f"{game.combat_enemy} attacks!", "yellow")
                self.p(f"💔 You take {damage} damage!", "red")
            
            game.health -= damage
            game.combat_defending = False
            
        elif action == "power":
            if random.random() < 0.4:  # 40% miss for enemy power
                self.p(f"{game.combat_enemy} attempts a heavy strike but misses!", "green")
            else:
                damage = random.randint(20, 30) if game.combat_is_training else random.randint(25, 35)
                
                if game.combat_defending:
                    damage = damage // 2
                    self.p(f"{game.combat_enemy} unleashes a POWER ATTACK!", "red")
                    self.p(f"🛡️  Your guard reduces the damage!", "green")
                    self.p(f"💔💔 You take {damage} damage!", "red")
                else:
                    self.p(f"{game.combat_enemy} unleashes a POWER ATTACK!", "red")
                    self.p(f"💔💔 You take {damage} damage!", "red")
                
                game.health -= damage
            
            game.combat_defending = False
        
        self.p("")
    
    def check_combat_end(self):
        """Check if combat is over"""
        if game.combat_enemy_hp <= 0:
            # Player wins!
            self.play_music("victory")  # Victory music
            self.p("="*60, "gold")
            self.p("⭐ VICTORY! ⭐", "gold")
            self.p("="*60, "gold")
            self.p("")
            
            if game.combat_is_training:
                self.p("RADHAGUPTA: 'Excellent! You have mastered combat!'", "green")
                self.p("RADHAGUPTA: 'You are ready for real battles now.'", "green")
                game.combat_training_complete = True
            else:
                self.p("Susima falls to his knees, defeated!", "green")
                self.p("")
                self.p("SUSIMA: 'Impossible... how did you...'", "yellow")
                self.p("")
                self.p("You stand victorious. The throne room is yours.", "green")
                self.p("")
                self.p("The royal cushion sits on the throne, waiting...", "magenta")
                self.p("")
            
            game.in_combat = False
            self.p("")
            
            # Return to exploration music after a delay
            self.root.after(3000, lambda: self.play_music("exploration"))
            
            if not game.combat_is_training:
                self.show_loc()
            
            return True
        
        if game.health <= 0:
            # Player dies!
            self.stop_music()  # Stop music on death
            self.p("="*60, "red")
            self.p("💀 DEFEAT 💀", "red")
            self.p("="*60, "red")
            self.p("")
            
            if game.combat_is_training:
                self.p("RADHAGUPTA: 'You fought well, but need more training.'", "yellow")
                self.p("RADHAGUPTA: 'Rest and try again when ready.'", "yellow")
                game.health = game.max_health  # Restore health after training
                game.in_combat = False
                self.play_music("exploration")  # Back to exploration
            else:
                self.p("Susima strikes you down!", "red")
                self.p("")
                self.p("YOU DIED", "red")
                self.p("")
                self.p("Restarting in 5 seconds...", "yellow")
                game.in_combat = False
                self.input_entry.config(state='disabled')
                self.root.after(5000, self.restart)
            
            self.p("")
            return True
        
        return False
    
    # ========== END COMBAT SYSTEM ==========
    
    # ========== TUTORIAL SYSTEM ==========
    
    def show_tutorial(self):
        """Display comprehensive tutorial"""
        self.clear()
        self.p("="*70, "bold")
        self.p("📖 ASHOKA'S PATH - COMPLETE TUTORIAL", "bold")
        self.p("="*70, "bold")
        self.p("")
        
        self.p("🎯 OBJECTIVE:", "gold")
        self.p("Collect FIVE sacred items to prove your worth as heir to the")
        self.p("Mauryan throne. Each item tests a different virtue.")
        self.p("")
        
        self.p("="*70, "cyan")
        self.p("MOVEMENT COMMANDS", "cyan")
        self.p("="*70, "cyan")
        self.p("  north, n       - Move north")
        self.p("  south, s       - Move south")
        self.p("  east, e        - Move east")
        self.p("  west, w        - Move west")
        self.p("")
        self.p("  look, l        - Re-examine current location")
        self.p("  map, m         - View palace map with your position")
        self.p("")
        
        self.p("="*70, "green")
        self.p("INFORMATION COMMANDS", "green")
        self.p("="*70, "green")
        self.p("  inventory, i   - Check items you've collected (X/5)")
        self.p("  quest, q       - View quest progress and active quests")
        self.p("  health, h      - Check your current health (combat)")
        self.p("  lore, history  - Read historical facts about location")
        self.p("  tutorial, tut  - Show this tutorial anytime")
        self.p("  help, ?        - Quick command reference")
        self.p("")
        
        self.p("="*70, "yellow")
        self.p("INTERACTION COMMANDS", "yellow")
        self.p("="*70, "yellow")
        self.p("  talk to <npc>  - Speak with characters")
        self.p("                   Example: 'talk to beggar'")
        self.p("")
        self.p("  pick <item>    - Pick up items (when available)")
        self.p("  get <item>     - Same as pick")
        self.p("  take <item>    - Same as pick")
        self.p("")
        
        self.p("="*70, "magenta")
        self.p("QUEST-SPECIFIC COMMANDS", "magenta")
        self.p("="*70, "magenta")
        self.p("Golden Bowl Quest (Marketplace):")
        self.p("  help vikram    - Investigate merchant's ledgers")
        self.p("")
        self.p("Elephant Quest (Royal Stables):")
        self.p("  test elephant  - Begin Airavata's worthiness test")
        self.p("  feed <food>    - Feed elephant (hay/mangoes/sugarcane)")
        self.p("  remove thorn   - Help elephant with thorn")
        self.p("  comfort elephant - Show compassion to Airavata")
        self.p("")
        self.p("Soma Quest (Temple District):")
        self.p("  ritual         - Begin sacred soma ritual")
        self.p("                   (Requires golden bowl first!)")
        self.p("")
        self.p("Cooking Quest (Palace Kitchen):")
        self.p("  learn cooking  - Learn honey cake recipe")
        self.p("  make cakes     - Cook the honey cakes")
        self.p("")
        self.p("Cushion Quest (Throne Room):")
        self.p("  pick cushion   - Take royal cushion")
        self.p("                   (Need 4+ items or death!)")
        self.p("")
        
        self.p("="*70, "red")
        self.p("COMBAT COMMANDS", "red")
        self.p("="*70, "red")
        self.p("  train combat   - Optional training at Royal Stables")
        self.p("                   (No death penalty, learn safely)")
        self.p("")
        self.p("During Battle (choose 1-4):")
        self.p("  1, attack      - Normal attack (15-25 damage)")
        self.p("  2, defend      - Guard (50% damage reduction)")
        self.p("  3, power       - Power attack (25-35 dmg, 30% miss)")
        self.p("  4, heal        - Restore 25 HP (3 uses per battle)")
        self.p("")
        
        self.p("="*70, "cyan")
        self.p("AUDIO COMMANDS", "cyan")
        self.p("="*70, "cyan")
        self.p("  music          - Toggle music on/off")
        self.p("  volume <0-100> - Set volume (0=mute, 100=max)")
        self.p("                   Example: 'volume 50'")
        self.p("")
        
        self.p("="*70, "gold")
        self.p("💡 IMPORTANT TIPS", "gold")
        self.p("="*70, "gold")
        self.p("")
        self.p("📍 QUEST ORDER:", "yellow")
        self.p("  1. Golden Bowl (beggar) - Unlocks soma ritual")
        self.p("  2. Imperial Elephant (stables) - Test of compassion")
        self.p("  3. Honey Cakes (kitchen) - Test of humility")
        self.p("  4. Soma (temple) - Needs bowl, vision quest")
        self.p("  5. Royal Cushion (throne) - Needs 4+ items!")
        self.p("")
        
        self.p("⚠️  DANGER ZONES:", "red")
        self.p("  • Throne Room with <4 items = INSTANT DEATH")
        self.p("  • Throne Room with 4+ items = BOSS BATTLE")
        self.p("  • Losing to Susima = GAME RESTART")
        self.p("  • Training battle = Safe practice (no penalty)")
        self.p("")
        
        self.p("✨ STRATEGY:", "green")
        self.p("  • Talk to everyone - NPCs give quest hints")
        self.p("  • Read lore at locations for immersion")
        self.p("  • Do training battle before Susima fight")
        self.p("  • Collect 4 items before throne room")
        self.p("  • Save heals for emergencies in combat")
        self.p("  • Return to Garden after all 5 items")
        self.p("")
        
        self.p("="*70, "bold")
        self.p("Press any key to continue...", "yellow")
        self.p("="*70, "bold")
        
        game.tutorial_shown = True
    
    def show_lore(self):
        """Display historical lore for current location"""
        location = LOCATIONS[game.current_location]
        
        if "lore" not in location:
            self.p("No historical information available here.", "yellow")
            return
        
        self.p("")
        self.p("="*70, "gold")
        self.p("📜 HISTORICAL LORE", "gold")
        self.p("="*70, "gold")
        self.p("")
        
        # Print lore with proper formatting
        lore_lines = location["lore"].split("\n")
        for line in lore_lines:
            if line.strip():
                self.p(line, "cyan" if line.startswith("HISTORICAL") else None)
            else:
                self.p("")
        
        self.p("")
        self.p("="*70, "gold")
    
    # ========== END TUTORIAL SYSTEM ==========
    
    def show_inv(self):
        if not game.inventory:
            self.p("Empty", "yellow")
        else:
            self.p("📦 INVENTORY:", "bold")
            for i, item in enumerate(game.inventory, 1):
                self.p(f"  {i}. {item.replace('_', ' ').title()}", "green")
            self.p(f"{len(game.inventory)}/5 items", "yellow")
    
    def show_quest(self):
        self.p("📜 QUESTS:", "bold")
        for name, q in game.quests.items():
            s = "✓" if q["status"] == "completed" else "◆" if q["status"] == "in_progress" else "○"
            c = "green" if q["status"] == "completed" else "yellow" if q["status"] == "in_progress" else "cyan"
            self.p(f"{s} {name}", c)
    
    def show_map(self):
        """Display game map with current location highlighted"""
        # Load map image if available
        self.load_img("map.png")
        
        self.p("")
        self.p("="*60, "bold")
        self.p("🗺️  PATALIPUTRA PALACE MAP", "bold")
        self.p("="*60, "bold")
        self.p("")
        
        # Create map with current location highlighted
        current = game.current_location
        
        # Map structure
        locations_map = {
            "fathers_chambers": "    [Throne Room]",
            "palace_courtyard_n": "          |",
            "palace_courtyard": "  [Palace Courtyard]",
            "mothers_chambers": "          |",
            "palace_kitchen": "[Kitchen]--+",
            "garden_pavilion": "[Garden]--+--[Stables]",
            "marketplace": "              |",
            "temple_district": "     [Temple]--[Market]"
        }
        
        # Print map with highlights
        self.p("         NORTH", "cyan")
        self.p("")
        
        # Throne Room
        if current == "fathers_chambers":
            self.p("    >>> [THRONE ROOM] <<<", "gold")
        else:
            self.p("        [Throne Room]", "cyan")
        
        self.p("              |", "cyan")
        
        # Palace Courtyard (center)
        if current == "palace_courtyard":
            self.p("                            ", "cyan")
            self.p("  [Garden]------>>> [YOU ARE HERE] <<<------[Stables]", "gold")
            self.p("                            ", "cyan")
        else:
            self.p("                            ", "cyan")
            self.p("  [Garden]--------[Courtyard]--------[Stables]", "cyan")
            self.p("                            ", "cyan")
        
        self.p("                      |                    |", "cyan")
        
        # Bottom row
        left = "              " if current != "garden_pavilion" else ">>> "
        middle_m = " [Mother's] " if current != "mothers_chambers" else " >>> [YOU] <<< "
        middle_k = "[Kitchen]" if current != "palace_kitchen" else ">>>[YOU]<<<"
        right_m = "[Market]" if current != "marketplace" else ">>>[YOU]<<<"
        right_t = "[Temple]" if current != "temple_district" else ">>>[YOU]<<<"
        right_s = " " if current != "royal_stables" else ""
        
        self.p(f"{left}          {middle_m}            {right_s}", "cyan" if current not in ["mothers_chambers", "royal_stables"] else "gold")
        self.p(f"                      |", "cyan")
        self.p(f"                 {middle_k}", "cyan" if current != "palace_kitchen" else "gold")
        self.p("")
        self.p(f"                                   {right_t}---{right_m}", "cyan" if current not in ["marketplace", "temple_district"] else "gold")
        
        self.p("")
        self.p(f"Current Location: {LOCATIONS[current]['name']}", "yellow")
        self.p("")
        self.p("="*60, "bold")
        self.p("")
    
    def show_help(self):
        self.p("COMMANDS:", "bold")
        self.p("Movement: n/s/e/w, look, map")
        self.p("Info: inventory, quest, lore, tutorial")
        self.p("Actions: talk to <npc>, pick <item>")
        self.p("Combat: train combat (at stables)")
        self.p("Audio: music (toggle), volume <0-100>")
        self.p("Quests: help vikram, test elephant, feed <food>,")
        self.p("        remove thorn, comfort elephant, ritual,")
        self.p("        learn cooking, make cakes")
        self.p("")
        self.p("Type 'tutorial' for detailed guide!", "yellow")
        self.p("Type 'lore' for historical facts!", "yellow")

def main():
    root = tk.Tk()
    app = AshokaGame(root)
    root.mainloop()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
